In [1]:
import shutil

# Update files, Update Type between Preliminary and Final and update Month.   Check to make sure there aren't too many weeks in the month for the "Time Total by Week" report and Run.  Update  "PTO_Accrual_Balances_After_Drawdowns" File to be the starting place for the PTO calculation

In [2]:
import datetime
targetmonth = datetime.datetime(2024, 9, 1)

In [3]:
import pandas as pd
import numpy as np
import openpyxl
from openpyxl import load_workbook
from openpyxl.styles import Font
from openpyxl.styles import Alignment
from openpyxl.styles.borders import Border, Side

In [4]:
currentmonthandyear = '9/2024'

In [5]:
timestampreport = targetmonth.strftime("%b-%Y")

In [6]:
timestampreport

'Sep-2024'

In [7]:
#Type = 'PRELIMINARY'

In [8]:
Type = 'FINAL'

In [9]:
filename = ' - Program Time Report - CONFIDENTIAL - '+Type+'.xlsx'

In [10]:
from dateutil import relativedelta

from pandas.tseries.offsets import MonthEnd
from pandas.tseries.offsets import MonthBegin


targetmonthend = pd.to_datetime(targetmonth, format="%Y%m") + relativedelta.relativedelta(months=1, day=1)
targetmonthendcalc = targetmonthend.date()
targetmonthendcalc

datetime.date(2024, 10, 1)

In [11]:
import dateutil.relativedelta

beginningmonth = targetmonth - dateutil.relativedelta.relativedelta(months=1)

targetmonthbeginning = pd.to_datetime(beginningmonth, format="%Y%m") + MonthBegin(1)

targetmonthbeginningcalc = targetmonthbeginning.date()

targetmonthbeginningcalc


datetime.date(2024, 9, 1)

In [12]:
import datetime

def last_day_of_month(any_day):
    # The day 28 exists in every month. 4 days later, it's always next month
    next_month = any_day.replace(day=28) + datetime.timedelta(days=4)
    # subtracting the number of the current day brings us back one month
    return next_month - datetime.timedelta(days=next_month.day)


MonthTemplateName = targetmonthbeginningcalc.strftime("%B")

TemplateDate1 = last_day_of_month(targetmonthbeginningcalc).strftime('%Y-%m-%d')

MonthTemplateName
TemplateDate1

'2024-09-30'

In [13]:
yeartemplate = targetmonthbeginningcalc.year

In [14]:
yeartemplate

2024

In [15]:
targetmonthbeginningcalc

datetime.date(2024, 9, 1)

In [16]:
targetmonthendcalc

datetime.date(2024, 10, 1)

In [17]:
AvailableDays = np.busday_count(targetmonthbeginningcalc, targetmonthendcalc)

In [18]:
AvailableDays = AvailableDays

In [19]:
AvailableDays

21

In [20]:
def returndaysfromhire(x):
    A = x.date()
    if (AvailableDays >= np.busday_count(A, targetmonthendcalc)) and (np.busday_count(A, targetmonthendcalc)) > 0:
        return(np.busday_count(A, targetmonthendcalc))
    else:
        return(0)

In [21]:
def returndaysfromtermination(x):
    B = x.date()
    if AvailableDays >= np.busday_count(targetmonthbeginningcalc, B) and np.busday_count(targetmonthbeginningcalc, B) > 0:
        return(np.busday_count(targetmonthbeginningcalc, B))
    else:
        return(0)

In [22]:
USChinaExchange = 0.1377230631592459

In [23]:
AvailableHours = AvailableDays*8


# Load Time Data

In [24]:
#TimeDataALL = './Source_Files/RMI_WorkerTimeBlocksforFinancialAllocationsDetailedforFinanceMASTER.xlsx'

TimeDataALL = './Source_Files/RMI_WorkerTimeBlocksforFinancialAllocationsDetailedforFinance_Sep2024FINAL.xlsx'

TimeDataDFALL = pd.read_excel(TimeDataALL, header=8, engine='openpyxl')

In [25]:
TimeDataDFALL = TimeDataDFALL.rename(columns={'Project Talent Pool': 'Talent Pool'})

In [26]:
len(TimeDataDFALL)

33112

In [27]:
TimeDataDFALL.head()

,Worker,Employee ID,Worker Cost Center,Talent Pool,Project Group,Utilization Target,Hire Date,Termination Date,Title,Hourly Rate,...,Project,Project Plan Task,Project Plan Task ID,Cost Center,Initiative,Lobbying,Lobbying Location,Reported Quantity,Time Cost Center,Time Initiative
0,Aaron Brickman,1436,US,Talent Pool: Sr. Principal,NaN,0.6,2022-02-28,NaT,Senior Principal,102.53,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN
1,Aaron Brickman,1436,US,Talent Pool: Sr. Principal,Grant Funded,0.6,2022-02-28,NaT,Senior Principal,102.53,...,BuildUS<>ACRE,General > Stakeholder engagement,PROJECT_PLAN_TASK-3-5841,US,US - Clean Regional Economic Development,NaN,NaN,4.0,US,US - Clean Regional Economic Development
2,Aaron Brickman,1436,US,Talent Pool: Sr. Principal,Grant Funded,0.6,2022-02-28,NaT,Senior Principal,102.53,...,BuildUS<>ACRE,General > Stakeholder engagement,PROJECT_PLAN_TASK-3-5841,US,US - Clean Regional Economic Development,NaN,NaN,4.0,US,US - Clean Regional Economic Development
3,Aaron Brickman,1436,US,Talent Pool: Sr. Principal,Grant Funded,0.6,2022-02-28,NaT,Senior Principal,102.53,...,BuildUS<>ACRE,General > Stakeholder engagement,PROJECT_PLAN_TASK-3-5841,US,US - Clean Regional Economic Development,NaN,NaN,4.0,US,US - Clean Regional Economic Development
4,Aaron Brickman,1436,US,Talent Pool: Sr. Principal,Grant Funded,0.6,2022-02-28,NaT,Senior Principal,102.53,...,BuildUS<>ACRE,General > Stakeholder engagement,PROJECT_PLAN_TASK-3-5841,US,US - Clean Regional Economic Development,NaN,NaN,4.0,US,US - Clean Regional Economic Development


In [28]:
TimeDataDFALL['Lobbying'].fillna("None", inplace = True)

In [29]:
TimeDataDFALL['Lobbying Location'].fillna("None", inplace = True)

# Manipulate Data

In [30]:
#TimeDataDFALL.loc[(TimeDataDFALL['Project'] == 'Influence Team [Admin]'), 'Project Group'] = 'Admin'
#TimeDataDFALL.loc[(TimeDataDFALL['Project'] == 'Program Services [Admin]'), 'Project Group'] = 'Admin'


In [31]:
TimeDataDFALL.loc[(TimeDataDFALL['Talent Pool'] == 'Talent Pool: Intern'), 'Utilization Target'] = .50


In [32]:
TimeDataDFALL.head()

,Worker,Employee ID,Worker Cost Center,Talent Pool,Project Group,Utilization Target,Hire Date,Termination Date,Title,Hourly Rate,...,Project,Project Plan Task,Project Plan Task ID,Cost Center,Initiative,Lobbying,Lobbying Location,Reported Quantity,Time Cost Center,Time Initiative
0,Aaron Brickman,1436,US,Talent Pool: Sr. Principal,NaN,0.6,2022-02-28,NaT,Senior Principal,102.53,...,NaN,NaN,NaN,NaN,NaN,None,None,8.0,NaN,NaN
1,Aaron Brickman,1436,US,Talent Pool: Sr. Principal,Grant Funded,0.6,2022-02-28,NaT,Senior Principal,102.53,...,BuildUS<>ACRE,General > Stakeholder engagement,PROJECT_PLAN_TASK-3-5841,US,US - Clean Regional Economic Development,None,None,4.0,US,US - Clean Regional Economic Development
2,Aaron Brickman,1436,US,Talent Pool: Sr. Principal,Grant Funded,0.6,2022-02-28,NaT,Senior Principal,102.53,...,BuildUS<>ACRE,General > Stakeholder engagement,PROJECT_PLAN_TASK-3-5841,US,US - Clean Regional Economic Development,None,None,4.0,US,US - Clean Regional Economic Development
3,Aaron Brickman,1436,US,Talent Pool: Sr. Principal,Grant Funded,0.6,2022-02-28,NaT,Senior Principal,102.53,...,BuildUS<>ACRE,General > Stakeholder engagement,PROJECT_PLAN_TASK-3-5841,US,US - Clean Regional Economic Development,None,None,4.0,US,US - Clean Regional Economic Development
4,Aaron Brickman,1436,US,Talent Pool: Sr. Principal,Grant Funded,0.6,2022-02-28,NaT,Senior Principal,102.53,...,BuildUS<>ACRE,General > Stakeholder engagement,PROJECT_PLAN_TASK-3-5841,US,US - Clean Regional Economic Development,None,None,4.0,US,US - Clean Regional Economic Development


In [33]:
len(TimeDataDFALL)

33112

In [34]:
TimeDataDFALL.dtypes

Worker                          object
Employee ID                      int64
Worker Cost Center              object
Talent Pool                     object
Project Group                   object
Utilization Target             float64
Hire Date               datetime64[ns]
Termination Date        datetime64[ns]
Title                           object
Hourly Rate                    float64
Monthly Salary                 float64
Monthly Bonus Amount           float64
Bonus %                        float64
Week                            object
Reported Date           datetime64[ns]
Time Type                       object
Entry Type                      object
Project                         object
Project Plan Task               object
Project Plan Task ID            object
Cost Center                     object
Initiative                      object
Lobbying                        object
Lobbying Location               object
Reported Quantity              float64
Time Cost Center         

Categorize PTO and Holiday

In [35]:
TimeDataDFALL.loc[(TimeDataDFALL['Time Type'] == 'Time Off'), 'Project Group'] = 'PTO'
TimeDataDFALL.loc[(TimeDataDFALL['Entry Type'] == 'Holiday Hours'), 'Project Group'] = 'Holiday'
TimeDataDFALL.loc[(TimeDataDFALL['Time Type'] == 'Time Off'), 'Project'] = 'PTO'
TimeDataDFALL.loc[(TimeDataDFALL['Entry Type'] == 'Holiday Hours'), 'Project'] = 'Holiday'
TimeDataDFALL.loc[(TimeDataDFALL['Time Type'] == 'Time Off'), 'Project Plan Task'] = 'PTO'
TimeDataDFALL.loc[(TimeDataDFALL['Entry Type'] == 'Holiday Hours'), 'Project Plan Task'] = 'Holiday'

In [36]:
TimeDataDFALL['MonthYear'] = pd.DatetimeIndex(TimeDataDFALL['Reported Date']).month.astype(str) + "/" + pd.DatetimeIndex(TimeDataDFALL['Reported Date']).year.astype(str)


In [37]:
TimeDataDFALL['MonthYear'] = TimeDataDFALL['MonthYear'].apply(lambda x: x.replace('.0',''))

## SELECT MONTH THAT WE WILL BE DOING TIME CODING FOR

In [38]:
TimeDataDF = TimeDataDFALL[TimeDataDFALL['MonthYear']==currentmonthandyear]

In [39]:
TimeDataDFALL.columns

Index(['Worker', 'Employee ID', 'Worker Cost Center', 'Talent Pool',
       'Project Group', 'Utilization Target', 'Hire Date', 'Termination Date',
       'Title', 'Hourly Rate', 'Monthly Salary', 'Monthly Bonus Amount',
       'Bonus %', 'Week', 'Reported Date', 'Time Type', 'Entry Type',
       'Project', 'Project Plan Task', 'Project Plan Task ID', 'Cost Center',
       'Initiative', 'Lobbying', 'Lobbying Location', 'Reported Quantity',
       'Time Cost Center', 'Time Initiative', 'MonthYear'],
      dtype='object')

In [40]:
TimeDataDF

,Worker,Employee ID,Worker Cost Center,Talent Pool,Project Group,Utilization Target,Hire Date,Termination Date,Title,Hourly Rate,...,Project Plan Task,Project Plan Task ID,Cost Center,Initiative,Lobbying,Lobbying Location,Reported Quantity,Time Cost Center,Time Initiative,MonthYear
0,Aaron Brickman,1436,US,Talent Pool: Sr. Principal,Holiday,0.6,2022-02-28,NaT,Senior Principal,102.53,...,Holiday,NaN,NaN,NaN,None,None,8.0,NaN,NaN,9/2024
1,Aaron Brickman,1436,US,Talent Pool: Sr. Principal,Grant Funded,0.6,2022-02-28,NaT,Senior Principal,102.53,...,General > Stakeholder engagement,PROJECT_PLAN_TASK-3-5841,US,US - Clean Regional Economic Development,None,None,4.0,US,US - Clean Regional Economic Development,9/2024
2,Aaron Brickman,1436,US,Talent Pool: Sr. Principal,Grant Funded,0.6,2022-02-28,NaT,Senior Principal,102.53,...,General > Stakeholder engagement,PROJECT_PLAN_TASK-3-5841,US,US - Clean Regional Economic Development,None,None,4.0,US,US - Clean Regional Economic Development,9/2024
3,Aaron Brickman,1436,US,Talent Pool: Sr. Principal,Grant Funded,0.6,2022-02-28,NaT,Senior Principal,102.53,...,General > Stakeholder engagement,PROJECT_PLAN_TASK-3-5841,US,US - Clean Regional Economic Development,None,None,4.0,US,US - Clean Regional Economic Development,9/2024
4,Aaron Brickman,1436,US,Talent Pool: Sr. Principal,Grant Funded,0.6,2022-02-28,NaT,Senior Principal,102.53,...,General > Stakeholder engagement,PROJECT_PLAN_TASK-3-5841,US,US - Clean Regional Economic Development,None,None,4.0,US,US - Clean Regional Economic Development,9/2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33107,Zsaria Diaz,1327,Islands,Talent Pool: Sr. Associate,Admin,0.7,2021-09-13,NaT,Senior Associate,44.30,...,Admin > Internal Meetings,PROJECT_PLAN_TASK-3-2452,Islands,IS - Admin,None,None,0.5,Islands,IS - Admin,9/2024
33108,Zsaria Diaz,1327,Islands,Talent Pool: Sr. Associate,Admin,0.7,2021-09-13,NaT,Senior Associate,44.30,...,Admin > Internal Meetings,PROJECT_PLAN_TASK-3-2452,Islands,IS - Admin,None,None,2.5,Islands,IS - Admin,9/2024
33109,Zsaria Diaz,1327,Islands,Talent Pool: Sr. Associate,Grant Funded,0.7,2021-09-13,NaT,Senior Associate,44.30,...,Project Preparation > Grenada,PROJECT_PLAN_TASK-3-6379,Islands,IS - Islands,None,None,1.0,Islands,IS - Islands,9/2024
33110,Zsaria Diaz,1327,Islands,Talent Pool: Sr. Associate,Admin,0.7,2021-09-13,NaT,Senior Associate,44.30,...,Admin > General,PROJECT_PLAN_TASK-3-2451,Islands,IS - Admin,None,None,6.0,Islands,IS - Admin,9/2024


# Create Time Project Data for Current Month

Create Master File for Program Financial Profile

In [41]:
MappingPath = './Source_Files/MappingSalaryCosting.xlsx'

ProjectsMaps = pd.read_excel(MappingPath, sheet_name = 'AdminvsProject', engine='openpyxl')

TimeDataDFALL = pd.merge(TimeDataDFALL, ProjectsMaps, on = 'Project Group', how = 'left')

In [42]:
TimeDataDFALL.columns

Index(['Worker', 'Employee ID', 'Worker Cost Center', 'Talent Pool',
       'Project Group', 'Utilization Target', 'Hire Date', 'Termination Date',
       'Title', 'Hourly Rate', 'Monthly Salary', 'Monthly Bonus Amount',
       'Bonus %', 'Week', 'Reported Date', 'Time Type', 'Entry Type',
       'Project', 'Project Plan Task', 'Project Plan Task ID', 'Cost Center',
       'Initiative', 'Lobbying', 'Lobbying Location', 'Reported Quantity',
       'Time Cost Center', 'Time Initiative', 'MonthYear', 'Project or Admin'],
      dtype='object')

In [43]:
TimeDataDFALL['Employee Hours'] = TimeDataDFALL.groupby(['Worker','MonthYear'])['Reported Quantity'].transform('sum')

In [44]:
TimeDataDFALL['Percent of Total Monthly Hours'] = TimeDataDFALL['Reported Quantity'] / TimeDataDFALL['Employee Hours']

In [45]:
TimeDataDFALL.columns

Index(['Worker', 'Employee ID', 'Worker Cost Center', 'Talent Pool',
       'Project Group', 'Utilization Target', 'Hire Date', 'Termination Date',
       'Title', 'Hourly Rate', 'Monthly Salary', 'Monthly Bonus Amount',
       'Bonus %', 'Week', 'Reported Date', 'Time Type', 'Entry Type',
       'Project', 'Project Plan Task', 'Project Plan Task ID', 'Cost Center',
       'Initiative', 'Lobbying', 'Lobbying Location', 'Reported Quantity',
       'Time Cost Center', 'Time Initiative', 'MonthYear', 'Project or Admin',
       'Employee Hours', 'Percent of Total Monthly Hours'],
      dtype='object')

In [46]:
TimeDataDFALL = TimeDataDFALL[['Worker', 'Employee ID', 'Worker Cost Center', 'Talent Pool',
       'Project Group', 'Utilization Target', 'Hire Date', 'Termination Date',
       'Title', 'Reported Date', 'Time Type', 'Entry Type',
       'Project', 'Project Plan Task', 'Project Plan Task ID', 'Cost Center',
       'Initiative', 'Reported Quantity', 'Week',
       'Time Cost Center', 'Time Initiative', 'MonthYear', 'Project or Admin',
       'Employee Hours', 'Percent of Total Monthly Hours']]

Print this code to see all historic Workday Data with Mavenlink data

# Load All Bonus Data

In [47]:
ALLRMIBonuses = './Source_Files/RMI_-_Bonus_Report_for_Finance (1).xlsx'

ALLRMISalariesBonuses = pd.read_excel(ALLRMIBonuses, engine='openpyxl')


ALLRMISalariesBonuses = ALLRMISalariesBonuses[ALLRMISalariesBonuses['Currency']!='INR']

In [48]:
ALLRMISalariesBonuses = ALLRMISalariesBonuses[ALLRMISalariesBonuses['Currency']!='INR']

In [49]:
len(ALLRMISalariesBonuses)

701

In [50]:
ALLRMISalariesBonuses['Hire Date Reformat'] = pd.to_datetime(ALLRMISalariesBonuses['Hire Date'], format='%m/%d/%Y')
cutoff_date = pd.to_datetime(TemplateDate1)

# Filter the DataFrame
ALLRMISalariesBonuses = ALLRMISalariesBonuses[ALLRMISalariesBonuses['Hire Date Reformat'] <= TemplateDate1]

In [51]:
len(ALLRMISalariesBonuses)

695

In [52]:
ALLRMISalariesBonuses.head()

,Company,Worker,Employee ID,Hire Date,Total Base Pay Annualized - Amount,Currency,Position,Bonus %,Bonus Annual,Bonus Monthly,Hire Date Reformat
0,Rocky Mountain Institute,Ting Li,1,2015-03-01,2109050.18,CNY,"Managing Director, RMI China",0.25,527262.545,43938.545417,2015-03-01
1,Rocky Mountain Institute,Li Ma | Cynthia Ma,4,2015-10-08,808995.65,CNY,Sr. Operations Manager - China,0.10,80899.565,6741.630417,2015-10-08
2,Rocky Mountain Institute,Zhe Wang,5,2015-10-20,1088591.61,CNY,Principal,0.20,217718.322,18143.193500,2015-10-20
3,Rocky Mountain Institute,Shutong Lu | Lucy Lu,6,2016-04-07,870315.68,CNY,Manager,0.10,87031.568,7252.630667,2016-04-07
4,Rocky Mountain Institute,Dan Li | Danielle Li,7,2016-10-18,791812.50,CNY,Sr Program Marketing Manager,0.10,79181.250,6598.437500,2016-10-18


In [53]:
ALLRMISalariesBonuses['Bonus Final'] = np.where(ALLRMISalariesBonuses['Currency']=='CNY',USChinaExchange*ALLRMISalariesBonuses['Bonus Monthly'] , ALLRMISalariesBonuses['Bonus Monthly'])

ALLRMISalariesBonuses = ALLRMISalariesBonuses[['Employee ID', 'Bonus Final']]

In [54]:
ALLRMISalariesBonuses['Bonus Final'].sum()

781074.5242969724

# Load All Employee Counts and Update Available days for Terminations and Hires

This file can't be pulled on a specific date.  I'm currently using it for Home Cost Centers, but I am using the number of Salaries paid to count salaries, and just using this file for a lookup table.

In [55]:
#click "Terminated" and leave other fields blank

ALLRMISalaries = './Source_Files/RMI_-_EE_Report_for_Accounting (7).xlsx'

SalaryCATS = pd.read_excel(ALLRMISalaries, engine='openpyxl')

In [56]:
len(SalaryCATS)

1111

In [57]:
SalaryCATS

,Employee Name,Employee ID,Company,Cost Center,Work Email,Employment Status,Worker Type,Employee Type,Contingent Worker Type,Time Type,Location,Hire Date,Termination Date,Title,Exempt/Non-Exempt,Annual Base (USD),Worker Utilization Target,Talent Pools
0,Ting Li,1,Rocky Mountain Institute,China,NaN,Active,Employee,Regular,NaN,Full time,Beijing,2015-03-01,NaT,"Managing Director, RMI China",Exempt,298756.86,0.55,Managing Director
1,Li Ma | Cynthia Ma,4,Rocky Mountain Institute,China,NaN,Active,Employee,Regular,NaN,Full time,Beijing,2015-10-08,NaT,Sr. Operations Manager - China,Exempt,114598.03,0.65,Manager
2,Zhe Wang,5,Rocky Mountain Institute,China,NaN,Active,Employee,Regular,NaN,Full time,Beijing,2015-10-20,NaT,Principal,Exempt,154204.11,0.60,Principal
3,Shutong Lu | Lucy Lu,6,Rocky Mountain Institute,China,NaN,Active,Employee,Regular,NaN,Full time,Beijing,2016-04-07,NaT,Manager,Exempt,123284.30,0.65,Manager
4,Dan Li | Danielle Li,7,Rocky Mountain Institute,China,NaN,Active,Employee,Regular,NaN,Full time,Beijing,2016-10-18,NaT,Sr Program Marketing Manager,Exempt,112163.96,0.65,Manager
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1106,Shuyi Wang | Lucy Wang,1985,Rocky Mountain Institute,Third Derivative,NaN,NaN,Contingent Worker,NaN,University Paid Intern,Full time,Remote - North Carolina,2024-06-04,2024-08-20,Intern,NaN,0.00,0.00,NaN
1107,Sadie Tetreault,1991,Rocky Mountain Institute,Climate Intelligence,NaN,NaN,Contingent Worker,NaN,University Paid Intern,Full time,Remote - North Carolina,2024-05-21,2024-08-06,Intern,NaN,0.00,0.00,NaN
1108,Sydney Williams,1992,Rocky Mountain Institute,Climate Finance Access Network,NaN,NaN,Contingent Worker,NaN,University Paid Intern,Full time,Remote - North Carolina,2024-05-21,2024-08-06,Intern,NaN,0.00,0.00,NaN
1109,Jessica Yan,1993,Rocky Mountain Institute,Finance,NaN,NaN,Contingent Worker,NaN,University Paid Intern,Full time,Remote - Arizona,2024-06-25,2024-09-17,Intern,NaN,0.00,0.00,NaN


In [58]:
UtilizationList = SalaryCATS[['Employee ID', 'Worker Utilization Target', 'Talent Pools']]

In [59]:
UtilizationList = UtilizationList.rename(columns={'Worker Utilization Target': 'Utilization Target'})

In [60]:
EmployeeCostCenters = SalaryCATS[['Employee ID', 'Cost Center', 'Employee Type', 'Hire Date', 'Termination Date','Location']].drop_duplicates()

In [61]:
HomeCostCenterlist = EmployeeCostCenters[['Cost Center', 'Employee ID']]
HomeCostCenterlist = HomeCostCenterlist.rename(columns={'Cost Center': 'Home Cost Center'})
#HomeCostCenterlist['Employee ID'] = HomeCostCenterlist['Employee ID'].astype(int)

In [62]:
Name_and_HomeCCList = SalaryCATS[['Employee ID', 'Employee Name']]

In [63]:
EmployeeCostCenters.dtypes

Employee ID                  int64
Cost Center                 object
Employee Type               object
Hire Date           datetime64[ns]
Termination Date    datetime64[ns]
Location                    object
dtype: object

In [64]:
len(EmployeeCostCenters)

1111

In [65]:
EmployeeCostCenters['Termination Date'].fillna('10-31-2040', inplace=True)
EmployeeCostCenters['Termination Date'] = EmployeeCostCenters['Termination Date'].astype('datetime64[ns]')

In [66]:
EmployeeCostCenters['Available Days1'] = EmployeeCostCenters['Hire Date'].apply(returndaysfromhire)
EmployeeCostCenters['Available Days2'] = EmployeeCostCenters['Termination Date'].apply(returndaysfromtermination)

In [67]:
EmployeeCostCenters['Termination Month'] = EmployeeCostCenters['Termination Date'].dt.month.astype(str).apply(lambda x: x.replace('.0',''))
EmployeeCostCenters['Termination Year'] = EmployeeCostCenters['Termination Date'].dt.year.astype(str).apply(lambda x: x.replace('.0',''))
EmployeeCostCenters['Termination Month&Year'] = EmployeeCostCenters['Termination Month'] + "/" + EmployeeCostCenters['Termination Year']

In [68]:
EmployeeCostCenters['Hire Month&Year'] = EmployeeCostCenters['Hire Date'].dt.month.astype(str) + '/' +EmployeeCostCenters['Hire Date'].dt.year.astype(str)

In [69]:
EmployeeCostCenters['Available Days3'] = EmployeeCostCenters['Available Days1'] + EmployeeCostCenters['Available Days2']
EmployeeCostCenters['Available Days3'] = EmployeeCostCenters['Available Days3'].replace(0,AvailableDays)
EmployeeCostCenters['Available DaysFINAL'] = np.where(EmployeeCostCenters['Available Days3'] > AvailableDays, EmployeeCostCenters['Available Days2'], EmployeeCostCenters['Available Days3'])

In [70]:
#EmployeeCostCenters.to_excel('/mnt/c/Users/jerem/anaconda3/envs/Code/RMICODING/Data/TimeCodingData/Outputs/EmployeeCostCenters2.xlsx')

In [71]:
EmployeeCostCenters.columns

Index(['Employee ID', 'Cost Center', 'Employee Type', 'Hire Date',
       'Termination Date', 'Location', 'Available Days1', 'Available Days2',
       'Termination Month', 'Termination Year', 'Termination Month&Year',
       'Hire Month&Year', 'Available Days3', 'Available DaysFINAL'],
      dtype='object')

In [72]:
EmployeeCostCenters = EmployeeCostCenters[['Employee ID', 'Cost Center', 'Employee Type', 'Hire Date',
       'Termination Date', 'Location',
       'Available DaysFINAL']]

In [73]:
EmployeeCostCenters = EmployeeCostCenters.rename(columns={'Available DaysFINAL': 'Available Days'})

In [74]:
EmployeeCostCenters['Employee Type'].fillna('Contingent', inplace=True)

In [75]:
EmployeeCostCenters['Available Days'].value_counts()

21    1081
20       8
15       4
16       3
6        3
5        3
11       2
10       2
4        2
1        2
9        1
Name: Available Days, dtype: int64

In [76]:
EmployeeCostCenters['Employee ID'] = EmployeeCostCenters['Employee ID'].astype(int)

In [77]:
EmployeeCostCenters = EmployeeCostCenters.rename(columns={'Employee Type': 'Category'})

In [78]:
LocationbyEmployee = EmployeeCostCenters[['Employee ID', 'Location']].drop_duplicates()

# Load Salary Data for RMI

In [79]:
# Loading this report returns what we have actually paid.  These are exactly the dollars that need to be allocated.


RMISalaries = './Source_Files/RMI_-_Payroll_Accounting_Detail_by_Ledger_Account_Spend_Category_and_Employee (3).xlsx'

#RMISalariesDF = pd.read_excel(RMISalaries)

RMISalariesDF = pd.read_excel(RMISalaries, header=1, engine='openpyxl')

In [80]:
RMISalariesDF

,Company,Ledger Account,Ledger/Budget Period,Spend Category,Spend Category ID,Employee,Employee ID,Cost Center,Net Amount,Paid Leave Amount
0,Rocky Mountain Institute,2100:Accrued Salaries and Bonuses,September-FY25 Actuals (Rocky Mountain Institute),Staff Accrued Bonuses,Staff Accrued Bonuses,Victor Olgyay,000130,Carbon-Free Buildings,47020.47,0
1,Rocky Mountain Institute,2100:Accrued Salaries and Bonuses,September-FY25 Actuals (Rocky Mountain Institute),Staff Accrued Bonuses,Staff Accrued Bonuses,Jacob Corvidae | Jacob Corvidae,000511,US,46884.43,0
2,Rocky Mountain Institute,2100:Accrued Salaries and Bonuses,September-FY25 Actuals (Rocky Mountain Institute),Staff Accrued Bonuses,Staff Accrued Bonuses,Deborah Gordon | Debbie Gordon,001206,Climate Intelligence,45148.26,0
3,Rocky Mountain Institute,2100:Accrued Salaries and Bonuses,September-FY25 Actuals (Rocky Mountain Institute),Staff Accrued Bonuses,Staff Accrued Bonuses,Kaitlin Crouch | Kaitlin Crouch Hess,001300,Finance,43522.00,0
4,Rocky Mountain Institute,2100:Accrued Salaries and Bonuses,September-FY25 Actuals (Rocky Mountain Institute),Staff Accrued Bonuses,Staff Accrued Bonuses,Suzanne Hopkins,000966,Strategic Operations,42932.01,0
...,...,...,...,...,...,...,...,...,...,...
7259,Rocky Mountain Institute,2100:Accrued Salaries and Bonuses,September-FY25 Actuals (Rocky Mountain Institute),Payroll Taxes Payable,Payroll Taxes Payable,Laurens Speelman,001200,Strategy Team,-26536.04,0
7260,Rocky Mountain Institute,2100:Accrued Salaries and Bonuses,September-FY25 Actuals (Rocky Mountain Institute),Payroll Taxes Payable,Payroll Taxes Payable,Cara Goldenberg,000589,Carbon-Free Electricity,-32627.87,0
7261,Rocky Mountain Institute,2100:Accrued Salaries and Bonuses,October-FY25 Actuals (Rocky Mountain Institute),Payroll Payables,Payroll Payables,(Blank),(Blank),(Blank),-1889632.45,0
7262,Rocky Mountain Institute,2100:Accrued Salaries and Bonuses,September-FY25 Actuals (Rocky Mountain Institute),Payroll Payables,Payroll Payables,(Blank),(Blank),(Blank),-5555701.45,0


In [81]:
RMISalariesDFfiltered = RMISalariesDF[RMISalariesDF['Spend Category']=="Salary Holding"]

In [82]:
RMISalariesDFfiltered = RMISalariesDFfiltered[RMISalariesDFfiltered['Company']!="Canary Media Inc."]
RMISalariesDFfiltered = RMISalariesDFfiltered[RMISalariesDFfiltered['Company']!="Total"]
RMISalariesDFfiltered = RMISalariesDFfiltered[RMISalariesDFfiltered['Net Amount']!=0]

## Create list of Employees Filtering out Employees who Changed Cost Centers

In [83]:
RMISalariesDF = RMISalariesDFfiltered[['Employee', 'Employee ID', 'Net Amount']]


In [84]:
RMISalariesDF['Employee ID'] = RMISalariesDF['Employee ID'].astype(np.int64)

C:\Users\Jwendt\AppData\Local\Temp\ipykernel_20800\2007134925.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RMISalariesDF['Employee ID'] = RMISalariesDF['Employee ID'].astype(np.int64)


In [85]:
#If this count decreases later in the script, it means that there were employees that changed Cost Centers in RMI.

len(RMISalariesDF)

604

In [86]:
RMISalariesDF = RMISalariesDF.groupby(['Employee ID', 'Employee'], as_index=False).sum()

Merge list of Employees with Cost Centers at end of Month

In [87]:
RMISalariesDF.columns

Index(['Employee ID', 'Employee', 'Net Amount'], dtype='object')

In [88]:
#RMISalariesDF = pd.merge(RMISalariesDF, EmployeeCostCenters, on = 'Employee ID', how = 'left')

In [89]:
RMISalariesDF.columns

Index(['Employee ID', 'Employee', 'Net Amount'], dtype='object')

In [90]:
EmployeeIDsandNames = SalaryCATS[['Employee ID', 'Employee Name']]
EmployeeIDsandNames = EmployeeIDsandNames.rename(columns={'Employee Name': 'Worker'})

In [91]:
len(RMISalariesDF)

602

In [92]:
len(TimeDataDF)

33112

In [93]:
#Fill any blanks with "Regular as a placeholder for now"

#RMISalariesDF['Category'].fillna("Regular", inplace = True)
RMISalariesDF

,Employee ID,Employee,Net Amount
0,107,Martha Pickett | Marty Pickett,28524.58
1,116,Virginia Yang | Ginny Yang,9090.14
2,122,Lena Hansen,26445.70
3,127,Jeannene Cruise Woolley | Frances Woolley,9283.74
4,130,Victor Olgyay,16042.22
...,...,...,...
597,2052,Camellia Moors,4258.34
598,2053,Annie Schonberger,1704.55
599,2054,Emily Siu,3181.82
600,2057,Mitchell Luti,1799.25


In [94]:
#RMISalariesDF = pd.merge(RMISalariesDF, ALLRMISalariesBonuses, on = 'Employee ID', how = 'left')

In [95]:
RMISalariesDF.columns

Index(['Employee ID', 'Employee', 'Net Amount'], dtype='object')

In [96]:
RMIBenefitsCount = len(RMISalariesDF)

In [97]:
RMISharedCostsCount = len(RMISalariesDF)

In [98]:
RMIBenefitsCount

602

In [99]:
RMISharedCostsCount

602

# Load Salary, Benefits, and Shared Cost for Beijing

In [100]:
TransDetailBeijing = './Source_Files//Transaction_Detail_ALL.xlsx'

TransDetailBeijingDF = pd.read_excel(TransDetailBeijing,engine='openpyxl')

In [101]:
# DO I USE "Source['Manual Journal']" to isolate what I need?


BeijingSalaries = TransDetailBeijingDF[(TransDetailBeijingDF['Source'] == 'Manual Journal') | (TransDetailBeijingDF['Source'] == 'Other Reclasses')]

BeijingSalaries = BeijingSalaries[(BeijingSalaries['Location'] == 'Beijing') & (BeijingSalaries['Spend Category'] == 'Salaries & Wages')]

# need to filter on Salaries and Wages Spend Category

In [102]:
BeijingSalaries.columns

Index(['Journal', 'Journal Number', 'CF LRV Ad Hoc Bank Number',
       'Operational Transaction', 'Company', 'Status', 'Accounting Date',
       'Source', 'Ledger', 'Currency', 'Ledger Account', 'Ledger Debit Amount',
       'Ledger Credit Amount', 'Net Amount', 'Line Memo',
       'Intercompany Affiliate', 'Bank Account', 'Revenue Category',
       'Spend Category', 'Cost Center', 'Initiative', 'Project', 'Project ID',
       'Project Plan Task', 'Project Plan Task ID', 'Gift', 'Fund',
       'Funding Source', 'Customer', 'Supplier', 'Employee', 'Employee ID',
       'Location', 'Worktags'],
      dtype='object')

In [103]:
BeijingSalaries = BeijingSalaries[['Employee', 'Employee ID', 'Net Amount']]

In [104]:
BeijingSalaries = BeijingSalaries.groupby(['Employee', 'Employee ID'], as_index=False).sum()

In [105]:
BeijingSalaries = pd.merge(BeijingSalaries, EmployeeCostCenters, on = 'Employee ID', how = 'left')

In [106]:
BeijingSalaries = pd.merge(BeijingSalaries, ALLRMISalariesBonuses, on = 'Employee ID', how = 'left')

In [107]:
BeijingBenefitsDF = TransDetailBeijingDF[TransDetailBeijingDF['Ledger Account'] == '5100:Employee Benefits']

BeijingBenefitsDF = BeijingBenefitsDF[(BeijingBenefitsDF['Location'] == 'Beijing')] 

BeijingBenefits = BeijingBenefitsDF[(BeijingBenefitsDF['Project'] == 'China [Admin]') | (BeijingBenefitsDF['Project'] == 'External Professional Development (HR/L&D)') | (BeijingBenefitsDF['Project'] == 'Learning and Development (HR/L&D)')]

BeijingBenefitsTotal = BeijingBenefits['Net Amount'].sum()

In [108]:
BeijingBenefitsTotal

75842.97000000002

In [109]:
BeijingSalariesTotal = BeijingSalaries['Net Amount'].sum()

In [110]:
BeijingSalariesTotal

290929.37000000005

In [111]:
BeijingEmployeeCount = BeijingSalaries['Employee ID'].nunique()
BeijingEmployeeCount

33

In [112]:
BeijingBenefitsperEmployee = BeijingBenefitsTotal/BeijingEmployeeCount
BeijingBenefitsperEmployee

2298.2718181818186

In [113]:
BeijingShared = TransDetailBeijingDF[(TransDetailBeijingDF['Ledger Account'] == '8100:Shared Costs') | (TransDetailBeijingDF['Ledger Account'] == '7100:Depreciation Expenses')]

BeijingShared = BeijingShared[BeijingShared['Location'] == 'Beijing']

BeijingSharedTotal = BeijingShared['Net Amount'].sum()

In [114]:
BeijingSharedTotal
#Includes Depreciation...this is why I don't tie to Whitney's number

63658.00000000001

In [115]:
BeijingSharedperEmployee = BeijingSharedTotal/BeijingEmployeeCount
BeijingSharedperEmployee

1929.0303030303032

In [116]:
BeijingSalaries["Benefits per Employee"] = BeijingBenefitsperEmployee

In [117]:
BeijingSalaries["Shared per Employee"] = BeijingSharedperEmployee

In [118]:
#Need to add here total shared costs for China and benefits for China

# Load Benefits, and Shared Cost for RMI and GP

In [119]:
TransDetailBenefitsShared_GP_RMI = './Source_Files/Transaction_Detail_ALL.xlsx'

TransDetailBenefitsShared_GP_RMI_DF = pd.read_excel(TransDetailBenefitsShared_GP_RMI, engine='openpyxl')

In [120]:
TransDetailBenefitsShared_GP_RMI_filteredDF = TransDetailBenefitsShared_GP_RMI_DF[TransDetailBenefitsShared_GP_RMI_DF['Location']!='Beijing']

In [121]:
GP_RMI_Benefits = TransDetailBenefitsShared_GP_RMI_filteredDF[TransDetailBenefitsShared_GP_RMI_filteredDF['Ledger Account'] == '5100:Employee Benefits'] 

GP_RMI_BenefitsTotal = GP_RMI_Benefits['Net Amount'].sum()

GP_RMI_BenefitsTotal

2183773.7800000003

In [122]:
GP_RMI_Shared = TransDetailBenefitsShared_GP_RMI_filteredDF[TransDetailBenefitsShared_GP_RMI_filteredDF['Ledger Account'] == '8100:Shared Costs']

GP_RMI_Shared = GP_RMI_Shared[GP_RMI_Shared['Spend Category']!="Shared Costs Clearing Account"]

GP_RMI_Shared = GP_RMI_Shared[GP_RMI_Shared['Location']!="New Delhi"]

GP_RMI_Shared = GP_RMI_Shared[GP_RMI_Shared['Initiative']!="CAI - MPP"]

GP_RMI_Depreciation = TransDetailBenefitsShared_GP_RMI_DF[(TransDetailBenefitsShared_GP_RMI_DF['Project'] == 'Operations [Admin]') | (TransDetailBenefitsShared_GP_RMI_DF['Project'] == 'Institution Expenses')] 


GP_RMI_Depreciation = GP_RMI_Depreciation[GP_RMI_Depreciation['Spend Category']=="Depreciation"] 

GP_RMI_SharedTotal = GP_RMI_Shared['Net Amount'].sum() + GP_RMI_Depreciation['Net Amount'].sum()

GP_RMI_SharedTotal

672855.34

In [123]:
GP_RMI_Shared['Net Amount'].sum()

595785.01

In [124]:
GP_RMI_Depreciation['Net Amount'].sum()

77070.33

# Load Salaries for GP

In [125]:
TransDetailGPSalaries = './Source_Files/Transaction_Detail_ALL.xlsx'

TransDetailGPSalariesDF = pd.read_excel(TransDetailGPSalaries, engine='openpyxl')

TransDetailGPSalariesDF = TransDetailGPSalariesDF[(TransDetailGPSalariesDF['Source'] == 'Supplier Invoice') | (TransDetailGPSalariesDF['Source'] == 'Other Reclasses')]

TransDetailGPSalariesDF = TransDetailGPSalariesDF[TransDetailGPSalariesDF['Spend Category'] == 'Salary Holding']




In [126]:
TransDetailGPSalariesDF.columns

Index(['Journal', 'Journal Number', 'CF LRV Ad Hoc Bank Number',
       'Operational Transaction', 'Company', 'Status', 'Accounting Date',
       'Source', 'Ledger', 'Currency', 'Ledger Account', 'Ledger Debit Amount',
       'Ledger Credit Amount', 'Net Amount', 'Line Memo',
       'Intercompany Affiliate', 'Bank Account', 'Revenue Category',
       'Spend Category', 'Cost Center', 'Initiative', 'Project', 'Project ID',
       'Project Plan Task', 'Project Plan Task ID', 'Gift', 'Fund',
       'Funding Source', 'Customer', 'Supplier', 'Employee', 'Employee ID',
       'Location', 'Worktags'],
      dtype='object')

In [127]:
GPSalariesDF = TransDetailGPSalariesDF[['Employee', 'Employee ID', 'Net Amount']]

In [128]:
GPSalariesDF = GPSalariesDF.groupby(['Employee', 'Employee ID'], as_index=False).sum()

In [129]:
#GPSalariesDF = pd.merge(GPSalariesDF, EmployeeCostCenters, on = 'Employee ID', how = 'left')

In [130]:
#GPSalariesDF = pd.merge(GPSalariesDF, ALLRMISalariesBonuses, on = 'Employee ID', how = 'left')

In [131]:
GPSalariesDF['Net Amount'].sum()

709255.99

In [132]:
RMISalariesDF['Net Amount'].sum()

5723404.880000001

In [133]:
TotalSalariesRMI = RMISalariesDF['Net Amount'].sum() + GPSalariesDF['Net Amount'].sum()

In [134]:
TotalSalariesRMI

6432660.870000001

# Create Combined GP + RMI Salaries File & Add Bonus

In [135]:
RMISalariesDF

,Employee ID,Employee,Net Amount
0,107,Martha Pickett | Marty Pickett,28524.58
1,116,Virginia Yang | Ginny Yang,9090.14
2,122,Lena Hansen,26445.70
3,127,Jeannene Cruise Woolley | Frances Woolley,9283.74
4,130,Victor Olgyay,16042.22
...,...,...,...
597,2052,Camellia Moors,4258.34
598,2053,Annie Schonberger,1704.55
599,2054,Emily Siu,3181.82
600,2057,Mitchell Luti,1799.25


In [136]:
RMIandGPSalariesDF = pd.concat([RMISalariesDF, GPSalariesDF])


In [137]:
RMIandGPSalariesDF = RMIandGPSalariesDF.groupby(['Employee', 'Employee ID'], as_index=False).sum()

In [138]:
RMIandGPSalariesDF = pd.merge(RMIandGPSalariesDF, EmployeeCostCenters, on = 'Employee ID', how = 'left')

In [139]:
RMIandGPSalariesDF = pd.merge(RMIandGPSalariesDF, ALLRMISalariesBonuses, on = 'Employee ID', how = 'left')

In [140]:
RMIandGPSalariesDF['Bonus Final'].fillna(0, inplace = True)

# Add Benefits and Shared Costs per Employee to Master Employee Files

In [141]:
GPHeadcount = len(GPSalariesDF)
GPHeadcount

72

In [142]:
GP_HC_TOTAL = GPSalariesDF[['Employee ID']]
len(GP_HC_TOTAL)

72

In [143]:
RMI_HC_TOTAL = RMISalariesDF[['Employee ID']]
len(RMI_HC_TOTAL)

602

In [144]:
TOTAL_RMI_HC = pd.concat([GP_HC_TOTAL, RMI_HC_TOTAL])

In [145]:
RMIHeadCountBenefits = TOTAL_RMI_HC.groupby(['Employee ID'], as_index=False).sum()
RMIHeadCountShahred  = TOTAL_RMI_HC.groupby(['Employee ID'], as_index=False).sum()

In [146]:
RMIHeadCountBenefits = len(RMIHeadCountBenefits)
RMIHeadCountBenefits

674

In [147]:
BenefitsperRMIHC = GP_RMI_BenefitsTotal/RMIHeadCountBenefits
BenefitsperRMIHC

3240.020445103858

In [148]:
RMIHeadCountShahred = len(RMIHeadCountShahred)

In [149]:
RMIHeadCountShahred

674

In [150]:
SharedperRMIHC = GP_RMI_SharedTotal/RMIHeadCountShahred
SharedperRMIHC

998.3016913946587

In [151]:
RMIandGPSalariesDF["Benefits per Employee"] = BenefitsperRMIHC

In [152]:
RMIandGPSalariesDF["Shared per Employee"] = SharedperRMIHC

# Manipulate Time Data Report

First create lists to seperate.  This code brings all salaries to split out into the report.

In [153]:
len(TimeDataDF)

33112

In [154]:
GPSalariesDF['Employee ID'] = GPSalariesDF['Employee ID'].astype(np.int64)

In [155]:
BeijingSalaries['Employee ID'] = BeijingSalaries['Employee ID'].astype(np.int64)

In [156]:
GPSalariesDF.columns

Index(['Employee', 'Employee ID', 'Net Amount'], dtype='object')

In [157]:
GPSalariesDF

,Employee,Employee ID,Net Amount
0,Alberto Rodriguez,1360,11663.85
1,Alex Axthelm,1530,9516.42
2,Alexander Hogeveen Rutter (Alexander Hogeveen ...,1893,10411.87
3,Alexander Walmsley | Alex Walmsley,1812,11378.20
4,Alexandra Khripko,1428,12183.08
...,...,...,...
67,Truc Huynh | Tracy Huynh (On Leave),1649,8265.70
68,Wayne Omonuwa,1293,13504.66
69,Wini Rizkiningayu,1835,14377.83
70,Xiyuan Liu,1372,8061.28


In [158]:
TimeDataDF.columns

Index(['Worker', 'Employee ID', 'Worker Cost Center', 'Talent Pool',
       'Project Group', 'Utilization Target', 'Hire Date', 'Termination Date',
       'Title', 'Hourly Rate', 'Monthly Salary', 'Monthly Bonus Amount',
       'Bonus %', 'Week', 'Reported Date', 'Time Type', 'Entry Type',
       'Project', 'Project Plan Task', 'Project Plan Task ID', 'Cost Center',
       'Initiative', 'Lobbying', 'Lobbying Location', 'Reported Quantity',
       'Time Cost Center', 'Time Initiative', 'MonthYear'],
      dtype='object')

# Create India File

In [159]:
TransDetailIndiaSalaries = './Source_Files/Transaction_Detail_ALL.xlsx'

TransDetailIndiaSalariesDF = pd.read_excel(TransDetailIndiaSalaries, engine='openpyxl')

TransDetailIndiaSalariesDF = TransDetailIndiaSalariesDF[TransDetailIndiaSalariesDF['Location'] == 'New Delhi']

TransDetailIndiaSalariesDF = TransDetailIndiaSalariesDF[TransDetailIndiaSalariesDF['Spend Category'] == 'Consultant Fees']

TransDetailIndiaSalariesDF = TransDetailIndiaSalariesDF[TransDetailIndiaSalariesDF['Ledger Account'] == '5200:Consultants and Contractors']
TransDetailIndiaSalariesDF = TransDetailIndiaSalariesDF[TransDetailIndiaSalariesDF['Project'] == 'People Team Operations Expenses']

In [160]:
TransDetailIndiaSalariesDF['Net Amount'].sum()

282181.81

In [161]:
IndiaALLIDs = TransDetailIndiaSalariesDF[["Employee", "Employee ID", "Net Amount"]]
IndiaALLIDs['India'] = 'Yes'
#IndiaALLIDs['Employee ID'] = IndiaALLIDs['Employee ID'].apply(add_zeros)

IndiaALLIDs = IndiaALLIDs.groupby(['Employee', 'Employee ID','India'], as_index=False).sum()

IndiaSalariesDF = IndiaALLIDs[['Employee', 'Employee ID', 'Net Amount']]
IndiaSalariesDF = IndiaSalariesDF.groupby(['Employee', 'Employee ID'], as_index=False).sum()

IndiaSalariesDF = pd.merge(IndiaSalariesDF, EmployeeCostCenters, on = 'Employee ID', how = 'left')
IndiaSalariesDF = pd.merge(IndiaSalariesDF, ALLRMISalariesBonuses, on = 'Employee ID', how = 'left')

IndiaSalariesDF["Benefits per Employee"] = 0
IndiaSalariesDF["Shared per Employee"] = 0
IndiaSalariesDF["Bonus Final"] = 0
IndiaSalariesDF['Employee ID'] = IndiaSalariesDF['Employee ID'].astype(np.int64)


C:\Users\Jwendt\AppData\Local\Temp\ipykernel_20800\3304116708.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  IndiaALLIDs['India'] = 'Yes'


In [162]:
#IndiaALLIDs.to_excel('/mnt/c/Users/jerem/anaconda3/envs/Code/RMICODING/Data/TimeCodingData/Outputs/Indiadata.xlsx')

In [163]:
IndiaSalariesDF['Net Amount'].sum()

282181.81

In [164]:
IndiaTime =  TimeDataDFALL[TimeDataDFALL['MonthYear']==currentmonthandyear]

IndiaTimeMaster = pd.merge(IndiaTime, IndiaALLIDs, on = 'Employee ID', how = 'left')
IndiaTimeMaster = IndiaTimeMaster[IndiaTimeMaster['India']=='Yes']



In [165]:
IndiaTimeMaster['Net Amount'].sum()

12741296.75

In [166]:
IndiaTimeMaster['SalaryCosted'] = 0
IndiaTimeMaster['ConsultingFeesCosted'] = IndiaTimeMaster['Percent of Total Monthly Hours'] * IndiaTimeMaster['Net Amount']
IndiaTimeMaster['BenefitsCosted'] = 0
IndiaTimeMaster['SharedCosted'] = 0
IndiaTimeMaster['BonusCosted'] = 0
IndiaTimeMaster['Type'] = "India"
IndiaTimeMaster['Admin Hours Percentage'] = 0
IndiaTimeMaster['Project Hours Percentage'] = 1

In [167]:
IndiaTimeMaster['ConsultingFeesCosted'].sum()

252459.01999999996

In [168]:
IndiaTimeMaster.columns

Index(['Worker', 'Employee ID', 'Worker Cost Center', 'Talent Pool',
       'Project Group', 'Utilization Target', 'Hire Date', 'Termination Date',
       'Title', 'Reported Date', 'Time Type', 'Entry Type', 'Project',
       'Project Plan Task', 'Project Plan Task ID', 'Cost Center',
       'Initiative', 'Reported Quantity', 'Week', 'Time Cost Center',
       'Time Initiative', 'MonthYear', 'Project or Admin', 'Employee Hours',
       'Percent of Total Monthly Hours', 'Employee', 'India', 'Net Amount',
       'SalaryCosted', 'ConsultingFeesCosted', 'BenefitsCosted',
       'SharedCosted', 'BonusCosted', 'Type', 'Admin Hours Percentage',
       'Project Hours Percentage'],
      dtype='object')

In [169]:
IndiaTimeMaster['Project Plan Task ID'] = np.where((IndiaTimeMaster['Project Plan Task'] == 'PTO') & (IndiaTimeMaster['Type'] == 'India') , 'PROJECT_PLAN_TASK-3-5727', IndiaTimeMaster['Project Plan Task ID'])

IndiaTimeMaster['Initiative'] = np.where((IndiaTimeMaster['Project Plan Task'] == 'PTO') & (IndiaTimeMaster['Type'] == 'India') , 'IN - Admin', IndiaTimeMaster['Initiative'])

IndiaTimeMaster['Cost Center'] = np.where((IndiaTimeMaster['Project Plan Task'] == 'PTO') & (IndiaTimeMaster['Type'] == 'India') , 'India', IndiaTimeMaster['Cost Center'])

IndiaTimeMaster['Project Plan Task'] = np.where((IndiaTimeMaster['Project Plan Task'] == 'PTO') & (IndiaTimeMaster['Type'] == 'India') , 'India > PTO', IndiaTimeMaster['Project Plan Task'])

IndiaTimeMaster['Project Plan Task ID'] = np.where((IndiaTimeMaster['Project Plan Task'] == 'Holiday') & (IndiaTimeMaster['Type'] == 'India') , 'PROJECT_PLAN_TASK-3-5727', IndiaTimeMaster['Project Plan Task ID'])

IndiaTimeMaster['Initiative'].fillna("IN - India", inplace = True)

IndiaTimeMaster['Cost Center'].fillna("India", inplace = True)

IndiaTimeMaster['Project Plan Task'] = np.where((IndiaTimeMaster['Project Plan Task'] == 'Holiday') & (IndiaTimeMaster['Type'] == 'India') , 'India > PTO', IndiaTimeMaster['Project Plan Task'])

In [170]:
IndiaTimeMasterAllocated = IndiaTimeMaster[['Worker', 'Employee ID', 'Project Plan Task', 'Project Plan Task ID',
       'Admin Hours Percentage', 'Project Hours Percentage', 'Cost Center',
       'Initiative', 'SalaryCosted', 'BenefitsCosted', 'SharedCosted', 
       'BonusCosted', 'ConsultingFeesCosted', 'Type',
       'Project or Admin', 'Project', 'Worker Cost Center']]


In [171]:
IndiaTimeMasterAllocated.head(30)

,Worker,Employee ID,Project Plan Task,Project Plan Task ID,Admin Hours Percentage,Project Hours Percentage,Cost Center,Initiative,SalaryCosted,BenefitsCosted,SharedCosted,BonusCosted,ConsultingFeesCosted,Type,Project or Admin,Project,Worker Cost Center
203,Abhishek Murali,2010,Phase 1 > Launch event,PROJECT_PLAN_TASK-3-8092,0,1,India,IN - India,0,0,0,0,174.012202,India,Project,[NLM] - 2047 Mobility Visioning Document,India
204,Abhishek Murali,2010,Phase 1 > Policy document development,PROJECT_PLAN_TASK-3-6609,0,1,India,IN - India,0,0,0,0,104.407321,India,Project,[ZET] - ZET Policy,India
205,Abhishek Murali,2010,Phase 1 > Policy document development,PROJECT_PLAN_TASK-3-6609,0,1,India,IN - India,0,0,0,0,104.407321,India,Project,[ZET] - ZET Policy,India
206,Abhishek Murali,2010,Phase 1 > Policy document development,PROJECT_PLAN_TASK-3-6609,0,1,India,IN - India,0,0,0,0,278.419524,India,Project,[ZET] - ZET Policy,India
207,Abhishek Murali,2010,Phase 1 > Policy document development,PROJECT_PLAN_TASK-3-6609,0,1,India,IN - India,0,0,0,0,278.419524,India,Project,[ZET] - ZET Policy,India
208,Abhishek Murali,2010,Phase 1 > Policy document development,PROJECT_PLAN_TASK-3-6609,0,1,India,IN - India,0,0,0,0,278.419524,India,Project,[ZET] - ZET Policy,India
209,Abhishek Murali,2010,Admin > Internal Meetings,PROJECT_PLAN_TASK-3-2434,0,1,India,IN - Admin,0,0,0,0,174.012202,India,Admin,India [Admin],India
210,Abhishek Murali,2010,Phase 1: e3W Report > Report Writing,PROJECT_PLAN_TASK-3-6542,0,1,India,IN - India,0,0,0,0,174.012202,India,Project,[NLM] - Accelerating e2/3W Adoption,India
211,Abhishek Murali,2010,Phase 1: e3W Report > Report Writing,PROJECT_PLAN_TASK-3-6542,0,1,India,IN - India,0,0,0,0,174.012202,India,Project,[NLM] - Accelerating e2/3W Adoption,India
212,Abhishek Murali,2010,Phase 1: e3W Report > Report Writing,PROJECT_PLAN_TASK-3-6542,0,1,India,IN - India,0,0,0,0,139.209762,India,Project,[NLM] - Accelerating e2/3W Adoption,India


## Create Master File With All Costs per EmployeeID

In [172]:
BeijingSalaries['FileType'] = "Beijing"

RMIandGPSalariesDF['FileType'] = "GP & RMI"


IndiaSalariesDF['FileType'] = "India"

In [173]:
BeijingSalaries['Consulting Per Employee'] = 0

RMIandGPSalariesDF['Consulting Per Employee'] = 0


IndiaSalariesDF.rename(columns={'Net Amount': 'Consulting Per Employee'}, inplace=True)

IndiaSalariesDF['Net Amount'] = 0

In [174]:
def add_zeros(x):
    y = str(x)
    while len(y) < 6:
        y = '0' + y
    return y

In [175]:
len(GPSalariesDF)

72

In [176]:
AllSalaries = pd.concat([BeijingSalaries, RMIandGPSalariesDF, IndiaSalariesDF], ignore_index=True)

In [177]:
len(AllSalaries)

734

In [178]:
AllSalaries.columns

Index(['Employee', 'Employee ID', 'Net Amount', 'Cost Center', 'Category',
       'Hire Date', 'Termination Date', 'Location', 'Available Days',
       'Bonus Final', 'Benefits per Employee', 'Shared per Employee',
       'FileType', 'Consulting Per Employee'],
      dtype='object')

In [179]:
#AllSalaries.to_excel('/mnt/c/Users/jerem/anaconda3/envs/Code/RMICODING/Data/TimeCodingData/Outputs/AllSalaries.xlsx')

# Add Bonus only to Salaries

In [180]:
AllBonusTable = ALLRMISalariesBonuses

In [181]:
AllBonusTable.rename(columns={'Bonus Final': 'Bonus ONLY'}, inplace=True)

In [182]:
BonusesOnlyDF = pd.merge(AllBonusTable, AllSalaries, on = 'Employee ID', how = 'left')

In [183]:
BonusesOnlyDF = BonusesOnlyDF[BonusesOnlyDF['Net Amount'].isnull()]

In [184]:
BonusesOnlyDF = BonusesOnlyDF[['Employee ID', 'Bonus ONLY']]

In [185]:
len(BonusesOnlyDF)

6

In [186]:
SalaryCATS.columns

Index(['Employee Name', 'Employee ID', 'Company', 'Cost Center', 'Work Email',
       'Employment Status', 'Worker Type', 'Employee Type',
       'Contingent Worker Type', 'Time Type', 'Location', 'Hire Date',
       'Termination Date', 'Title', 'Exempt/Non-Exempt', 'Annual Base (USD)',
       'Worker Utilization Target', 'Talent Pools'],
      dtype='object')

In [187]:
AllSalaries.columns

Index(['Employee', 'Employee ID', 'Net Amount', 'Cost Center', 'Category',
       'Hire Date', 'Termination Date', 'Location', 'Available Days',
       'Bonus Final', 'Benefits per Employee', 'Shared per Employee',
       'FileType', 'Consulting Per Employee'],
      dtype='object')

In [188]:
BonusesOnlyDF = pd.merge(BonusesOnlyDF, SalaryCATS, on = 'Employee ID', how = 'left')

In [189]:
BonusesOnlyDF.columns

Index(['Employee ID', 'Bonus ONLY', 'Employee Name', 'Company', 'Cost Center',
       'Work Email', 'Employment Status', 'Worker Type', 'Employee Type',
       'Contingent Worker Type', 'Time Type', 'Location', 'Hire Date',
       'Termination Date', 'Title', 'Exempt/Non-Exempt', 'Annual Base (USD)',
       'Worker Utilization Target', 'Talent Pools'],
      dtype='object')

In [190]:
BonusesOnlyDF = BonusesOnlyDF[['Employee ID', 'Bonus ONLY', 'Employee Name', 'Cost Center','Employee Type', 'Hire Date', 'Termination Date', 'Location']]

In [191]:
BonusesOnlyDF.rename(columns={'Employee Name': 'Employee', 'Employee Type': 'Category', 'Bonus ONLY': 'Bonus Final'}, inplace=True)

In [192]:
BonusesOnlyDF['Net Amount'] = 0
BonusesOnlyDF['Available Days'] = 0
BonusesOnlyDF['Benefits per Employee'] = 0
BonusesOnlyDF['Shared per Employee'] = 0
BonusesOnlyDF['Consulting Per Employee'] = 0
BonusesOnlyDF['FileType'] = 'Bonus Only'

In [193]:
AllSalaries = pd.concat([AllSalaries, BonusesOnlyDF], ignore_index=True)

In [194]:
len(AllSalaries)

740

In [195]:
len(AllSalaries)

740

In [196]:
HomeProjects_RMI = './Source_Files/RMI-Adminprojects,Generaltasks.xlsx'

HomeProjects_DF = pd.read_excel(HomeProjects_RMI, engine='openpyxl')

In [197]:
HomeProjects_RMI_PTO = './Source_Files/Extract_Project_Structure_-_Ref_IDs_PTO.xlsx'

HomeProjects_DF_PTO = pd.read_excel(HomeProjects_RMI_PTO, engine='openpyxl')

In [198]:
#click "Terminated" and leave other fields blank

PartTimeUtilization_RMI = './Source_Files/RMI_-_Accounting_Part_Time_Workers.xlsx'

PartTime_DF = pd.read_excel(PartTimeUtilization_RMI, header=0, engine='openpyxl')





In [199]:
PercentTable = PartTime_DF[['Worker', 'FTE']]

In [200]:
PercentTable.rename(columns={'Worker': 'Employee'}, inplace=True)

C:\Users\Jwendt\AppData\Local\anaconda3\envs\new_env_name\lib\site-packages\pandas\core\frame.py:4300: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [201]:
AllSalaries = pd.merge(AllSalaries, PercentTable, on = 'Employee', how = 'left')

In [202]:
AllSalaries['FTE'].fillna(1, inplace=True)

In [203]:
AllSalaries['Available Days'].value_counts()

21    711
20      7
0       6
15      3
5       3
16      2
10      2
4       2
11      1
9       1
6       1
1       1
Name: Available Days, dtype: int64

In [204]:
len(AllSalaries)

740

In [205]:
AllSalaries['Adjusted Hours'] = AllSalaries['Available Days'] * 8 * AllSalaries['FTE'] 

In [206]:
AllSalaries.head()

,Employee,Employee ID,Net Amount,Cost Center,Category,Hire Date,Termination Date,Location,Available Days,Bonus Final,Benefits per Employee,Shared per Employee,FileType,Consulting Per Employee,FTE,Adjusted Hours
0,Boya Zhang （张博雅）,1841.0,6134.01,China,Regular,2023-06-26,2040-10-31,Beijing,21,299.468391,2298.271818,1929.030303,Beijing,0.0,1.0,168.0
1,Canyang Xie （谢璨阳）,2028.0,5964.92,China,Regular,2024-07-08,2040-10-31,Beijing,21,290.746011,2298.271818,1929.030303,Beijing,0.0,1.0,168.0
2,Dan Li | Danielle Li,7.0,9229.44,China,Regular,2016-10-18,2040-10-31,Beijing,21,908.757025,2298.271818,1929.030303,Beijing,0.0,1.0,168.0
3,Guangxu Wang,1275.0,7407.45,China,Regular,2021-08-02,2040-10-31,Beijing,21,363.197323,2298.271818,1929.030303,Beijing,0.0,1.0,168.0
4,Guangyu Tan,1395.0,7250.10,China,Regular,2022-01-17,2040-10-31,Beijing,21,355.323110,2298.271818,1929.030303,Beijing,0.0,1.0,168.0


In [207]:
AdjustedHoursToMasterTime = AllSalaries[['Adjusted Hours', 'Employee ID']]

In [208]:
len(AdjustedHoursToMasterTime)

740

In [209]:
TimeDataDF = pd.merge(TimeDataDF, AdjustedHoursToMasterTime, on = 'Employee ID', how = 'left')

In [210]:
TimeDataDF['Needed Project Hours'] = TimeDataDF['Utilization Target'] * TimeDataDF['Adjusted Hours']
TimeDataDF['Needed Project Hours'].fillna(AvailableHours, inplace = True)

In [211]:
len(TimeDataDF)

33112

In [212]:
TimeDataDF.head()

,Worker,Employee ID,Worker Cost Center,Talent Pool,Project Group,Utilization Target,Hire Date,Termination Date,Title,Hourly Rate,...,Cost Center,Initiative,Lobbying,Lobbying Location,Reported Quantity,Time Cost Center,Time Initiative,MonthYear,Adjusted Hours,Needed Project Hours
0,Aaron Brickman,1436,US,Talent Pool: Sr. Principal,Holiday,0.6,2022-02-28,NaT,Senior Principal,102.53,...,NaN,NaN,None,None,8.0,NaN,NaN,9/2024,168.0,100.8
1,Aaron Brickman,1436,US,Talent Pool: Sr. Principal,Grant Funded,0.6,2022-02-28,NaT,Senior Principal,102.53,...,US,US - Clean Regional Economic Development,None,None,4.0,US,US - Clean Regional Economic Development,9/2024,168.0,100.8
2,Aaron Brickman,1436,US,Talent Pool: Sr. Principal,Grant Funded,0.6,2022-02-28,NaT,Senior Principal,102.53,...,US,US - Clean Regional Economic Development,None,None,4.0,US,US - Clean Regional Economic Development,9/2024,168.0,100.8
3,Aaron Brickman,1436,US,Talent Pool: Sr. Principal,Grant Funded,0.6,2022-02-28,NaT,Senior Principal,102.53,...,US,US - Clean Regional Economic Development,None,None,4.0,US,US - Clean Regional Economic Development,9/2024,168.0,100.8
4,Aaron Brickman,1436,US,Talent Pool: Sr. Principal,Grant Funded,0.6,2022-02-28,NaT,Senior Principal,102.53,...,US,US - Clean Regional Economic Development,None,None,4.0,US,US - Clean Regional Economic Development,9/2024,168.0,100.8


In [213]:
HomeProjects_DF.columns

Index(['Project', 'Project Hierarchy - Level 02', 'Project Ref ID',
       'External Project Reference', 'Cost Center', 'Cost Center Ref ID',
       'Initiative', 'Initiative Ref ID', 'Project Phase', 'Project Task',
       'Project Plan Task Home', 'Project Plan Task ID Home',
       'Project Plan Task WID'],
      dtype='object')

In [214]:
HomeProjects_DF_Final = HomeProjects_DF[['Project Plan Task Home', 'Project Plan Task ID Home', 'Cost Center', 'Initiative']]

In [215]:
HomeProjects_DF_PTO = HomeProjects_DF_PTO.rename(columns={'Project Plan Task': 'Project Plan Task PTO', 'Project Plan Task ID': 'Project Plan Task ID PTO'})

In [216]:
HomeProjects_DF_Final_PTO = HomeProjects_DF_PTO[['Project Plan Task PTO', 'Project Plan Task ID PTO', 'Cost Center']]

In [217]:
AllSalariesFinal = pd.merge(AllSalaries, HomeProjects_DF_Final, on = 'Cost Center', how = 'left')

In [218]:
AllSalariesFinal = pd.merge(AllSalariesFinal, HomeProjects_DF_Final_PTO, on = 'Cost Center', how = 'left')

In [219]:
len(AllSalariesFinal)

740

In [220]:
AllSalariesFinal['Hire Date'] = AllSalariesFinal['Hire Date'].dt.strftime('%m/%d/%Y')
AllSalariesFinal['Termination Date'] = AllSalariesFinal['Termination Date'].dt.strftime('%m/%d/%Y')

In [221]:
len(AllSalariesFinal)

740

# Count days between Termination Date and end date of Month

In [222]:
AllSalaries.rename(columns={'Net Amount': 'SalarytoAllocate'}, inplace=True)

# Run calculations for TimeSHeets and Project Time and Admin Time, etc

Now we need to bring the Home Cost Center Groupings (Department vs Program) and Project Groups in (Project or Admin).

In [223]:
TimeDataDF.columns

Index(['Worker', 'Employee ID', 'Worker Cost Center', 'Talent Pool',
       'Project Group', 'Utilization Target', 'Hire Date', 'Termination Date',
       'Title', 'Hourly Rate', 'Monthly Salary', 'Monthly Bonus Amount',
       'Bonus %', 'Week', 'Reported Date', 'Time Type', 'Entry Type',
       'Project', 'Project Plan Task', 'Project Plan Task ID', 'Cost Center',
       'Initiative', 'Lobbying', 'Lobbying Location', 'Reported Quantity',
       'Time Cost Center', 'Time Initiative', 'MonthYear', 'Adjusted Hours',
       'Needed Project Hours'],
      dtype='object')

In [224]:
#MappingPath = '/mnt/c/Users/jerem/anaconda3/envs/Code/RMICODING/Data/TimeCodingData/MappingSalaryCosting.xlsx'

#ProjectsMaps = pd.read_excel(MappingPath, sheet_name = 'CostCenter')

#TimeDataDF = pd.merge(TimeDataDF, ProjectsMaps, on = 'Worker Cost Center', how = 'left')

In [225]:
MappingPath = './Source_Files/MappingSalaryCosting.xlsx'

ProjectsMaps = pd.read_excel(MappingPath, sheet_name = 'AdminvsProject', engine='openpyxl')

TimeDataDF = pd.merge(TimeDataDF, ProjectsMaps, on = 'Project Group', how = 'left')

In [226]:
TimeDataDF['Project or Admin'].value_counts()

Project    25315
Admin       7797
Name: Project or Admin, dtype: int64

In [227]:
len(TimeDataDF)

33112

Now we need to calculate Utilization Rates.  We also need to find the list of employees that didn't submit timesheets.

Also need to collect "Total Hours" and "Total Project Hours" to save for later calculations.

Calculate Total Monthly Hours

In [228]:
TotalHoursbyEmployee = TimeDataDF[['Employee ID', 'Project or Admin', 'Reported Quantity']]

In [229]:
TotalHoursbyEmployee = TotalHoursbyEmployee[['Employee ID', 'Reported Quantity']]

In [230]:
TotalHoursbyEmployee = TotalHoursbyEmployee.groupby(['Employee ID'], as_index=False).sum()

In [231]:
TotalHoursbyEmployee.rename(columns={"Reported Quantity": "Total Monthly Hours"}, inplace=True)

In [232]:
TimeDataDF = pd.merge(TimeDataDF, TotalHoursbyEmployee, on = 'Employee ID', how = 'left')

In [233]:
len(TimeDataDF)

33112

Calculate Total Project Hours

In [234]:
TimeDataDF.columns

Index(['Worker', 'Employee ID', 'Worker Cost Center', 'Talent Pool',
       'Project Group', 'Utilization Target', 'Hire Date', 'Termination Date',
       'Title', 'Hourly Rate', 'Monthly Salary', 'Monthly Bonus Amount',
       'Bonus %', 'Week', 'Reported Date', 'Time Type', 'Entry Type',
       'Project', 'Project Plan Task', 'Project Plan Task ID', 'Cost Center',
       'Initiative', 'Lobbying', 'Lobbying Location', 'Reported Quantity',
       'Time Cost Center', 'Time Initiative', 'MonthYear', 'Adjusted Hours',
       'Needed Project Hours', 'Project or Admin', 'Total Monthly Hours'],
      dtype='object')

In [235]:
ProjectHoursbyEmployee = TimeDataDF[['Employee ID', 'Project or Admin', 'Reported Quantity', 'Adjusted Hours']]

In [236]:
NeededHoursbyEmployee = TimeDataDF[['Employee ID','Needed Project Hours', 'Adjusted Hours']].drop_duplicates()

In [237]:
len(ProjectHoursbyEmployee)

33112

In [238]:
ProjectHoursbyEmployee = ProjectHoursbyEmployee[ProjectHoursbyEmployee['Project or Admin']=='Project']

In [239]:
ProjectHoursbyEmployee = ProjectHoursbyEmployee[['Employee ID', 'Reported Quantity']]

In [240]:
ProjectHoursbyEmployee = ProjectHoursbyEmployee.groupby(['Employee ID'], as_index=False).sum()

In [241]:
ProjectHoursbyEmployee

,Employee ID,Reported Quantity
0,1,96.0
1,4,120.0
2,5,152.0
3,6,104.0
4,7,120.0
...,...,...
695,2052,77.0
696,2053,25.0
697,2054,72.0
698,2057,1.0


In [242]:
ProjectHoursbyEmployee.rename(columns={"Reported Quantity": "Total Project Hours"}, inplace=True)

In [243]:
ProjectHoursbyEmployee = pd.merge(ProjectHoursbyEmployee, NeededHoursbyEmployee, on = 'Employee ID', how = 'left')

In [244]:
ProjectHoursbyEmployee

,Employee ID,Total Project Hours,Needed Project Hours,Adjusted Hours
0,1,96.0,92.4,168.0
1,4,120.0,109.2,168.0
2,5,152.0,100.8,168.0
3,6,104.0,109.2,168.0
4,7,120.0,109.2,168.0
...,...,...,...,...
695,2052,77.0,84.0,120.0
696,2053,25.0,28.0,40.0
697,2054,72.0,56.0,80.0
698,2057,1.0,28.0,40.0


In [245]:
HoursTable = ProjectHoursbyEmployee

In [246]:
ProjectHoursbyEmployee['Actual Utilization Rate'] = ProjectHoursbyEmployee['Total Project Hours']/ProjectHoursbyEmployee['Adjusted Hours']

In [247]:
UtilizationRateTable = ProjectHoursbyEmployee[['Employee ID',  'Actual Utilization Rate', 'Total Project Hours']]

In [248]:
ActualRateTable = ProjectHoursbyEmployee[['Employee ID',  'Actual Utilization Rate']]

In [249]:
TimeDataDF = pd.merge(TimeDataDF, UtilizationRateTable, on = 'Employee ID', how = 'left')

In [250]:
TimeDataDF['Total Project Hours'].fillna(0, inplace=True)
TimeDataDF['Actual Utilization Rate'].fillna(0, inplace=True)

Here we need to update a dataset to isolate the timesheets not submitted or "Fully Utilized" employees with no Project Time

In [251]:
TimeDataDF['Fully Utilized'] = np.where(TimeDataDF['Utilization Target'] > TimeDataDF['Actual Utilization Rate'], "Underutilized", np.where(TimeDataDF['Utilization Target'] <=  TimeDataDF['Actual Utilization Rate'], "Fully Utilized",'NA')) 

In [252]:
TimeDataDF.columns

Index(['Worker', 'Employee ID', 'Worker Cost Center', 'Talent Pool',
       'Project Group', 'Utilization Target', 'Hire Date', 'Termination Date',
       'Title', 'Hourly Rate', 'Monthly Salary', 'Monthly Bonus Amount',
       'Bonus %', 'Week', 'Reported Date', 'Time Type', 'Entry Type',
       'Project', 'Project Plan Task', 'Project Plan Task ID', 'Cost Center',
       'Initiative', 'Lobbying', 'Lobbying Location', 'Reported Quantity',
       'Time Cost Center', 'Time Initiative', 'MonthYear', 'Adjusted Hours',
       'Needed Project Hours', 'Project or Admin', 'Total Monthly Hours',
       'Actual Utilization Rate', 'Total Project Hours', 'Fully Utilized'],
      dtype='object')

In [253]:
TimeDataDFforLookup = TimeDataDF
TimeDataDFforLookup['Total Employee Project Hours'] = TimeDataDFforLookup.groupby(['Employee ID','Project'])['Reported Quantity'].transform('sum')



In [254]:
TimeDataDFforLookup.columns
EmployeeTimeProjectLookupTable = TimeDataDFforLookup[['Employee ID', 'Talent Pool', 'Worker', 'Worker Cost Center', 'Project', 'Hire Date', 'Termination Date', 'Needed Project Hours','Total Employee Project Hours','Project Group', 'Actual Utilization Rate', 'Utilization Target', 'Total Project Hours', 'Fully Utilized']].drop_duplicates()

In [255]:
TimesheetsSubmittedDF = TimeDataDF[['Employee ID', 'Fully Utilized','Needed Project Hours', 'Total Project Hours']].drop_duplicates()

In [256]:
TimesheetsSubmittedDF['Timesheet Submitted'] = "Yes"

In [257]:
SalaryCompare = pd.merge(AllSalariesFinal, TimesheetsSubmittedDF, on = 'Employee ID', how = 'left')

Now Combine this list with TimeSheets Not Submitted Since the Result is the same

In [258]:
TimesheetsNotSubmitted1 = SalaryCompare[SalaryCompare['Timesheet Submitted'] != "Yes"]

TimesheetsNotSubmitted2 = SalaryCompare[(SalaryCompare['Total Project Hours'] == 0) & (SalaryCompare['Fully Utilized']=="Fully Utilized")]

TimesheetsNotSubmitted = pd.concat([TimesheetsNotSubmitted2, TimesheetsNotSubmitted1], ignore_index=True)


In [259]:
TimesheetsNotSubmitted1.to_excel('./Outputs/timesheets.xlsx')

In [260]:
TimesheetsNotSubmitted1

,Employee,Employee ID,Net Amount,Cost Center,Category,Hire Date,Termination Date,Location,Available Days,Bonus Final,...,Adjusted Hours,Project Plan Task Home,Project Plan Task ID Home,Initiative,Project Plan Task PTO,Project Plan Task ID PTO,Fully Utilized,Needed Project Hours,Total Project Hours,Timesheet Submitted
64,Amar Shah,913.0,5048.16,Carbon-Free Buildings,Regular,03/02/2020,10/31/2040,Remote - California,21,1130.972417,...,168.0,Admin > General,PROJECT_PLAN_TASK-3-2271,BLD - Admin,Carbon-Free Buildings > PTO,PROJECT_PLAN_TASK-3-5718,NaN,NaN,NaN,NaN
81,Andrew Waddell (On Leave),1711.0,4180.13,Climate Aligned Industries,Regular,01/17/2023,10/31/2040,Remote - California,21,879.166667,...,168.0,Admin > General,PROJECT_PLAN_TASK-3-2343,CAI - Admin,Climate Aligned Industries > PTO,PROJECT_PLAN_TASK-3-5723,NaN,NaN,NaN,NaN
113,Ben Webster,1553.0,15416.68,MiQ Foundation,Regular,06/06/2022,10/31/2040,Remote - Colorado,21,0.000000,...,168.0,General > General,PROJECT_PLAN_TASK-3-4631,MiQ - Admin,MiQ > PTO,PROJECT_PLAN_TASK-3-5730,NaN,NaN,NaN,NaN
313,James Mitchell (On Leave),645.0,17263.55,Finance,Regular,07/01/2021,10/31/2040,Remote-United Kingdom,21,2985.877333,...,168.0,Admin > General,PROJECT_PLAN_TASK-3-2415,GCF - Admin,Finance > PTO,PROJECT_PLAN_TASK-3-5726,NaN,NaN,NaN,NaN
316,James Sun (On Leave),1716.0,6437.76,Climate Aligned Industries,Regular,01/17/2023,10/31/2040,Remote - Michigan,21,321.875000,...,168.0,Admin > General,PROJECT_PLAN_TASK-3-2343,CAI - Admin,Climate Aligned Industries > PTO,PROJECT_PLAN_TASK-3-5723,NaN,NaN,NaN,NaN
330,Jennifer Grizzard Ekzarkhov,1394.0,11058.13,Development,Regular,01/10/2022,10/31/2040,Remote - Maryland,21,1105.811417,...,168.0,Admin > General,PROJECT_PLAN_TASK-3-2181,DEV - Admin,Development > PTO,PROJECT_PLAN_TASK-3-5770,NaN,NaN,NaN,NaN
333,Jeremy Wendt,1015.0,12114.17,Org Finance,Regular,01/19/2021,10/31/2040,Remote - Colorado,21,1211.415167,...,168.0,Admin > General,PROJECT_PLAN_TASK-3-1652,ACT - Accounting,Accounting > PTO,PROJECT_PLAN_TASK-3-5772,NaN,NaN,NaN,NaN
365,Julia Thayne (Terminated),1308.0,7113.08,Climate Aligned Industries,Regular,09/13/2021,09/30/2024,Remote - California,20,0.000000,...,160.0,Admin > General,PROJECT_PLAN_TASK-3-2343,CAI - Admin,Climate Aligned Industries > PTO,PROJECT_PLAN_TASK-3-5723,NaN,NaN,NaN,NaN
409,Lara Owens,1008.0,13333.35,MiQ Foundation,Regular,12/15/2020,10/31/2040,Remote - Pennsylvania,21,0.000000,...,168.0,General > General,PROJECT_PLAN_TASK-3-4631,MiQ - Admin,MiQ > PTO,PROJECT_PLAN_TASK-3-5730,NaN,NaN,NaN,NaN
427,Leslyn Simmons | Lyn Simmons,1466.0,12795.43,People Team,Regular,04/11/2022,10/31/2040,Remote - Virginia,21,2559.083333,...,168.0,Admin > General,PROJECT_PLAN_TASK-3-2235,HRS - Admin,People Team > PTO,PROJECT_PLAN_TASK-3-5769,NaN,NaN,NaN,NaN


In [261]:
TimesheetsNotSubmitted['Termination Date'].fillna('10-31-2040', inplace=True)

# Add Bonuses for Unpaid Employees to Timesheets not Submitted

And finally print out the final Employees Dataset

In [262]:
AllSalariesFinal = pd.merge(AllSalariesFinal, TimesheetsSubmittedDF, on = 'Employee ID', how = 'left')

In [263]:
len(AllSalariesFinal)

740

In [264]:
AllSalariesFinal.columns

Index(['Employee', 'Employee ID', 'Net Amount', 'Cost Center', 'Category',
       'Hire Date', 'Termination Date', 'Location', 'Available Days',
       'Bonus Final', 'Benefits per Employee', 'Shared per Employee',
       'FileType', 'Consulting Per Employee', 'FTE', 'Adjusted Hours',
       'Project Plan Task Home', 'Project Plan Task ID Home', 'Initiative',
       'Project Plan Task PTO', 'Project Plan Task ID PTO', 'Fully Utilized',
       'Needed Project Hours', 'Total Project Hours', 'Timesheet Submitted'],
      dtype='object')

In [265]:
len(AllSalariesFinal)

740

In [266]:
AllSalariesFinal['Bonus Final'].fillna(0, inplace = True)
AllSalariesFinal['Net Amount'].fillna(0, inplace = True)
AllSalariesFinal['Benefits per Employee'].fillna(0, inplace = True)
AllSalariesFinal['Shared per Employee'].fillna(0, inplace = True)

In [267]:
AllSalariesFinal['Total Cost'] = AllSalariesFinal['Net Amount'] + AllSalariesFinal['Bonus Final'] + AllSalariesFinal['Benefits per Employee'] + AllSalariesFinal['Shared per Employee'] + AllSalariesFinal['Consulting Per Employee']

In [268]:
AllSalariesFinal['Hours for Project Hour Calc'] = np.where(AllSalariesFinal['Total Project Hours'] > AllSalariesFinal['Needed Project Hours'], AllSalariesFinal['Total Project Hours'], AllSalariesFinal['Needed Project Hours'])

In [269]:
AllSalariesFinal['Cost per Project Hour'] = AllSalariesFinal['Total Cost']/AllSalariesFinal['Hours for Project Hour Calc']

In [270]:
AllSalariesFinal['Cost per Project Hour'].fillna(AllSalariesFinal['Total Cost']/AllSalariesFinal['Needed Project Hours'], inplace = True)

In [271]:
AllSalariesFinal['Termination Date'].fillna('10-31-2040', inplace=True)

In [272]:
EmployeeCostperHourTable = AllSalariesFinal[['Employee ID', 'Cost per Project Hour']]

In [273]:
AllSalariesFinal['Standard Monthly Cost'] = AllSalariesFinal['Total Cost'] / AllSalariesFinal['Needed Project Hours']
StandardCostTable = AllSalariesFinal[['Employee ID', 'Standard Monthly Cost']]

Final List of Salaries for Employees that did not submit timesheets is here:

In [274]:
#AllSalariesFinal = pd.merge(AllSalariesFinal, NeededHoursbyEmployee, on = 'Employee ID', how = 'left')

In [275]:
TimeDataDF['Fully Utilized'].value_counts()

Fully Utilized    25052
Underutilized      8060
Name: Fully Utilized, dtype: int64

# Calculate Project Split for Fully Utilized Employees

In [276]:
FullyUtilizedProgramTime = TimeDataDF[(TimeDataDF['Project or Admin']=='Project') & (TimeDataDF['Fully Utilized']=="Fully Utilized") & (TimeDataDF['Total Project Hours'] != 0)]

In [277]:
FullyUtilizedProgramTime['Percent of Total Project Hours'] = FullyUtilizedProgramTime['Reported Quantity'] / FullyUtilizedProgramTime['Total Project Hours']

C:\Users\Jwendt\AppData\Local\Temp\ipykernel_20800\1968767996.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FullyUtilizedProgramTime['Percent of Total Project Hours'] = FullyUtilizedProgramTime['Reported Quantity'] / FullyUtilizedProgramTime['Total Project Hours']


# Allocate Fully Utilized Employees to Tasks and Print Outcome

In [278]:
AllSalariesFinal.columns

Index(['Employee', 'Employee ID', 'Net Amount', 'Cost Center', 'Category',
       'Hire Date', 'Termination Date', 'Location', 'Available Days',
       'Bonus Final', 'Benefits per Employee', 'Shared per Employee',
       'FileType', 'Consulting Per Employee', 'FTE', 'Adjusted Hours',
       'Project Plan Task Home', 'Project Plan Task ID Home', 'Initiative',
       'Project Plan Task PTO', 'Project Plan Task ID PTO', 'Fully Utilized',
       'Needed Project Hours', 'Total Project Hours', 'Timesheet Submitted',
       'Total Cost', 'Hours for Project Hour Calc', 'Cost per Project Hour',
       'Standard Monthly Cost'],
      dtype='object')

In [279]:
AllSalariesFinal = pd.merge(AllSalariesFinal, UtilizationList, on = 'Employee ID', how = 'left')

In [280]:
len(AllSalariesFinal)

740

In [281]:
AllSalariesFinal.columns

Index(['Employee', 'Employee ID', 'Net Amount', 'Cost Center', 'Category',
       'Hire Date', 'Termination Date', 'Location', 'Available Days',
       'Bonus Final', 'Benefits per Employee', 'Shared per Employee',
       'FileType', 'Consulting Per Employee', 'FTE', 'Adjusted Hours',
       'Project Plan Task Home', 'Project Plan Task ID Home', 'Initiative',
       'Project Plan Task PTO', 'Project Plan Task ID PTO', 'Fully Utilized',
       'Needed Project Hours', 'Total Project Hours', 'Timesheet Submitted',
       'Total Cost', 'Hours for Project Hour Calc', 'Cost per Project Hour',
       'Standard Monthly Cost', 'Utilization Target', 'Talent Pools'],
      dtype='object')

In [282]:
CostperProjectHourTable = AllSalariesFinal[['Employee ID', 'Cost per Project Hour']]
CostperProjectHourTable

,Employee ID,Cost per Project Hour
0,1841.0,78.388092
1,2028.0,68.966896
2,7.0,119.712493
3,1275.0,83.319093
4,1395.0,77.846877
...,...,...
735,617.0,NaN
736,628.0,NaN
737,813.0,NaN
738,868.0,NaN


## Add Final Cats Needed to All Salaries Final

In [283]:
CostingDataFullyUtilized = AllSalariesFinal[['Employee ID', 'Net Amount', 'Category', 'Bonus Final', 'Benefits per Employee',
       'Shared per Employee', 'Adjusted Hours', 'Project Plan Task ID Home', 'Project Plan Task Home', 'Cost Center', 'Initiative']]

In [284]:
CostingDataFullyUtilized = CostingDataFullyUtilized.rename(columns={'Cost Center': 'Home Cost Center', 'Initiative': 'Home Initiative', })

In [285]:
len(CostingDataFullyUtilized)

740

In [286]:
CostingDataFullyUtilized['Employee ID'] = CostingDataFullyUtilized['Employee ID'].astype(int)

In [287]:
FullyUtilizedProgramTime = FullyUtilizedProgramTime[['Worker', 'Employee ID', 'Worker Cost Center', 'Talent Pool',
       'Project Group', 'Utilization Target', 'Hire Date', 'Termination Date',
       'Title', 'Hourly Rate', 'Monthly Salary', 'Monthly Bonus Amount',
       'Bonus %', 'Week', 'Reported Date', 'Time Type', 'Entry Type',
       'Project', 'Project Plan Task', 'Project Plan Task ID', 'Cost Center',
       'Initiative', 'Lobbying', 'Lobbying Location', 'Reported Quantity',
       'Needed Project Hours', 'Project or Admin', 'Total Monthly Hours',
       'Actual Utilization Rate', 'Total Project Hours', 'Fully Utilized',
       'Percent of Total Project Hours']]

In [288]:
FullyUtilizedProgramTime = pd.merge(FullyUtilizedProgramTime, CostingDataFullyUtilized, on = 'Employee ID', how = 'left')

In [289]:
FullyUtilizedProgramTime['SalaryCosted'] = FullyUtilizedProgramTime['Percent of Total Project Hours'] * FullyUtilizedProgramTime['Net Amount']
FullyUtilizedProgramTime['BenefitsCosted'] = FullyUtilizedProgramTime['Percent of Total Project Hours'] * FullyUtilizedProgramTime['Benefits per Employee']
FullyUtilizedProgramTime['SharedCosted'] = FullyUtilizedProgramTime['Percent of Total Project Hours'] * FullyUtilizedProgramTime['Shared per Employee']
FullyUtilizedProgramTime['BonusCosted'] = FullyUtilizedProgramTime['Percent of Total Project Hours'] * FullyUtilizedProgramTime['Bonus Final']

In [290]:
FullyUtilizedProgramTime.columns

Index(['Worker', 'Employee ID', 'Worker Cost Center', 'Talent Pool',
       'Project Group', 'Utilization Target', 'Hire Date', 'Termination Date',
       'Title', 'Hourly Rate', 'Monthly Salary', 'Monthly Bonus Amount',
       'Bonus %', 'Week', 'Reported Date', 'Time Type', 'Entry Type',
       'Project', 'Project Plan Task', 'Project Plan Task ID', 'Cost Center',
       'Initiative', 'Lobbying', 'Lobbying Location', 'Reported Quantity',
       'Needed Project Hours', 'Project or Admin', 'Total Monthly Hours',
       'Actual Utilization Rate', 'Total Project Hours', 'Fully Utilized',
       'Percent of Total Project Hours', 'Net Amount', 'Category',
       'Bonus Final', 'Benefits per Employee', 'Shared per Employee',
       'Adjusted Hours', 'Project Plan Task ID Home', 'Project Plan Task Home',
       'Home Cost Center', 'Home Initiative', 'SalaryCosted', 'BenefitsCosted',
       'SharedCosted', 'BonusCosted'],
      dtype='object')

In [291]:
FullyUtilizedProgramTime = FullyUtilizedProgramTime[['Project', 'Worker Cost Center','Worker', 'Employee ID', 'Project Plan Task', 'Project Plan Task ID', 'SalaryCosted', 'BenefitsCosted',
       'SharedCosted', 'BonusCosted', 'Cost Center', 'Initiative', 'Lobbying', 'Lobbying Location', 'Project or Admin']]

FullyUtilizedProgramTime['Project Hours Percentage'] = '1'
FullyUtilizedProgramTime['Admin Hours Percentage'] = '0'

In [292]:
FullyUtilizedProgramTime = FullyUtilizedProgramTime.groupby(['Worker', 'Worker Cost Center','Employee ID', 'Project', 'Project Plan Task', 'Project Plan Task ID', 'Project Hours Percentage', 'Admin Hours Percentage', 'Cost Center', 'Initiative', 'Lobbying', 'Lobbying Location', 'Project or Admin'], as_index=False).sum()

In [293]:
FullyUtilizedProgramTime['Type'] = "Fully Utilized"

# Calculate Project Split for Under Utilized Employees

## Do Project Allocations first

In [294]:
UnderUtilizedProgramTime = TimeDataDF[(TimeDataDF['Fully Utilized']=="Underutilized") & (TimeDataDF['Project or Admin'] == 'Project')]

In [295]:
UnderUtilizedProgramTime['Percent of Total Project Hours'] = UnderUtilizedProgramTime['Reported Quantity'] / UnderUtilizedProgramTime['Total Project Hours']
UnderUtilizedProgramTime['Project Hours Percentage'] = UnderUtilizedProgramTime['Total Project Hours'] / UnderUtilizedProgramTime['Needed Project Hours']
UnderUtilizedProgramTime['Admin Hours Percentage'] = 1 - UnderUtilizedProgramTime['Project Hours Percentage']

C:\Users\Jwendt\AppData\Local\Temp\ipykernel_20800\1922050398.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  UnderUtilizedProgramTime['Percent of Total Project Hours'] = UnderUtilizedProgramTime['Reported Quantity'] / UnderUtilizedProgramTime['Total Project Hours']
C:\Users\Jwendt\AppData\Local\Temp\ipykernel_20800\1922050398.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  UnderUtilizedProgramTime['Project Hours Percentage'] = UnderUtilizedProgramTime['Total Project Hours'] / UnderUtilizedProgramTi

# Create Master Utilization Rates Table for UnderUtilized so I can Calculate the percentage of Salary to Push to Admin

In [296]:
AllPercentages = UnderUtilizedProgramTime[['Admin Hours Percentage', 'Project Hours Percentage', 'Employee ID']].drop_duplicates()

In [297]:
UnderUtilizedALL = TimeDataDF[(TimeDataDF['Fully Utilized']=="Underutilized")]
UnderUtilizedALLIDs = UnderUtilizedALL[["Worker", "Employee ID"]].drop_duplicates()

In [298]:
AdminNumbers = pd.merge(UnderUtilizedALLIDs, AllPercentages, on = 'Employee ID', how = 'left')

AdminNumbers['Admin Hours Percentage'].fillna(1, inplace = True)
AdminNumbers['Project Hours Percentage'].fillna(0, inplace = True)

AdminNumbers

,Worker,Employee ID,Admin Hours Percentage,Project Hours Percentage
0,Adefunke Sonaike | Addy Sonaike,1029,0.107143,0.892857
1,Adrienne Tecza,1417,1.000000,0.000000
2,Aimee Schwartze,981,0.084249,0.915751
3,Aldine Francois,1740,0.389881,0.610119
4,Alex Axthelm,1530,0.560440,0.439560
...,...,...,...,...
175,Xinxin Hu,1288,0.128401,0.871599
176,Yu Ann Tan,989,0.319728,0.680272
177,Yuning Liu,872,0.084249,0.915751
178,Zihe Meng,424,0.065934,0.934066


# Continue Calculation for Project Allocations

In [299]:
UnderUtilizedProgramTime = pd.merge(UnderUtilizedProgramTime, CostingDataFullyUtilized, on = 'Employee ID', how = 'left')

In [300]:
UnderUtilizedProgramTime['SalaryCosted'] = np.where(UnderUtilizedProgramTime['Project or Admin']=="Project", UnderUtilizedProgramTime['Percent of Total Project Hours'] * UnderUtilizedProgramTime['Net Amount'] * UnderUtilizedProgramTime['Project Hours Percentage'],0)
UnderUtilizedProgramTime['BenefitsCosted'] = np.where(UnderUtilizedProgramTime['Project or Admin']=="Project", UnderUtilizedProgramTime['Percent of Total Project Hours'] * UnderUtilizedProgramTime['Benefits per Employee'] * UnderUtilizedProgramTime['Project Hours Percentage'],0)
UnderUtilizedProgramTime['SharedCosted'] = np.where(UnderUtilizedProgramTime['Project or Admin']=="Project", UnderUtilizedProgramTime['Percent of Total Project Hours'] * UnderUtilizedProgramTime['Shared per Employee'] * UnderUtilizedProgramTime['Project Hours Percentage'],0)
UnderUtilizedProgramTime['BonusCosted'] = np.where(UnderUtilizedProgramTime['Project or Admin']=="Project", UnderUtilizedProgramTime['Percent of Total Project Hours'] * UnderUtilizedProgramTime['Bonus Final'] * UnderUtilizedProgramTime['Project Hours Percentage'],0)

In [301]:
UnderUtilizedProgramProjectTime = UnderUtilizedProgramTime[UnderUtilizedProgramTime['Project or Admin'] == 'Project']

UnderUtilizedProgramProjectTime['PTO more than Admin'] = 'NA'

In [302]:
UnderUtilizedProgramTimeFormatted = UnderUtilizedProgramProjectTime[['Project','Worker Cost Center', 'Worker', 'Employee ID', 'Project Plan Task', 'Project Plan Task ID', 'SalaryCosted', 'BenefitsCosted',
       'SharedCosted', 'BonusCosted', 'Project Hours Percentage', 'Admin Hours Percentage', 'Cost Center', 'Initiative', 'Lobbying', 'Lobbying Location', 'Project or Admin', 'PTO more than Admin']]

In [303]:
UnderUtilizedProgramTimeFormatted = UnderUtilizedProgramTimeFormatted.groupby(['Worker', 'Worker Cost Center', 'Employee ID', 'Project', 'Project Plan Task', 'Project Plan Task ID', 'Project Hours Percentage', 'Admin Hours Percentage','Cost Center', 'Initiative', 'Lobbying', 'Lobbying Location', 'Project or Admin', 'PTO more than Admin'], as_index=False).sum()

In [304]:
UnderUtilizedProgramTimeFormatted["Type"] = "UnderUtilizedProject"

# Do Admin Allocations

## Build list of Admin Time per Employee UnderUtilized

First build split between PTO and Admin

In [305]:
Admintable = TimeDataDFALL[TimeDataDFALL['Project or Admin'] == 'Admin']
MonthAdminTable = Admintable[Admintable['MonthYear']==currentmonthandyear]
PTOtable = MonthAdminTable[(MonthAdminTable['Project Group']=='PTO') | (MonthAdminTable['Project Group']=='Holiday')]

In [306]:
MonthAdminTable['Project Group'].value_counts()

Admin      5789
PTO        1366
Holiday     642
Name: Project Group, dtype: int64

In [307]:


PTOtable = PTOtable[['Employee ID', 'Reported Quantity']]
PTOtable = PTOtable.groupby(['Employee ID'], as_index=False).sum()
PTOtable = PTOtable.rename(columns={'Reported Quantity': 'Total Monthly PTO Hours'})


AdminHoursbyEmployee = MonthAdminTable[['Employee ID', 'Reported Quantity']]
AdminHoursbyEmployee = AdminHoursbyEmployee.groupby(['Employee ID'], as_index=False).sum()
AdminHoursbyEmployee = AdminHoursbyEmployee.rename(columns={'Reported Quantity': 'Total Monthly Admin Hours'})


AdminHoursbyEmployee = pd.merge(AdminHoursbyEmployee, PTOtable, on = 'Employee ID', how = 'left')

AdminHoursbyEmployee['Total Monthly PTO Hours'].fillna(0, inplace=True)
AdminHoursbyEmployee['Percent of Admin PTO'] = AdminHoursbyEmployee['Total Monthly PTO Hours'] / AdminHoursbyEmployee['Total Monthly Admin Hours']




In [308]:
AdminHoursbyEmployee

,Employee ID,Total Monthly Admin Hours,Total Monthly PTO Hours,Percent of Admin PTO
0,1,72.0,8.0,0.111111
1,4,48.0,8.0,0.166667
2,5,16.0,16.0,1.000000
3,6,64.0,48.0,0.750000
4,7,48.0,8.0,0.166667
...,...,...,...,...
694,2050,9.5,8.0,0.842105
695,2051,14.0,0.0,0.000000
696,2052,43.0,0.0,0.000000
697,2053,3.0,0.0,0.000000


In [309]:
UnderUtilizedAdminCalcs = pd.merge(AdminNumbers, AdminHoursbyEmployee, on = 'Employee ID', how = 'left')
UnderUtilizedAdminCalcs['Percent of Admin PTO'].fillna(0, inplace=True)

In [310]:
len(UnderUtilizedAdminCalcs)

180

In [311]:

#CostperProjectHourTable and CostingDataFullyUtilized should be 767
len(CostingDataFullyUtilized)

740

In [312]:
UnderUtilizedAdminCalcs = pd.merge(UnderUtilizedAdminCalcs, CostingDataFullyUtilized, on = 'Employee ID', how = 'left')

In [313]:
len(UnderUtilizedAdminCalcs)

180

In [314]:
UnderUtilizedAdminCalcs = pd.merge(UnderUtilizedAdminCalcs, CostperProjectHourTable, on = 'Employee ID', how = 'left')

In [315]:
len(CostperProjectHourTable)

740

In [316]:
len(UnderUtilizedAdminCalcs)

180

In [317]:
UnderUtilizedAdminCalcs['PTO MAX'] = UnderUtilizedAdminCalcs['Total Monthly PTO Hours'] * UnderUtilizedAdminCalcs['Cost per Project Hour'] 
UnderUtilizedAdminCalcs['PTO MAX'].fillna(0, inplace = True)

UnderUtilizedAdminCalcs['ALLCOMP'] = UnderUtilizedAdminCalcs['Net Amount'] + UnderUtilizedAdminCalcs['Bonus Final'] + UnderUtilizedAdminCalcs['Benefits per Employee'] + UnderUtilizedAdminCalcs['Shared per Employee']
UnderUtilizedAdminCalcs['Total Admin'] = UnderUtilizedAdminCalcs['ALLCOMP'] *  UnderUtilizedAdminCalcs['Admin Hours Percentage']
UnderUtilizedAdminCalcs['PTO more than Admin'] = np.where(UnderUtilizedAdminCalcs['PTO MAX'] > UnderUtilizedAdminCalcs['Total Admin'], "PTO Discount","PTO Project Hour")

UnderUtilizedAdminCalcs['Salarypercent'] = UnderUtilizedAdminCalcs['Net Amount']/UnderUtilizedAdminCalcs['ALLCOMP']
UnderUtilizedAdminCalcs['Bonuspercent'] = UnderUtilizedAdminCalcs['Bonus Final']/UnderUtilizedAdminCalcs['ALLCOMP']
UnderUtilizedAdminCalcs['Benefitspercent'] = UnderUtilizedAdminCalcs['Benefits per Employee']/UnderUtilizedAdminCalcs['ALLCOMP']
UnderUtilizedAdminCalcs['Sharedpercent'] = UnderUtilizedAdminCalcs['Shared per Employee']/UnderUtilizedAdminCalcs['ALLCOMP']

UnderUtilizedAdminCalcs['PTO more than Admin'] = np.where(UnderUtilizedAdminCalcs['PTO MAX'] > UnderUtilizedAdminCalcs['Total Admin'], "PTO Discount","PTO Project Hour")


UnderUtilizedAdminCalcs['SalaryCostedPTO'] = np.where(UnderUtilizedAdminCalcs['PTO more than Admin'] == "PTO Project Hour", UnderUtilizedAdminCalcs['Salarypercent'] *  UnderUtilizedAdminCalcs['PTO MAX'], UnderUtilizedAdminCalcs['Salarypercent'] *  UnderUtilizedAdminCalcs['Total Admin'])
UnderUtilizedAdminCalcs['BenefitsCostedPTO'] = np.where(UnderUtilizedAdminCalcs['PTO more than Admin'] == "PTO Project Hour", UnderUtilizedAdminCalcs['Benefitspercent'] *  UnderUtilizedAdminCalcs['PTO MAX'], UnderUtilizedAdminCalcs['Benefitspercent'] *  UnderUtilizedAdminCalcs['Total Admin'])
UnderUtilizedAdminCalcs['SharedCostedPTO'] = np.where(UnderUtilizedAdminCalcs['PTO more than Admin'] == "PTO Project Hour", UnderUtilizedAdminCalcs['Sharedpercent'] *  UnderUtilizedAdminCalcs['PTO MAX'], UnderUtilizedAdminCalcs['Sharedpercent'] *  UnderUtilizedAdminCalcs['Total Admin']) 
UnderUtilizedAdminCalcs['BonusCostedPTO'] = np.where(UnderUtilizedAdminCalcs['PTO more than Admin'] == "PTO Project Hour", UnderUtilizedAdminCalcs['Bonuspercent'] *  UnderUtilizedAdminCalcs['PTO MAX'], UnderUtilizedAdminCalcs['Bonuspercent'] *  UnderUtilizedAdminCalcs['Total Admin']) 

In [318]:
len(UnderUtilizedAdminCalcs)

180

In [319]:
UnderUtilizedAdminCalcs['SalaryCostedAdmin'] = np.where(UnderUtilizedAdminCalcs['PTO more than Admin'] == "PTO Project Hour", (UnderUtilizedAdminCalcs['Total Admin'] - UnderUtilizedAdminCalcs['PTO MAX']) *  UnderUtilizedAdminCalcs['Salarypercent'],0) 
UnderUtilizedAdminCalcs['BenefitsCostedAdmin'] = np.where(UnderUtilizedAdminCalcs['PTO more than Admin'] == "PTO Project Hour", (UnderUtilizedAdminCalcs['Total Admin'] - UnderUtilizedAdminCalcs['PTO MAX']) *  UnderUtilizedAdminCalcs['Benefitspercent'],0) 
UnderUtilizedAdminCalcs['SharedCostedAdmin'] = np.where(UnderUtilizedAdminCalcs['PTO more than Admin'] == "PTO Project Hour", (UnderUtilizedAdminCalcs['Total Admin'] - UnderUtilizedAdminCalcs['PTO MAX']) *  UnderUtilizedAdminCalcs['Sharedpercent'],0) 
UnderUtilizedAdminCalcs['BonusCostedAdmin'] = np.where(UnderUtilizedAdminCalcs['PTO more than Admin'] == "PTO Project Hour", (UnderUtilizedAdminCalcs['Total Admin'] - UnderUtilizedAdminCalcs['PTO MAX']) *  UnderUtilizedAdminCalcs['Bonuspercent'],0) 

In [320]:
AllSalariesFinal.columns

Index(['Employee', 'Employee ID', 'Net Amount', 'Cost Center', 'Category',
       'Hire Date', 'Termination Date', 'Location', 'Available Days',
       'Bonus Final', 'Benefits per Employee', 'Shared per Employee',
       'FileType', 'Consulting Per Employee', 'FTE', 'Adjusted Hours',
       'Project Plan Task Home', 'Project Plan Task ID Home', 'Initiative',
       'Project Plan Task PTO', 'Project Plan Task ID PTO', 'Fully Utilized',
       'Needed Project Hours', 'Total Project Hours', 'Timesheet Submitted',
       'Total Cost', 'Hours for Project Hour Calc', 'Cost per Project Hour',
       'Standard Monthly Cost', 'Utilization Target', 'Talent Pools'],
      dtype='object')

In [321]:
PTOTaskLookup = AllSalariesFinal[['Employee ID', 'Project Plan Task PTO', 'Project Plan Task ID PTO']]

PTOTaskLookup['Employee ID'].fillna('NA', inplace=True)
PTOTaskLookup['Project Plan Task PTO'].fillna('NA', inplace=True)
PTOTaskLookup['Project Plan Task ID PTO'].fillna('NA', inplace=True)



C:\Users\Jwendt\AppData\Local\anaconda3\envs\new_env_name\lib\site-packages\pandas\core\series.py:4530: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [322]:
len(UnderUtilizedAdminCalcs)

180

In [323]:
UnderUtilizedAdminCalcs = pd.merge(UnderUtilizedAdminCalcs, PTOTaskLookup, on = 'Employee ID', how = 'left')



In [324]:
len(UnderUtilizedAdminCalcs)
len(PTOTaskLookup)

740

In [325]:
UnderUtilizedAdminCalcs.columns

Index(['Worker', 'Employee ID', 'Admin Hours Percentage',
       'Project Hours Percentage', 'Total Monthly Admin Hours',
       'Total Monthly PTO Hours', 'Percent of Admin PTO', 'Net Amount',
       'Category', 'Bonus Final', 'Benefits per Employee',
       'Shared per Employee', 'Adjusted Hours', 'Project Plan Task ID Home',
       'Project Plan Task Home', 'Home Cost Center', 'Home Initiative',
       'Cost per Project Hour', 'PTO MAX', 'ALLCOMP', 'Total Admin',
       'PTO more than Admin', 'Salarypercent', 'Bonuspercent',
       'Benefitspercent', 'Sharedpercent', 'SalaryCostedPTO',
       'BenefitsCostedPTO', 'SharedCostedPTO', 'BonusCostedPTO',
       'SalaryCostedAdmin', 'BenefitsCostedAdmin', 'SharedCostedAdmin',
       'BonusCostedAdmin', 'Project Plan Task PTO',
       'Project Plan Task ID PTO'],
      dtype='object')

In [326]:
UnderUtilizedPTOCalcs = UnderUtilizedAdminCalcs[['Worker', 'Employee ID', 'Project Plan Task PTO',
       'Project Plan Task ID PTO', 'SalaryCostedPTO', 'BenefitsCostedPTO',
       'SharedCostedPTO', 'BonusCostedPTO', 'Admin Hours Percentage',
       'Project Hours Percentage', 'Home Cost Center',
       'Home Initiative', 'PTO more than Admin']]

In [327]:
UnderUtilizedPTOCalcs = UnderUtilizedPTOCalcs.groupby(['Worker', 'Employee ID', 'Project Plan Task PTO', 'Project Plan Task ID PTO', 'Admin Hours Percentage',
       'Project Hours Percentage', 'Home Cost Center',
       'Home Initiative', 'PTO more than Admin'], as_index=False).sum()

In [328]:
UnderUtilizedPTOCalcs.columns

Index(['Worker', 'Employee ID', 'Project Plan Task PTO',
       'Project Plan Task ID PTO', 'Admin Hours Percentage',
       'Project Hours Percentage', 'Home Cost Center', 'Home Initiative',
       'PTO more than Admin', 'SalaryCostedPTO', 'BenefitsCostedPTO',
       'SharedCostedPTO', 'BonusCostedPTO'],
      dtype='object')

In [329]:
UnderUtilizedAdminCalcs = UnderUtilizedAdminCalcs[['Worker', 'Employee ID', 'Project Plan Task Home', 'Project Plan Task ID Home', 'SalaryCostedAdmin', 'BenefitsCostedAdmin',
       'SharedCostedAdmin', 'BonusCostedAdmin', 'Admin Hours Percentage',
       'Project Hours Percentage', 'Home Cost Center',
       'Home Initiative', 'PTO more than Admin']]

In [330]:
UnderUtilizedAdminCalcs = UnderUtilizedAdminCalcs.groupby(['Worker', 'Employee ID', 'Project Plan Task Home', 'Project Plan Task ID Home', 'Admin Hours Percentage',
       'Project Hours Percentage', 'Home Cost Center',
       'Home Initiative', 'PTO more than Admin'], as_index=False).sum()

In [331]:
UnderUtilizedPTOCalcs = UnderUtilizedPTOCalcs.rename(columns={'Project Plan Task PTO': 'Project Plan Task', 'Project Plan Task ID PTO': 'Project Plan Task ID', 'SalaryCostedPTO' : 'SalaryCosted', 'BenefitsCostedPTO' : 'BenefitsCosted', 'SharedCostedPTO' : 'SharedCosted',
       'BonusCostedPTO': 'BonusCosted'})

UnderUtilizedAdminCalcs = UnderUtilizedAdminCalcs.rename(columns={'Project Plan Task Home': 'Project Plan Task', 'Project Plan Task ID Home': 'Project Plan Task ID', 'SalaryCostedAdmin': 'SalaryCosted', 'BenefitsCostedAdmin' : 'BenefitsCosted',
       'SharedCostedAdmin' : 'SharedCosted', 'BonusCostedAdmin' : 'BonusCosted'})

In [332]:
UnderUtilizedAdminCalcs['Type'] = "Admin Time for UnderUtilized"
UnderUtilizedPTOCalcs['Type'] = "PTO Time for UnderUtilized"

In [333]:
UnderUtilizedAdminCalcs.rename(columns={'Home Cost Center': 'Cost Center', 'Home Initiative': 'Initiative'}, inplace=True)
UnderUtilizedPTOCalcs.rename(columns={'Home Cost Center': 'Cost Center', 'Home Initiative': 'Initiative'}, inplace=True)

In [334]:
UnderUtilizedAdminCalcs['Lobbying'] = "None"
UnderUtilizedAdminCalcs['Lobbying Location'] = "None"
UnderUtilizedAdminCalcs['Project or Admin'] = "Admin"
UnderUtilizedAdminCalcs['Project'] = "Admin"
UnderUtilizedAdminCalcs['Worker Cost Center'] = UnderUtilizedAdminCalcs['Cost Center'] 

In [335]:
UnderUtilizedAdminCalcs

,Worker,Employee ID,Project Plan Task,Project Plan Task ID,Admin Hours Percentage,Project Hours Percentage,Cost Center,Initiative,PTO more than Admin,SalaryCosted,BenefitsCosted,SharedCosted,BonusCosted,Type,Lobbying,Lobbying Location,Project or Admin,Project,Worker Cost Center
0,Adefunke Sonaike | Addy Sonaike,1029,Admin > General,PROJECT_PLAN_TASK-3-2559,0.107143,0.892857,US,US - Admin,PTO Discount,0.000000,0.000000,0.000000,0.000000,Admin Time for UnderUtilized,None,None,Admin,Admin,US
1,Adrienne Tecza,1417,Admin > General,PROJECT_PLAN_TASK-3-2379,1.000000,0.000000,Climate Intelligence,CIP - Admin,PTO Project Hour,3019.651633,1917.563121,590.831613,260.746990,Admin Time for UnderUtilized,None,None,Admin,Admin,Climate Intelligence
2,Aimee Schwartze,981,Admin > General,PROJECT_PLAN_TASK-3-2253,0.084249,0.915751,Africa,AF - Admin,PTO Discount,0.000000,0.000000,0.000000,0.000000,Admin Time for UnderUtilized,None,None,Admin,Admin,Africa
3,Aldine Francois,1740,Admin > General,PROJECT_PLAN_TASK-3-2145,0.389881,0.610119,Influence Team,COM - Admin,PTO Project Hour,1692.711042,877.505537,270.373375,84.635417,Admin Time for UnderUtilized,None,None,Admin,Admin,Influence Team
4,Alex Axthelm,1530,Admin > General,PROJECT_PLAN_TASK-3-2415,0.560440,0.439560,Finance,GCF - Admin,PTO Discount,0.000000,0.000000,0.000000,0.000000,Admin Time for UnderUtilized,None,None,Admin,Admin,Finance
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,Xinxin Hu,1288,Admin > General,PROJECT_PLAN_TASK-3-2271,0.128401,0.871599,Carbon-Free Buildings,BLD - Admin,PTO Discount,0.000000,0.000000,0.000000,0.000000,Admin Time for UnderUtilized,None,None,Admin,Admin,Carbon-Free Buildings
176,Yu Ann Tan,989,Admin > General,PROJECT_PLAN_TASK-3-2271,0.319728,0.680272,Carbon-Free Buildings,BLD - Admin,PTO Discount,0.000000,0.000000,0.000000,0.000000,Admin Time for UnderUtilized,None,None,Admin,Admin,Carbon-Free Buildings
177,Yuning Liu,872,Admin > General,PROJECT_PLAN_TASK-3-2559,0.084249,0.915751,US,US - Admin,PTO Discount,0.000000,0.000000,0.000000,0.000000,Admin Time for UnderUtilized,None,None,Admin,Admin,US
178,Zihe Meng,424,Admin > General,PROJECT_PLAN_TASK-3-2253,0.065934,0.934066,Africa,AF - Admin,PTO Discount,0.000000,0.000000,0.000000,0.000000,Admin Time for UnderUtilized,None,None,Admin,Admin,Africa


In [336]:
UnderUtilizedPTOCalcs['Lobbying'] = "None"
UnderUtilizedPTOCalcs['Lobbying Location'] = "None"
UnderUtilizedPTOCalcs['Project or Admin'] = "Admin"
UnderUtilizedPTOCalcs['Project'] = "Admin"
UnderUtilizedPTOCalcs['Worker Cost Center'] = UnderUtilizedAdminCalcs['Cost Center'] 

In [337]:
UnderUtilizedPTOCalcs = UnderUtilizedPTOCalcs[UnderUtilizedPTOCalcs['SalaryCosted']!=0]

In [338]:
UnderUtilizedCombined = pd.concat([UnderUtilizedAdminCalcs, UnderUtilizedPTOCalcs, UnderUtilizedProgramTimeFormatted], ignore_index=True)

In [339]:
UnderUtilizedPTOCalcs

,Worker,Employee ID,Project Plan Task,Project Plan Task ID,Admin Hours Percentage,Project Hours Percentage,Cost Center,Initiative,PTO more than Admin,SalaryCosted,BenefitsCosted,SharedCosted,BonusCosted,Type,Lobbying,Lobbying Location,Project or Admin,Project,Worker Cost Center
0,Adefunke Sonaike | Addy Sonaike,1029,US Program > PTO,PROJECT_PLAN_TASK-3-5735,0.107143,0.892857,US,US - Admin,PTO Discount,791.091429,347.145048,106.960896,39.554518,PTO Time for UnderUtilized,None,None,Admin,Admin,US
1,Adrienne Tecza,1417,Climate Intelligence > PTO,PROJECT_PLAN_TASK-3-5722,1.000000,0.000000,Climate Intelligence,CIP - Admin,PTO Project Hour,2082.518367,1322.457325,407.470078,179.825510,PTO Time for UnderUtilized,None,None,Admin,Admin,Climate Intelligence
2,Aimee Schwartze,981,Africa > PTO,PROJECT_PLAN_TASK-3-5719,0.084249,0.915751,Africa,AF - Admin,PTO Discount,584.659158,272.968755,84.106003,29.232853,PTO Time for UnderUtilized,None,None,Admin,Admin,Africa
3,Aldine Francois,1740,Influence Team > PTO,PROJECT_PLAN_TASK-3-5737,0.389881,0.610119,Influence Team,COM - Admin,PTO Project Hour,744.048810,385.716720,118.845439,37.202381,PTO Time for UnderUtilized,None,None,Admin,Admin,Influence Team
4,Alex Axthelm,1530,Finance > PTO,PROJECT_PLAN_TASK-3-5726,0.560440,0.439560,Finance,GCF - Admin,PTO Discount,5333.378242,1815.835634,559.487761,247.737638,PTO Time for UnderUtilized,None,None,Admin,Admin,Finance
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,Xinxin Hu,1288,Carbon-Free Buildings > PTO,PROJECT_PLAN_TASK-3-5718,0.128401,0.871599,Carbon-Free Buildings,BLD - Admin,PTO Discount,986.972466,416.023033,128.183295,49.348329,PTO Time for UnderUtilized,None,None,Admin,Admin,Carbon-Free Buildings
176,Yu Ann Tan,989,Carbon-Free Buildings > PTO,PROJECT_PLAN_TASK-3-5718,0.319728,0.680272,Carbon-Free Buildings,BLD - Admin,PTO Discount,2679.984762,1035.924904,319.184895,133.998825,PTO Time for UnderUtilized,None,None,Admin,Admin,Carbon-Free Buildings
177,Yuning Liu,872,US Program > PTO,PROJECT_PLAN_TASK-3-5735,0.084249,0.915751,US,US - Admin,PTO Discount,788.311941,272.968755,84.106003,78.831117,PTO Time for UnderUtilized,None,None,Admin,Admin,US
178,Zihe Meng,424,Africa > PTO,PROJECT_PLAN_TASK-3-5719,0.065934,0.934066,Africa,AF - Admin,PTO Discount,639.518242,213.627722,65.822090,63.951462,PTO Time for UnderUtilized,None,None,Admin,Admin,Africa


In [340]:
UnderUtilizedCombined['Type'].value_counts()

UnderUtilizedProject            845
Admin Time for UnderUtilized    180
PTO Time for UnderUtilized      160
Name: Type, dtype: int64

In [341]:
#UnderUtilizedCombined.to_excel('/mnt/c/Users/jerem/anaconda3/envs/Code/RMICODING/Data/TimeCodingData/Outputs/UnderUtilizedCombined.xlsx')

# Put All time for Employees that Didn't submit Timesheets to Admin

Add logic to put in Cost per hour and Utilization Rates that Didn't submit Timesheets as well.

In [342]:
TimesheetsNotSubmitted.columns

Index(['Employee', 'Employee ID', 'Net Amount', 'Cost Center', 'Category',
       'Hire Date', 'Termination Date', 'Location', 'Available Days',
       'Bonus Final', 'Benefits per Employee', 'Shared per Employee',
       'FileType', 'Consulting Per Employee', 'FTE', 'Adjusted Hours',
       'Project Plan Task Home', 'Project Plan Task ID Home', 'Initiative',
       'Project Plan Task PTO', 'Project Plan Task ID PTO', 'Fully Utilized',
       'Needed Project Hours', 'Total Project Hours', 'Timesheet Submitted'],
      dtype='object')

In [343]:
TimesheetsNotSubmitted['Fully Utilized'] = "No Timesheet"

In [344]:
TimesheetsNotSubmitted.rename(columns={'Employee': 'Worker', 'Project Ref ID': 'Project Plan Task ID', 'Admin Project': 'Project Plan Task', 'Net Amount': 'SalaryCosted', 'Benefits per Employee': 'BenefitsCosted', 'Shared per Employee': 'SharedCosted', 'Bonus Final': 'BonusCosted', 'Consulting Per Employee': 'ConsultingFeesCosted'}, inplace=True)

In [345]:
TimesheetsNotSubmitted = TimesheetsNotSubmitted[['Worker', 'Employee ID', 'Project Plan Task Home', 'Project Plan Task ID Home', 'SalaryCosted', 'BenefitsCosted',
       'SharedCosted', 'BonusCosted', 'Initiative', 'Cost Center', 'ConsultingFeesCosted']]

In [346]:
TimesheetsNotSubmitted["Type"] = "TimeSheetsNotSubmitted"

Combine all Costed Timesheets and Print

In [347]:
TimesheetsNotSubmitted['Project Hours Percentage'] = '0'
TimesheetsNotSubmitted['Admin Hours Percentage'] = '1'

In [348]:
TimesheetsNotSubmitted = TimesheetsNotSubmitted.rename(columns={'Project Plan Task Home': 'Project Plan Task', 'Project Plan Task ID Home': 'Project Plan Task ID', })

In [349]:
TimesheetsNotSubmitted['Lobbying'] = "None"
TimesheetsNotSubmitted['Lobbying Location'] = "None"
TimesheetsNotSubmitted['Project or Admin'] = "Admin"
TimesheetsNotSubmitted['Project'] = "Admin"
TimesheetsNotSubmitted['Worker Cost Center'] = TimesheetsNotSubmitted['Cost Center']

In [350]:
TimesheetsNotSubmitted.columns

Index(['Worker', 'Employee ID', 'Project Plan Task', 'Project Plan Task ID',
       'SalaryCosted', 'BenefitsCosted', 'SharedCosted', 'BonusCosted',
       'Initiative', 'Cost Center', 'ConsultingFeesCosted', 'Type',
       'Project Hours Percentage', 'Admin Hours Percentage', 'Lobbying',
       'Lobbying Location', 'Project or Admin', 'Project',
       'Worker Cost Center'],
      dtype='object')

In [351]:
TimesheetsNotSubmitted['PTO more than Admin'] = 'NA'

Move tasks to "No Timesheet Submitted Task"

In [352]:
TimesheetsNotSubmitted = TimesheetsNotSubmitted[['Worker', 'Employee ID',
       'SalaryCosted', 'BenefitsCosted', 'SharedCosted', 'BonusCosted',
       'Initiative', 'Cost Center', 'ConsultingFeesCosted', 'Type',
       'Project Hours Percentage', 'Admin Hours Percentage', 'Lobbying',
       'Lobbying Location', 'Project or Admin', 'Project',
       'Worker Cost Center']]

In [353]:
Notimesheetaccounts = './Source_Files/NotimeSheetMapping.xlsx'

Notimesheetaccounts = pd.read_excel(Notimesheetaccounts, engine='openpyxl')

In [354]:
Notimesheetaccounts.columns

Index(['Project', 'Project Hierarchy - Level 02', 'Project Ref ID',
       'External Project Reference', 'Cost Center', 'Cost Center Ref ID',
       'Initiative', 'Initiative Ref ID', 'Project Phase', 'Project Task',
       'Project Plan Task', 'Project Plan Task ID', 'Project Plan Task WID'],
      dtype='object')

In [355]:
Notimesheetaccounts = Notimesheetaccounts[['Cost Center',
       'Project Plan Task', 'Project Plan Task ID']]
Notimesheetaccounts = Notimesheetaccounts.rename(columns={'Cost Center': 'Worker Cost Center'})
Notimesheetaccounts.columns

Index(['Worker Cost Center', 'Project Plan Task', 'Project Plan Task ID'], dtype='object')

In [356]:
TimesheetsNotSubmitted = pd.merge(TimesheetsNotSubmitted, Notimesheetaccounts, on = 'Worker Cost Center', how = 'left')

In [357]:
TimesheetsNotSubmitted.columns

Index(['Worker', 'Employee ID', 'SalaryCosted', 'BenefitsCosted',
       'SharedCosted', 'BonusCosted', 'Initiative', 'Cost Center',
       'ConsultingFeesCosted', 'Type', 'Project Hours Percentage',
       'Admin Hours Percentage', 'Lobbying', 'Lobbying Location',
       'Project or Admin', 'Project', 'Worker Cost Center',
       'Project Plan Task', 'Project Plan Task ID'],
      dtype='object')

In [358]:

UnderUtilizedCombined['ConsultingFeesCosted'] = 0
FullyUtilizedProgramTime['ConsultingFeesCosted'] = 0
FullyUtilizedProgramTime['PTO more than Admin'] = 'NA'
IndiaTimeMasterAllocated['PTO more than Admin'] = 'NA'

C:\Users\Jwendt\AppData\Local\Temp\ipykernel_20800\3658725513.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  IndiaTimeMasterAllocated['PTO more than Admin'] = 'NA'


In [359]:
AllCostsAllocated = pd.concat([TimesheetsNotSubmitted, UnderUtilizedCombined, FullyUtilizedProgramTime, IndiaTimeMasterAllocated], ignore_index=True)

In [360]:
AllCostsAllocated = pd.merge(AllCostsAllocated, LocationbyEmployee, on = 'Employee ID', how = 'left')

In [361]:
LocationbyEmployee

,Employee ID,Location
0,1,Beijing
1,4,Beijing
2,5,Beijing
3,6,Beijing
4,7,Beijing
...,...,...
1106,1985,Remote - North Carolina
1107,1991,Remote - North Carolina
1108,1992,Remote - North Carolina
1109,1993,Remote - Arizona


In [362]:
UtilizationRatesTalentPoolTable = AllSalariesFinal[['Employee ID','Talent Pools', 'Utilization Target']]
UtilizationRatesTalentPoolTable['Employee ID'] = UtilizationRatesTalentPoolTable['Employee ID'].astype(int)

C:\Users\Jwendt\AppData\Local\Temp\ipykernel_20800\3315622932.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  UtilizationRatesTalentPoolTable['Employee ID'] = UtilizationRatesTalentPoolTable['Employee ID'].astype(int)


In [363]:
UtilizationRatesTalentPoolTable = UtilizationRatesTalentPoolTable.drop_duplicates()

In [364]:
len(UtilizationRatesTalentPoolTable)

740

In [365]:
UtilizationRatesTalentPoolTable.dtypes

Employee ID             int32
Talent Pools           object
Utilization Target    float64
dtype: object

In [366]:
UtilizationRatesTalentPoolTable.dropna(subset=['Talent Pools'], inplace=True)

In [367]:
len(UtilizationRatesTalentPoolTable)

734

In [368]:
MissingTimeSheetIDs = TimesheetsNotSubmitted[['Employee ID']]

In [369]:
MissingTimeSheetIDs = pd.merge(MissingTimeSheetIDs, UtilizationList, on = 'Employee ID', how = 'left')

In [370]:
MissingTimeSheetIDs['Talent Pools'] = 'Missing'

In [371]:
MissingTimeSheetIDs

,Employee ID,Utilization Target,Talent Pools
0,913.0,0.65,Missing
1,1711.0,0.65,Missing
2,1553.0,0.60,Missing
3,645.0,0.60,Missing
4,1716.0,0.70,Missing
5,1394.0,0.65,Missing
6,1015.0,0.65,Missing
7,1308.0,0.60,Missing
8,1008.0,0.60,Missing
9,1466.0,0.60,Missing


In [372]:
len(AllCostsAllocated)

4819

In [373]:
HoursTable

,Employee ID,Total Project Hours,Needed Project Hours,Adjusted Hours,Actual Utilization Rate
0,1,96.0,92.4,168.0,0.571429
1,4,120.0,109.2,168.0,0.714286
2,5,152.0,100.8,168.0,0.904762
3,6,104.0,109.2,168.0,0.619048
4,7,120.0,109.2,168.0,0.714286
...,...,...,...,...,...
695,2052,77.0,84.0,120.0,0.641667
696,2053,25.0,28.0,40.0,0.625000
697,2054,72.0,56.0,80.0,0.900000
698,2057,1.0,28.0,40.0,0.025000


In [374]:
MissingTimeSheetIDs['Needed Project Hours'] = MissingTimeSheetIDs['Utilization Target'] * AvailableHours
MissingTimeSheetIDs['Total Project Hours'] = 0
MissingTimeSheetIDs['Adjusted Hours'] = 0
MissingTimeSheetIDs['Actual Utilization Rate'] = 0





In [375]:
MissingTimeSheetIDs = MissingTimeSheetIDs[['Employee ID', 'Needed Project Hours', 'Total Project Hours', 'Adjusted Hours', 'Actual Utilization Rate']]

In [376]:
HoursTable2 = pd.concat([HoursTable, MissingTimeSheetIDs], ignore_index=True)

In [377]:
len(AllCostsAllocated)

4819

In [378]:
AllCostsAllocated = pd.merge(AllCostsAllocated, HoursTable2, on = 'Employee ID', how = 'left')

In [379]:
len(AllCostsAllocated)

4819

In [380]:
AllCostsAllocated = pd.merge(AllCostsAllocated, EmployeeCostperHourTable, on = 'Employee ID', how = 'left')

In [381]:
len(AllCostsAllocated)

4819

In [382]:
AllCostsAllocated = pd.merge(AllCostsAllocated, UtilizationList, on = 'Employee ID', how = 'left')

In [383]:
AllCostsAllocated['SalaryCosted'] = AllCostsAllocated['SalaryCosted'].round(2)
AllCostsAllocated['BenefitsCosted'] = AllCostsAllocated['BenefitsCosted'].round(2)
AllCostsAllocated['SharedCosted'] = AllCostsAllocated['SharedCosted'].round(2)
AllCostsAllocated['BonusCosted'] = AllCostsAllocated['BonusCosted'].round(2)
AllCostsAllocated['ConsultingFeesCosted'] = AllCostsAllocated['ConsultingFeesCosted'].round(2)

In [384]:
AllCostsAllocated['BonusCosted'].fillna(0, inplace = True)
AllCostsAllocated['ConsultingFeesCosted'].fillna(0, inplace = True)

In [385]:
AllCostsAllocated['Total Cost'] = AllCostsAllocated['SalaryCosted'] + AllCostsAllocated['BenefitsCosted'] + AllCostsAllocated['SharedCosted'] + AllCostsAllocated['BonusCosted'] + AllCostsAllocated['ConsultingFeesCosted']

In [386]:
AllCostsAllocated['Admin Hours Percentage'] = pd.to_numeric(AllCostsAllocated['Admin Hours Percentage'])

In [387]:
UtilizationList

,Employee ID,Utilization Target,Talent Pools
0,1,0.55,Managing Director
1,4,0.65,Manager
2,5,0.60,Principal
3,6,0.65,Manager
4,7,0.65,Manager
...,...,...,...
1106,1985,0.00,NaN
1107,1991,0.00,NaN
1108,1992,0.00,NaN
1109,1993,0.00,NaN


In [388]:
AllCostsAllocated['Cost per Project Hour'] = np.where(AllCostsAllocated['Talent Pools']=='Missing', AllCostsAllocated['Total Cost'] / AllCostsAllocated['Needed Project Hours'], AllCostsAllocated['Cost per Project Hour'])

In [389]:
AllCostsAllocated['Cost per Project Hour'].value_counts()

137.631052    132
57.786636     108
65.878042      97
392.335813     83
68.801867      68
             ... 
187.159858      1
84.202512       1
92.285555       1
64.159515       1
102.397809      1
Name: Cost per Project Hour, Length: 710, dtype: int64

In [390]:
AllCostsAllocated['Cost per Project Hour'].fillna(0, inplace=True)

In [391]:
AllCostsAllocated['File'] = Type

In [392]:
len(AllCostsAllocated)

4819

In [393]:
AllCostsAllocated['Type'].value_counts()

Fully Utilized                  2540
India                           1070
UnderUtilizedProject             845
Admin Time for UnderUtilized     180
PTO Time for UnderUtilized       160
TimeSheetsNotSubmitted            24
Name: Type, dtype: int64

In [394]:
AllCostsAllocated['Project PTO or Admin'] = np.where(AllCostsAllocated['Type'] == 'PTO Time for UnderUtilized', 'PTO', np.where(AllCostsAllocated['Type'] == 'Admin Time for UnderUtilized', 'Admin', np.where(AllCostsAllocated['Type'] == 'TimeSheetsNotSubmitted', 'Admin','Project')))

In [395]:
AllCostsAllocated['Project PTO or Admin'].value_counts()

Project    4455
Admin       204
PTO         160
Name: Project PTO or Admin, dtype: int64

In [396]:

AllCostsAllocated = pd.merge(AllCostsAllocated, StandardCostTable, on = 'Employee ID', how = 'left')

In [397]:
AllCostsAllocated['Amount Banked per Hour'] = AllCostsAllocated['Standard Monthly Cost'] - AllCostsAllocated['Cost per Project Hour'] 

In [398]:
AllCostsAllocated.dtypes

Worker                       object
Employee ID                 float64
SalaryCosted                float64
BenefitsCosted              float64
SharedCosted                float64
BonusCosted                 float64
Initiative                   object
Cost Center                  object
ConsultingFeesCosted        float64
Type                         object
Project Hours Percentage     object
Admin Hours Percentage      float64
Lobbying                     object
Lobbying Location            object
Project or Admin             object
Project                      object
Worker Cost Center           object
Project Plan Task            object
Project Plan Task ID         object
PTO more than Admin          object
Location                     object
Total Project Hours         float64
Needed Project Hours        float64
Adjusted Hours              float64
Actual Utilization Rate     float64
Cost per Project Hour       float64
Utilization Target          float64
Talent Pools                

In [399]:
AllCostsAllocated['Hours'] = AllCostsAllocated['Total Cost'] / AllCostsAllocated['Cost per Project Hour']
AllCostsAllocated['Hours'] = AllCostsAllocated['Hours'].round(2)

In [400]:
AllCostsAllocated['Amount Banked'] = AllCostsAllocated['Hours'] * AllCostsAllocated['Amount Banked per Hour']

In [401]:
AllCostsAllocated = pd.merge(AllCostsAllocated, PTOtable, on = 'Employee ID', how = 'left')

In [402]:
AllSalariesFinal.columns

Index(['Employee', 'Employee ID', 'Net Amount', 'Cost Center', 'Category',
       'Hire Date', 'Termination Date', 'Location', 'Available Days',
       'Bonus Final', 'Benefits per Employee', 'Shared per Employee',
       'FileType', 'Consulting Per Employee', 'FTE', 'Adjusted Hours',
       'Project Plan Task Home', 'Project Plan Task ID Home', 'Initiative',
       'Project Plan Task PTO', 'Project Plan Task ID PTO', 'Fully Utilized',
       'Needed Project Hours', 'Total Project Hours', 'Timesheet Submitted',
       'Total Cost', 'Hours for Project Hour Calc', 'Cost per Project Hour',
       'Standard Monthly Cost', 'Utilization Target', 'Talent Pools'],
      dtype='object')

In [403]:
AllCostsAllocated.columns

Index(['Worker', 'Employee ID', 'SalaryCosted', 'BenefitsCosted',
       'SharedCosted', 'BonusCosted', 'Initiative', 'Cost Center',
       'ConsultingFeesCosted', 'Type', 'Project Hours Percentage',
       'Admin Hours Percentage', 'Lobbying', 'Lobbying Location',
       'Project or Admin', 'Project', 'Worker Cost Center',
       'Project Plan Task', 'Project Plan Task ID', 'PTO more than Admin',
       'Location', 'Total Project Hours', 'Needed Project Hours',
       'Adjusted Hours', 'Actual Utilization Rate', 'Cost per Project Hour',
       'Utilization Target', 'Talent Pools', 'Total Cost', 'File',
       'Project PTO or Admin', 'Standard Monthly Cost',
       'Amount Banked per Hour', 'Hours', 'Amount Banked',
       'Total Monthly PTO Hours'],
      dtype='object')

In [404]:
AllCostsAllocated['Cost per PTO Hour'] = np.where(AllCostsAllocated['Project PTO or Admin'] == 'PTO', AllCostsAllocated['Total Cost']/AllCostsAllocated['Total Monthly PTO Hours'],0)

In [405]:
AllCostsAllocated['Project & PTO Hours'] = np.where(AllCostsAllocated['Project PTO or Admin'] == 'Project', AllCostsAllocated['Hours'],np.where(AllCostsAllocated['Project PTO or Admin'] == 'PTO', AllCostsAllocated['Total Monthly PTO Hours'],0))

In [406]:
AllCostsAllocated.columns

Index(['Worker', 'Employee ID', 'SalaryCosted', 'BenefitsCosted',
       'SharedCosted', 'BonusCosted', 'Initiative', 'Cost Center',
       'ConsultingFeesCosted', 'Type', 'Project Hours Percentage',
       'Admin Hours Percentage', 'Lobbying', 'Lobbying Location',
       'Project or Admin', 'Project', 'Worker Cost Center',
       'Project Plan Task', 'Project Plan Task ID', 'PTO more than Admin',
       'Location', 'Total Project Hours', 'Needed Project Hours',
       'Adjusted Hours', 'Actual Utilization Rate', 'Cost per Project Hour',
       'Utilization Target', 'Talent Pools', 'Total Cost', 'File',
       'Project PTO or Admin', 'Standard Monthly Cost',
       'Amount Banked per Hour', 'Hours', 'Amount Banked',
       'Total Monthly PTO Hours', 'Cost per PTO Hour', 'Project & PTO Hours'],
      dtype='object')

In [407]:
AllCostsAllocated = AllCostsAllocated[['Worker', 'Employee ID', 'Project Plan Task', 'Project Plan Task ID',
       'SalaryCosted', 'BenefitsCosted', 'SharedCosted', 'BonusCosted',
       'Initiative', 'Cost Center', 'ConsultingFeesCosted', 'Type',
       'Project Hours Percentage', 'Admin Hours Percentage', 'Lobbying',
       'Lobbying Location', 'Project or Admin', 'Project',
       'Worker Cost Center', 'PTO more than Admin', 'Location',
       'Total Project Hours', 'Needed Project Hours', 'Adjusted Hours',
       'Actual Utilization Rate', 'Cost per Project Hour',
       'Utilization Target', 'Talent Pools', 'Total Cost', 'File',
       'Project PTO or Admin', 'Standard Monthly Cost',
       'Amount Banked per Hour', 'Amount Banked', 'Cost per PTO Hour', 'Project & PTO Hours']]

In [408]:
#AllCostsAllocated['Employee ID'] = AllCostsAllocated['Employee ID'].apply(add_zeros)

In [409]:
AllCostsAllocated.to_excel('./Outputs/AllCostsAllocated'+Type+timestampreport+'.xlsx')

# Create Hours Report Data Table

In [410]:
allsalariestotime = AllSalariesFinal[['Employee', 'Employee ID', 'Cost Center', 'Category',
       'Hire Date', 'Termination Date', 'Location', 'Available Days','FileType', 'FTE', 'Adjusted Hours', 'Project Plan Task Home',
       'Project Plan Task ID Home', 'Initiative', 'Fully Utilized',
       'Total Project Hours', 'Timesheet Submitted', 'Talent Pools',
       'Utilization Target']]

In [411]:
EmployeeTimeProjectLookupTable.columns

Index(['Employee ID', 'Talent Pool', 'Worker', 'Worker Cost Center', 'Project',
       'Hire Date', 'Termination Date', 'Needed Project Hours',
       'Total Employee Project Hours', 'Project Group',
       'Actual Utilization Rate', 'Utilization Target', 'Total Project Hours',
       'Fully Utilized'],
      dtype='object')

In [412]:
AllSalariesFinal['Needed Project Hours'].fillna(.1, inplace=True)
AllSalariesFinal['Total Project Hours'].fillna(0, inplace=True)
AllSalariesFinal['Timesheet Submitted'].fillna('No', inplace=True)


In [413]:
AllSalariesFinal['Percent Utilized'] = AllSalariesFinal['Total Project Hours']/AllSalariesFinal['Needed Project Hours']

In [414]:
AllSalariesFinal['On Leave'] = AllSalariesFinal['Employee'].str.contains("On Leave")

In [415]:
AllSalariesFinal['Bank per Hour'] = AllSalariesFinal['Standard Monthly Cost'] - AllSalariesFinal['Cost per Project Hour']

In [416]:
AllSalariesFinal['Total Banked'] = np.where(AllSalariesFinal['Fully Utilized'] == "Fully Utilized", AllSalariesFinal['Total Project Hours'] * AllSalariesFinal['Bank per Hour'], 0)

In [417]:
AllSalariesFinal.columns

Index(['Employee', 'Employee ID', 'Net Amount', 'Cost Center', 'Category',
       'Hire Date', 'Termination Date', 'Location', 'Available Days',
       'Bonus Final', 'Benefits per Employee', 'Shared per Employee',
       'FileType', 'Consulting Per Employee', 'FTE', 'Adjusted Hours',
       'Project Plan Task Home', 'Project Plan Task ID Home', 'Initiative',
       'Project Plan Task PTO', 'Project Plan Task ID PTO', 'Fully Utilized',
       'Needed Project Hours', 'Total Project Hours', 'Timesheet Submitted',
       'Total Cost', 'Hours for Project Hour Calc', 'Cost per Project Hour',
       'Standard Monthly Cost', 'Utilization Target', 'Talent Pools',
       'Percent Utilized', 'On Leave', 'Bank per Hour', 'Total Banked'],
      dtype='object')

In [418]:
AllSalariesFinal['SalaryPercentage'] = AllSalariesFinal['Net Amount']/AllSalariesFinal['Total Cost']
AllSalariesFinal['BenefitsPercentage'] = AllSalariesFinal['Benefits per Employee']/AllSalariesFinal['Total Cost']
AllSalariesFinal['SharedPercentage'] = AllSalariesFinal['Shared per Employee']/AllSalariesFinal['Total Cost']
AllSalariesFinal['BonusPercentage'] = AllSalariesFinal['Bonus Final']/AllSalariesFinal['Total Cost']


In [419]:
salarypercentages_table = AllSalariesFinal[['Employee ID', 'SalaryPercentage', 'BenefitsPercentage', 'SharedPercentage', 'BonusPercentage']]

In [420]:
AllSalariesFinal['Employee ID'] = AllSalariesFinal['Employee ID'].apply(add_zeros)

In [421]:
AllSalariesFinal.to_excel('./Outputs/AllSalaries'+Type+timestampreport+'.xlsx')

In [422]:
EmployeeTimeProjectLookupTable.to_excel('./Outputs/ProjectTimeTable'+Type+timestampreport+'.xlsx')

# Create PTO Bank File

In [423]:
PTOBank = AllCostsAllocated

In [424]:
PTOBank["Bank Month"] = timestampreport

In [425]:
FundingSourceRules = './Source_Files/Find_Funding_Source_Rule_w_Project_Task_10.11.24.xlsx'

FundingSourceRulesData = pd.read_excel(FundingSourceRules, header=1, engine='openpyxl')

In [426]:
FundingSourceRulesData.columns

Index(['Project Plan Task ID', 'Project Task', 'Project ID',
       'Funding Source Rule', 'Project/Project Hierarchy',
       'Company/Company Hierarchy', 'Evaluation Date Option',
       'Default Funding Source', 'Inactive', 'Funding Source Condition Rule',
       'Funding Source Priority', 'Funding Source', 'Funding Source Count',
       'Funding Source Greater than 1', 'Detail 1', 'Detail 2', 'Detail 3',
       'Detail 4', '% - Detail 1', '% - Detail 2', '% - Detail 3',
       '% - Detail 4', 'Funding Source Priority Details'],
      dtype='object')

In [427]:
FundingSourceRulesData['Detail 1B'] = FundingSourceRulesData.groupby('Project ID')['Detail 1'].transform('first')
FundingSourceRulesData['Detail 2B'] = FundingSourceRulesData.groupby('Project ID')['Detail 2'].transform('first')
FundingSourceRulesData['Detail 3B'] = FundingSourceRulesData.groupby('Project ID')['Detail 3'].transform('first')
FundingSourceRulesData['Detail 4B'] = FundingSourceRulesData.groupby('Project ID')['Detail 4'].transform('first')
FundingSourceRulesData['% - Detail 1B'] = FundingSourceRulesData.groupby('Project ID')['% - Detail 1'].transform('first')
FundingSourceRulesData['% - Detail 2B'] = FundingSourceRulesData.groupby('Project ID')['% - Detail 2'].transform('first')
FundingSourceRulesData['% - Detail 3B'] = FundingSourceRulesData.groupby('Project ID')['% - Detail 3'].transform('first')
FundingSourceRulesData['% - Detail 4B'] = FundingSourceRulesData.groupby('Project ID')['% - Detail 4'].transform('first')



In [428]:
FundingSourceRulesData['Detail 1C'] = FundingSourceRulesData['Detail 1'].fillna(FundingSourceRulesData['Detail 1B'])
FundingSourceRulesData['Detail 2C'] = FundingSourceRulesData['Detail 2'].fillna(FundingSourceRulesData['Detail 2B'])
FundingSourceRulesData['Detail 3C'] = FundingSourceRulesData['Detail 3'].fillna(FundingSourceRulesData['Detail 3B'])
FundingSourceRulesData['Detail 4C'] = FundingSourceRulesData['Detail 4'].fillna(FundingSourceRulesData['Detail 4B'])
FundingSourceRulesData['% - Detail 1C'] = FundingSourceRulesData['% - Detail 1'].fillna(FundingSourceRulesData['% - Detail 1B'])
FundingSourceRulesData['% - Detail 2C'] = FundingSourceRulesData['% - Detail 2'].fillna(FundingSourceRulesData['% - Detail 2B'])
FundingSourceRulesData['% - Detail 3C'] = FundingSourceRulesData['% - Detail 3'].fillna(FundingSourceRulesData['% - Detail 3B'])
FundingSourceRulesData['% - Detail 4C'] = FundingSourceRulesData['% - Detail 4'].fillna(FundingSourceRulesData['% - Detail 4B'])

In [429]:
FundingSourceRulesData = FundingSourceRulesData[['Project Plan Task ID', 'Project Task', 'Project ID',
       'Funding Source Rule', 'Project/Project Hierarchy',
       'Company/Company Hierarchy', 'Evaluation Date Option',
       'Default Funding Source', 'Inactive', 'Funding Source Condition Rule',
       'Funding Source Priority', 'Funding Source', 'Funding Source Count',
       'Funding Source Greater than 1', 'Detail 1C', 'Detail 2C', 'Detail 3C',
       'Detail 4C', '% - Detail 1C', '% - Detail 2C', '% - Detail 3C',
       '% - Detail 4C']]

In [430]:
FundingSourceRulesData = FundingSourceRulesData.rename(columns={'Detail 1C': 'Detail 1', 'Detail 2C': 'Detail 2', 'Detail 3C': 'Detail 3', 'Detail 4C': 'Detail 4', '% - Detail 1C': '% - Detail 1', '% - Detail 2C': '% - Detail 2', '% - Detail 3C': '% - Detail 3', '% - Detail 4C': '% - Detail 4'})

In [431]:
FundingSourceRulesData.to_excel('./Outputs/FundingSourceRulesCLEANED'+Type+timestampreport+'.xlsx')

In [432]:
'''FundingSourceRulesLookup = FundingSourceRulesData[['Project Plan Task ID', 
       'Funding Source Count', 'Funding Source Greater than 1', 'Detail 1',
       'Detail 2', 'Detail 3', 'Detail 4', '% - Detail 1', '% - Detail 2',
       '% - Detail 3', '% - Detail 4']]
def return_second(x):
    try:
        y = x[1]
    except:
        y = ""
    return y

FundingSourceRulesLookup.dropna(subset=['Funding Source Count'], inplace=True)

FundingSourceRulesLookup['Funding Source 1'] = FundingSourceRulesLookup['Detail 1'].str.split(r"Funding Source - ").apply(return_second)

FundingSourceRulesLookup['Funding Source 2'] = FundingSourceRulesLookup['Detail 2'].str.split(r"Funding Source - ").apply(return_second)

try:
    FundingSourceRulesLookup['Funding Source 3'] = FundingSourceRulesLookup['Detail 3'].str.split(r"Funding Source - ").apply(return_second)
except:
    FundingSourceRulesLookup['Funding Source 3'] = ''

    
    FundingSourceRulesLookup = FundingSourceRulesLookup[['Project Plan Task ID', 'Funding Source Count',
       'Funding Source Greater than 1', 'Detail 1', 'Detail 2', 'Detail 3',
       'Detail 4', '% - Detail 1', '% - Detail 2', '% - Detail 3',
       '% - Detail 4', 'Funding Source 1',
       'Funding Source 2', 
       'Funding Source 3']]
       
FundingSourceReport = PTOBank       
PTOBank = PTOBank[PTOBank['Type']=='Fully Utilized']       
PTOBank = pd.merge(PTOBank, FundingSourceRulesLookup, on = 'Project Plan Task ID', how = 'left')
PTOBank['Amount Banked 1'] = PTOBank['Amount Banked'] * (PTOBank['% - Detail 1']/100)
PTOBank['Amount Banked 2'] = PTOBank['Amount Banked'] * (PTOBank['% - Detail 2']/100)
PTOBank['Amount Banked 3'] = PTOBank['Amount Banked'] * (PTOBank['% - Detail 3']/100)


       '''

'FundingSourceRulesLookup = FundingSourceRulesData[[\'Project Plan Task ID\', \n       \'Funding Source Count\', \'Funding Source Greater than 1\', \'Detail 1\',\n       \'Detail 2\', \'Detail 3\', \'Detail 4\', \'% - Detail 1\', \'% - Detail 2\',\n       \'% - Detail 3\', \'% - Detail 4\']]\ndef return_second(x):\n    try:\n        y = x[1]\n    except:\n        y = ""\n    return y\n\nFundingSourceRulesLookup.dropna(subset=[\'Funding Source Count\'], inplace=True)\n\nFundingSourceRulesLookup[\'Funding Source 1\'] = FundingSourceRulesLookup[\'Detail 1\'].str.split(r"Funding Source - ").apply(return_second)\n\nFundingSourceRulesLookup[\'Funding Source 2\'] = FundingSourceRulesLookup[\'Detail 2\'].str.split(r"Funding Source - ").apply(return_second)\n\ntry:\n    FundingSourceRulesLookup[\'Funding Source 3\'] = FundingSourceRulesLookup[\'Detail 3\'].str.split(r"Funding Source - ").apply(return_second)\nexcept:\n    FundingSourceRulesLookup[\'Funding Source 3\'] = \'\'\n\n    \n    Fundin

In [433]:
'''PTOBank1 = PTOBank[['Worker', 'Employee ID', 'Project Plan Task', 'Project Plan Task ID',
       'SalaryCosted', 'BenefitsCosted', 'SharedCosted', 'BonusCosted',
       'Initiative', 'Cost Center', 'Type', 'Project Hours Percentage',
       'Admin Hours Percentage', 'Lobbying', 'Lobbying Location',
       'Project or Admin', 'Project', 'Worker Cost Center', 'Location',
       'Total Project Hours', 'Needed Project Hours', 'Adjusted Hours',
       'Actual Utilization Rate', 'Cost per Project Hour',
       'Utilization Target', 'Talent Pools', 'Total Cost', 'File',
       'Project PTO or Admin', 'Standard Monthly Cost',
       'Amount Banked per Hour', 'Hours', 'Bank Month',
       'Funding Source Count', 'Funding Source Greater than 1', 'Detail 1', '% - Detail 1','Funding Source 1', 'Amount Banked 1']]

PTOBank1 = PTOBank1.rename(columns={'Detail 1': 'Detail', '% - Detail 1': '% - Detail', 'Funding Source 1': 'Funding Source', 'Amount Banked 1': 'Amount Banked'})'''

"PTOBank1 = PTOBank[['Worker', 'Employee ID', 'Project Plan Task', 'Project Plan Task ID',\n       'SalaryCosted', 'BenefitsCosted', 'SharedCosted', 'BonusCosted',\n       'Initiative', 'Cost Center', 'Type', 'Project Hours Percentage',\n       'Admin Hours Percentage', 'Lobbying', 'Lobbying Location',\n       'Project or Admin', 'Project', 'Worker Cost Center', 'Location',\n       'Total Project Hours', 'Needed Project Hours', 'Adjusted Hours',\n       'Actual Utilization Rate', 'Cost per Project Hour',\n       'Utilization Target', 'Talent Pools', 'Total Cost', 'File',\n       'Project PTO or Admin', 'Standard Monthly Cost',\n       'Amount Banked per Hour', 'Hours', 'Bank Month',\n       'Funding Source Count', 'Funding Source Greater than 1', 'Detail 1', '% - Detail 1','Funding Source 1', 'Amount Banked 1']]\n\nPTOBank1 = PTOBank1.rename(columns={'Detail 1': 'Detail', '% - Detail 1': '% - Detail', 'Funding Source 1': 'Funding Source', 'Amount Banked 1': 'Amount Banked'})"

In [434]:
'''PTOBank2 = PTOBank[['Worker', 'Employee ID', 'Project Plan Task', 'Project Plan Task ID',
       'SalaryCosted', 'BenefitsCosted', 'SharedCosted', 'BonusCosted',
       'Initiative', 'Cost Center', 'Type', 'Project Hours Percentage',
       'Admin Hours Percentage', 'Lobbying', 'Lobbying Location',
       'Project or Admin', 'Project', 'Worker Cost Center', 'Location',
       'Total Project Hours', 'Needed Project Hours', 'Adjusted Hours',
       'Actual Utilization Rate', 'Cost per Project Hour',
       'Utilization Target', 'Talent Pools', 'Total Cost', 'File',
       'Project PTO or Admin', 'Standard Monthly Cost',
       'Amount Banked per Hour', 'Hours', 'Bank Month',
       'Funding Source Count', 'Funding Source Greater than 1', 'Detail 2', '% - Detail 2','Funding Source 2', 'Amount Banked 2']]

PTOBank2 = PTOBank2.rename(columns={'Detail 2': 'Detail', '% - Detail 2': '% - Detail', 'Funding Source 2': 'Funding Source', 'Amount Banked 2': 'Amount Banked'})
PTOBank2.dropna(subset=['Amount Banked'], inplace=True)'''

"PTOBank2 = PTOBank[['Worker', 'Employee ID', 'Project Plan Task', 'Project Plan Task ID',\n       'SalaryCosted', 'BenefitsCosted', 'SharedCosted', 'BonusCosted',\n       'Initiative', 'Cost Center', 'Type', 'Project Hours Percentage',\n       'Admin Hours Percentage', 'Lobbying', 'Lobbying Location',\n       'Project or Admin', 'Project', 'Worker Cost Center', 'Location',\n       'Total Project Hours', 'Needed Project Hours', 'Adjusted Hours',\n       'Actual Utilization Rate', 'Cost per Project Hour',\n       'Utilization Target', 'Talent Pools', 'Total Cost', 'File',\n       'Project PTO or Admin', 'Standard Monthly Cost',\n       'Amount Banked per Hour', 'Hours', 'Bank Month',\n       'Funding Source Count', 'Funding Source Greater than 1', 'Detail 2', '% - Detail 2','Funding Source 2', 'Amount Banked 2']]\n\nPTOBank2 = PTOBank2.rename(columns={'Detail 2': 'Detail', '% - Detail 2': '% - Detail', 'Funding Source 2': 'Funding Source', 'Amount Banked 2': 'Amount Banked'})\nPTOBank2.

In [435]:
'''PTOBank3 = PTOBank[['Worker', 'Employee ID', 'Project Plan Task', 'Project Plan Task ID',
       'SalaryCosted', 'BenefitsCosted', 'SharedCosted', 'BonusCosted',
       'Initiative', 'Cost Center', 'Type', 'Project Hours Percentage',
       'Admin Hours Percentage', 'Lobbying', 'Lobbying Location',
       'Project or Admin', 'Project', 'Worker Cost Center', 'Location',
       'Total Project Hours', 'Needed Project Hours', 'Adjusted Hours',
       'Actual Utilization Rate', 'Cost per Project Hour',
       'Utilization Target', 'Talent Pools', 'Total Cost', 'File',
       'Project PTO or Admin', 'Standard Monthly Cost',
       'Amount Banked per Hour', 'Hours', 'Bank Month',
       'Funding Source Count', 'Funding Source Greater than 1', 'Detail 3', '% - Detail 3','Funding Source 3', 'Amount Banked 3']]

PTOBank3 = PTOBank3.rename(columns={'Detail 3': 'Detail', '% - Detail 3': '% - Detail', 'Funding Source 3': 'Funding Source', 'Amount Banked 3': 'Amount Banked'})
PTOBank3.dropna(subset=['Amount Banked'], inplace=True)'''

"PTOBank3 = PTOBank[['Worker', 'Employee ID', 'Project Plan Task', 'Project Plan Task ID',\n       'SalaryCosted', 'BenefitsCosted', 'SharedCosted', 'BonusCosted',\n       'Initiative', 'Cost Center', 'Type', 'Project Hours Percentage',\n       'Admin Hours Percentage', 'Lobbying', 'Lobbying Location',\n       'Project or Admin', 'Project', 'Worker Cost Center', 'Location',\n       'Total Project Hours', 'Needed Project Hours', 'Adjusted Hours',\n       'Actual Utilization Rate', 'Cost per Project Hour',\n       'Utilization Target', 'Talent Pools', 'Total Cost', 'File',\n       'Project PTO or Admin', 'Standard Monthly Cost',\n       'Amount Banked per Hour', 'Hours', 'Bank Month',\n       'Funding Source Count', 'Funding Source Greater than 1', 'Detail 3', '% - Detail 3','Funding Source 3', 'Amount Banked 3']]\n\nPTOBank3 = PTOBank3.rename(columns={'Detail 3': 'Detail', '% - Detail 3': '% - Detail', 'Funding Source 3': 'Funding Source', 'Amount Banked 3': 'Amount Banked'})\nPTOBank3.

In [436]:
#PTOBankFINAL = pd.concat([PTOBank1, PTOBank2, PTOBank3], ignore_index=True)

In [437]:
PTOBankFINAL = pd.merge(AllCostsAllocated, FundingSourceRulesData, on = 'Project Plan Task ID', how = 'left')

In [438]:
PTOBankFINAL['Standard Monthly Cost'] = PTOBankFINAL['Standard Monthly Cost'].round(2)
PTOBankFINAL['Cost per Project Hour'] = PTOBankFINAL['Cost per Project Hour'].round(2)
PTOBankFINAL['Amount Banked per Hour'] = PTOBankFINAL['Amount Banked per Hour'].round(2)
PTOBankFINAL['Amount Banked'] = PTOBankFINAL['Amount Banked'].round(2)
PTOBankFINAL['Standard Monthly Cost'] = PTOBankFINAL['Standard Monthly Cost'].round(2)

In [439]:
PTOBankFINAL = PTOBankFINAL[PTOBankFINAL['Type']=="Fully Utilized"]
PTOBankFINAL.dropna(subset=['Amount Banked'], inplace=True)
PTOBankFINAL = PTOBankFINAL[PTOBankFINAL['Total Cost']!=0]
PTOBankFINAL = PTOBankFINAL[PTOBankFINAL['Needed Project Hours']!=0]

In [440]:
PTOBankFINAL.to_excel('./Outputs/PTO1_PTOBank'+Type+timestampreport+'.xlsx')

In [441]:
len(PTOBank)

4819

# Sum PTO Bank Files together

In [442]:
#Load last month's file

LastMonthBankBalance= './Source_Files/PTO8_PTO_Accrual_Balances_After_DrawdownsFINALAug-2024.xlsx'

LastMonthBankBalance = pd.read_excel(LastMonthBankBalance, engine='openpyxl')

In [443]:
LastMonthBankBalance.columns

Index(['Unnamed: 0', 'Employee ID', 'Project Plan Task',
       'Project Plan Task ID', 'Initiative', 'Cost Center', 'Project',
       'Location', 'Lobbying', 'Lobbying Location', 'Amount Banked',
       'Total Banked', 'Percent of Total Bank', 'PTO Funded', 'TaskDrawdowns',
       'RemainingPTOBankBalance', 'Employee Name'],
      dtype='object')

In [444]:
LastMonthBankBalance.rename(columns={"Amount Banked": "Amount BankedORIGINAL"}, inplace=True)
LastMonthBankBalance.rename(columns={"RemainingPTOBankBalance": "Amount Banked"}, inplace=True)

In [445]:
PTOBankFINALSIMPLE = PTOBankFINAL[['Employee ID',  'Project Plan Task', 'Project Plan Task ID','Initiative', 'Cost Center', 'Project', 'Location', 'Lobbying',
       'Lobbying Location', 'Amount Banked']]

In [446]:
PTOBankFINALSIMPLE.columns

Index(['Employee ID', 'Project Plan Task', 'Project Plan Task ID',
       'Initiative', 'Cost Center', 'Project', 'Location', 'Lobbying',
       'Lobbying Location', 'Amount Banked'],
      dtype='object')

In [447]:
BankBalanceFINALSIMPLE = LastMonthBankBalance[['Employee ID',  'Project Plan Task', 'Project Plan Task ID','Initiative', 'Cost Center', 'Project', 'Location', 'Lobbying',
       'Lobbying Location', 'Amount Banked']]

In [448]:
BankBalanceFINALSIMPLE.columns

Index(['Employee ID', 'Project Plan Task', 'Project Plan Task ID',
       'Initiative', 'Cost Center', 'Project', 'Location', 'Lobbying',
       'Lobbying Location', 'Amount Banked'],
      dtype='object')

In [449]:
print(PTOBankFINALSIMPLE.shape)
print(BankBalanceFINALSIMPLE.shape)

(2424, 10)
(2268, 10)


In [450]:
PTOBankFINALSIMPLE['Amount Banked'].sum()


2553752.33

In [451]:
BankBalanceFINALSIMPLE['Amount Banked'].sum()

3136101.0599999996

In [452]:
#PTOBankComulative = pd.concat([PTOBankFINALSIMPLE, BankBalanceFINALSIMPLE], ignore_index=True)
PTOBankComulative = pd.concat([PTOBankFINALSIMPLE, BankBalanceFINALSIMPLE], ignore_index=True)

In [453]:
PTOBankComulative['Amount Banked'].sum()

5689853.390000001

In [454]:
PTOBankSimple = PTOBankComulative[['Employee ID',  'Project Plan Task', 'Project Plan Task ID','Initiative', 'Cost Center', 'Project', 'Location', 'Lobbying',
       'Lobbying Location', 'Amount Banked']]

In [455]:
#PTOBankSimple.to_excel('./Outputs/test.xlsx')

In [456]:
len(PTOBankSimple)

4692

In [457]:
PTOBankSimple = PTOBankSimple.groupby(['Employee ID',  'Project Plan Task', 'Project Plan Task ID', 'Initiative', 'Cost Center', 'Project', 'Location', 'Lobbying',
       'Lobbying Location']).agg({'Amount Banked': 'sum'}).reset_index()

In [458]:
len(PTOBankSimple)

3468

In [459]:
PTOBankSimple['Amount Banked'].sum()

5689853.39

In [460]:
CumulativeBank = PTOBankSimple

Eliminate PTO Bank if Project is Closed

In [461]:
TasksPath = './Source_Files/Extract_Project_Structure_-_Ref_IDs (6).xlsx'
TaskMaps = pd.read_excel(TasksPath, engine='openpyxl')

In [462]:
len(PTOBankSimple)

3468

In [463]:


TaskMaps['Keep'] = 'Yes'
TaskIDs = TaskMaps[['Project Plan Task ID', 'Keep']]
PTOBankSimple = pd.merge(PTOBankSimple, TaskIDs, on = 'Project Plan Task ID', how = 'left')
PTOBankSimple = PTOBankSimple[PTOBankSimple['Keep']=='Yes']

In [464]:
PTOBankSimple.columns

Index(['Employee ID', 'Project Plan Task', 'Project Plan Task ID',
       'Initiative', 'Cost Center', 'Project', 'Location', 'Lobbying',
       'Lobbying Location', 'Amount Banked', 'Keep'],
      dtype='object')

In [465]:
PTOBankSimple = PTOBankSimple[['Employee ID', 'Project Plan Task', 'Project Plan Task ID',
       'Initiative', 'Cost Center', 'Project', 'Location', 'Lobbying',
       'Lobbying Location', 'Amount Banked']]

In [466]:
len(PTOBankSimple)

3468

In [467]:
GovMLBLPath = './Source_Files/Copy of ML-BL restricted gifts_omit PTO Bank.xlsx'
GovProjMaps = pd.read_excel(GovMLBLPath, engine='openpyxl')

In [468]:
PTOBankSimple.columns

Index(['Employee ID', 'Project Plan Task', 'Project Plan Task ID',
       'Initiative', 'Cost Center', 'Project', 'Location', 'Lobbying',
       'Lobbying Location', 'Amount Banked'],
      dtype='object')

In [469]:
len(PTOBankSimple)

3468

In [470]:

GovProjMaps['Keep'] = 'Yes'
GovProjNames = GovProjMaps[['Project/Project Hierarchy', 'Keep']]
GovProjNames.rename(columns={"Project/Project Hierarchy": "Project"}, inplace=True)
PTOBankSimple = pd.merge(PTOBankSimple, GovProjNames, on = 'Project', how = 'left')


C:\Users\Jwendt\AppData\Local\anaconda3\envs\new_env_name\lib\site-packages\pandas\core\frame.py:4300: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [471]:
PTOBankSimple['Amount Banked'].sum()

5689853.39

In [472]:
PTOBankSimple.head()

,Employee ID,Project Plan Task,Project Plan Task ID,Initiative,Cost Center,Project,Location,Lobbying,Lobbying Location,Amount Banked,Keep
0,1.0,Clean & Low Carbon Techs > Stakeholder Engagem...,PROJECT_PLAN_TASK-3-3612,"CH - Transportation, Technology, and Investmen...",China,Provincial Carbon Neutrality,Beijing,None,None,117.78,NaN
1,1.0,General > General (11/01/2023 - 11/01/2026),PROJECT_PLAN_TASK-3-5200,CH - Heavy Industry and Recycling (HIR),China,Transitioning Heavy Industry,Beijing,None,None,1688.85,NaN
2,1.0,Green certificate for NG > Outreach & Engageme...,PROJECT_PLAN_TASK-3-6195,CH - Methane and Non-CO2 mitigation (MNC),China,China Methane Mitigation 2024,Beijing,None,None,1688.85,NaN
3,4.0,General > Conferences/workshops/seminars,PROJECT_PLAN_TASK-3-822,"CH - Transportation, Technology, and Investmen...",China,China green hydrogen H2-100,Beijing,None,None,270.39,NaN
4,4.0,General > General (11/01/2023 - 11/01/2026),PROJECT_PLAN_TASK-3-5200,CH - Heavy Industry and Recycling (HIR),China,Transitioning Heavy Industry,Beijing,None,None,1503.00,NaN


In [473]:
PTOBankSimple.to_excel('./Outputs/PTO2a_CumulativeBankBeforePTODraw'+Type+timestampreport+'.xlsx')

In [474]:
PTOBankSimple = PTOBankSimple[PTOBankSimple['Keep']!='Yes']

In [475]:
len(PTOBankSimple)

3411

In [476]:
PTOBankSimple['Amount Banked'].sum()

5562954.003645629

In [477]:
PTOBankSimple.to_excel('./Outputs/PTO2b_CumulativeBankBeforePTODraw'+Type+timestampreport+'.xlsx')

Calculate total bank per employee and then calculate percentage table

In [478]:
TotalbankbyEmployee = PTOBankSimple[['Employee ID','Amount Banked']]

In [479]:
TotalbankbyEmployee.rename(columns={"Amount Banked": "Total Banked"}, inplace=True)

C:\Users\Jwendt\AppData\Local\anaconda3\envs\new_env_name\lib\site-packages\pandas\core\frame.py:4300: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [480]:
TotalbankbyEmployee = TotalbankbyEmployee.groupby(['Employee ID'], as_index=False).sum()

In [481]:
TotalbankbyEmployee.columns

Index(['Employee ID', 'Total Banked'], dtype='object')

In [482]:
PTOBankSimple = PTOBankSimple[['Employee ID', 'Project Plan Task', 'Project Plan Task ID', 'Initiative', 'Cost Center', 'Project', 'Location', 'Lobbying',
       'Lobbying Location', 'Amount Banked']]

In [483]:
PTOBankSimple = pd.merge(PTOBankSimple, TotalbankbyEmployee, on = 'Employee ID', how = 'left')

In [484]:
PTOBankSimple['Percent of Total Bank'] = PTOBankSimple['Amount Banked']/PTOBankSimple['Total Banked']

In [485]:
#PTOBankSimple.to_excel('/mnt/c/Users/jerem/anaconda3/envs/Code/RMICODING/Data/TimeCodingData/Outputs/PTOPercentages.xlsx')

Now Calculate how much PTO will be utilized from Bank and where

In [486]:
PTOtoFund = AllCostsAllocated[AllCostsAllocated['Type']=='PTO Time for UnderUtilized']

In [487]:
PTOtoFund.columns

Index(['Worker', 'Employee ID', 'Project Plan Task', 'Project Plan Task ID',
       'SalaryCosted', 'BenefitsCosted', 'SharedCosted', 'BonusCosted',
       'Initiative', 'Cost Center', 'ConsultingFeesCosted', 'Type',
       'Project Hours Percentage', 'Admin Hours Percentage', 'Lobbying',
       'Lobbying Location', 'Project or Admin', 'Project',
       'Worker Cost Center', 'PTO more than Admin', 'Location',
       'Total Project Hours', 'Needed Project Hours', 'Adjusted Hours',
       'Actual Utilization Rate', 'Cost per Project Hour',
       'Utilization Target', 'Talent Pools', 'Total Cost', 'File',
       'Project PTO or Admin', 'Standard Monthly Cost',
       'Amount Banked per Hour', 'Amount Banked', 'Cost per PTO Hour',
       'Project & PTO Hours', 'Bank Month'],
      dtype='object')

In [488]:
MonthlyPTObyEmployee = PTOtoFund[['Employee ID', 'Project Plan Task', 'Project Plan Task ID', 'Initiative', 'Cost Center',  'Total Cost']]

In [489]:
MonthlyPTObyEmployee.rename(columns={"Total Cost": "PTO to Pay For"}, inplace=True)

In [490]:
MonthlyPTObyEmployee['PTO to Pay For'].sum()

407915.54000000004

In [491]:
MonthlyPTObyEmployee = pd.merge(MonthlyPTObyEmployee, TotalbankbyEmployee, on = 'Employee ID', how = 'left')

In [492]:
TotalbankbyEmployee.to_excel('./Outputs/PTO3_bankbyemployee'+Type+timestampreport+'.xlsx')

In [493]:
MonthlyPTObyEmployee.columns

Index(['Employee ID', 'Project Plan Task', 'Project Plan Task ID',
       'Initiative', 'Cost Center', 'PTO to Pay For', 'Total Banked'],
      dtype='object')

In [494]:
MonthlyPTObyEmployee['PTO Funded'] = np.where(MonthlyPTObyEmployee['PTO to Pay For']<=MonthlyPTObyEmployee['Total Banked'], MonthlyPTObyEmployee['PTO to Pay For'], np.where(MonthlyPTObyEmployee['PTO to Pay For']>MonthlyPTObyEmployee['Total Banked'], MonthlyPTObyEmployee['Total Banked'],0))

In [495]:
MonthlyPTObyEmployee.to_excel('./Outputs/PTO4_PTOPaidFor'+Type+timestampreport+'.xlsx')

In [496]:
PTOAllowableTable = MonthlyPTObyEmployee[['Employee ID', 'PTO Funded']]

In [497]:
PTOBankDrawdowns = pd.merge(PTOBankSimple, PTOAllowableTable, on = 'Employee ID', how = 'left')

In [498]:
PTOBankDrawdowns.columns

Index(['Employee ID', 'Project Plan Task', 'Project Plan Task ID',
       'Initiative', 'Cost Center', 'Project', 'Location', 'Lobbying',
       'Lobbying Location', 'Amount Banked', 'Total Banked',
       'Percent of Total Bank', 'PTO Funded'],
      dtype='object')

In [499]:
PTOBankDrawdowns['TaskDrawdowns'] = PTOBankDrawdowns['Percent of Total Bank'] * PTOBankDrawdowns['PTO Funded']

In [500]:
PTOBankDrawdowns['TaskDrawdowns'].fillna(0, inplace = True)
PTOBankDrawdowns['PTO Funded'].fillna(0, inplace = True)

In [501]:
PTOBankDrawdowns['RemainingPTOBankBalance'] = PTOBankDrawdowns['Amount Banked'] - PTOBankDrawdowns['TaskDrawdowns']

Create PTO Balances After PTO Drawdown

In [502]:
PTOBankDrawdowns = pd.merge(PTOBankDrawdowns, Name_and_HomeCCList, on = 'Employee ID', how = 'left')

In [503]:
PTOBankDrawdowns.to_excel('./Outputs/PTO8_PTO_Accrual_Balances_After_Drawdowns'+Type+timestampreport+'.xlsx')

Create New AllCosts Allocated File and reduce PTO accruals by amounts calculated

In [504]:
PTOtoFund.head()

,Worker,Employee ID,Project Plan Task,Project Plan Task ID,SalaryCosted,BenefitsCosted,SharedCosted,BonusCosted,Initiative,Cost Center,...,Talent Pools,Total Cost,File,Project PTO or Admin,Standard Monthly Cost,Amount Banked per Hour,Amount Banked,Cost per PTO Hour,Project & PTO Hours,Bank Month
204,Adefunke Sonaike | Addy Sonaike,1029.0,US Program > PTO,PROJECT_PLAN_TASK-3-5735,791.09,347.15,106.96,39.55,US - Admin,US,...,Associate,1284.75,FINAL,PTO,101.964436,0.0,0.0,32.118750,40.0,Sep-2024
205,Adrienne Tecza,1417.0,Climate Intelligence > PTO,PROJECT_PLAN_TASK-3-5722,2082.52,1322.46,407.47,179.83,CIP - Admin,Climate Intelligence,...,Sr. Associate,3992.28,FINAL,PTO,83.172318,0.0,0.0,83.172500,48.0,Sep-2024
206,Aimee Schwartze,981.0,Africa > PTO,PROJECT_PLAN_TASK-3-5719,584.66,272.97,84.11,29.23,AF - Admin,Africa,...,Program Operations Lead,970.97,FINAL,PTO,105.539866,0.0,0.0,20.228542,48.0,Sep-2024
207,Aldine Francois,1740.0,Influence Team > PTO,PROJECT_PLAN_TASK-3-5737,744.05,385.72,118.85,37.20,COM - Admin,Influence Team,...,Program Assistant/Executive Assistant,1285.82,FINAL,PTO,160.726669,0.0,0.0,160.727500,8.0,Sep-2024
208,Alex Axthelm,1530.0,Finance > PTO,PROJECT_PLAN_TASK-3-5726,5333.38,1815.84,559.49,247.74,GCF - Admin,Finance,...,Manager,7956.45,FINAL,PTO,130.007178,0.0,0.0,66.303750,120.0,Sep-2024


In [505]:
PTO_Original = PTOtoFund[['Employee ID','Total Cost']]

In [506]:
PTO_Original['Total Cost'].sum()

407915.54000000004

In [507]:
PTOreduction = PTOBankDrawdowns[PTOBankDrawdowns['TaskDrawdowns']!=0]

In [508]:
PTOreduction['PTO Adjustment'] = -PTOBankDrawdowns['TaskDrawdowns']

C:\Users\Jwendt\AppData\Local\Temp\ipykernel_20800\824079842.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PTOreduction['PTO Adjustment'] = -PTOBankDrawdowns['TaskDrawdowns']


In [509]:
PTOreduction.columns

Index(['Employee ID', 'Project Plan Task', 'Project Plan Task ID',
       'Initiative', 'Cost Center', 'Project', 'Location', 'Lobbying',
       'Lobbying Location', 'Amount Banked', 'Total Banked',
       'Percent of Total Bank', 'PTO Funded', 'TaskDrawdowns',
       'RemainingPTOBankBalance', 'Employee Name', 'PTO Adjustment'],
      dtype='object')

In [510]:
PTOreductionsumming = PTOreduction[['Employee ID','PTO Adjustment']]


In [511]:
PTOreductionsumming = PTOreductionsumming.rename(columns={'PTO Adjustment': 'Total Cost'})

In [512]:
PTOreductionsumming['Total Cost'].sum()

-124785.64000000001

In [513]:
PTOFunded = pd.concat([PTOreductionsumming, PTO_Original], ignore_index=True)

In [514]:
RemainingPTO = PTOFunded.groupby(['Employee ID'], as_index=False).sum()

This amount should tie to the amount of PTO after PTO Bank was applied

In [515]:
RemainingPTO['Total Cost'].sum()

283129.9

In [516]:
PTOtoFund.columns

Index(['Worker', 'Employee ID', 'Project Plan Task', 'Project Plan Task ID',
       'SalaryCosted', 'BenefitsCosted', 'SharedCosted', 'BonusCosted',
       'Initiative', 'Cost Center', 'ConsultingFeesCosted', 'Type',
       'Project Hours Percentage', 'Admin Hours Percentage', 'Lobbying',
       'Lobbying Location', 'Project or Admin', 'Project',
       'Worker Cost Center', 'PTO more than Admin', 'Location',
       'Total Project Hours', 'Needed Project Hours', 'Adjusted Hours',
       'Actual Utilization Rate', 'Cost per Project Hour',
       'Utilization Target', 'Talent Pools', 'Total Cost', 'File',
       'Project PTO or Admin', 'Standard Monthly Cost',
       'Amount Banked per Hour', 'Amount Banked', 'Cost per PTO Hour',
       'Project & PTO Hours', 'Bank Month'],
      dtype='object')

In [517]:
RemainingPTO.to_excel('./Outputs/PTO6_RemainingPTO'+Type+timestampreport+'.xlsx')

In [518]:
PTOtoFund['Total Cost'].sum()

407915.54000000004

In [519]:
PTOtoFundrevised = PTOtoFund[['Worker', 'Employee ID', 'Project Plan Task', 'Project Plan Task ID',
       'Initiative', 'Cost Center', 'ConsultingFeesCosted', 'Type',
       'Project Hours Percentage', 'Admin Hours Percentage', 'Lobbying',
       'Lobbying Location', 'Project or Admin', 'Project',
       'Worker Cost Center', 'PTO more than Admin', 'Location',
       'Total Project Hours', 'Needed Project Hours', 'Adjusted Hours',
       'Actual Utilization Rate', 'Standard Monthly Cost', 'Amount Banked per Hour', 'Amount Banked','Cost per Project Hour',
       'Utilization Target', 'Talent Pools', 'File',
       'Project PTO or Admin',  'Cost per PTO Hour',
       'Project & PTO Hours', 'Bank Month']]




len(PTOtoFundrevised)

160

In [520]:
#PTOtoFundrevised = PTOtoFundrevised.drop_duplicates(subset = 'Worker', keep='first')
#len(PTOtoFundrevised)

In [521]:
len(RemainingPTO)

160

In [522]:
AllCostsAllocatedPTOUpdate = pd.merge(RemainingPTO, PTOtoFundrevised, on = 'Employee ID', how = 'left')

In [523]:
len(AllCostsAllocatedPTOUpdate)

160

In [524]:
AllCostsAllocatedPTOUpdate.columns

Index(['Employee ID', 'Total Cost', 'Worker', 'Project Plan Task',
       'Project Plan Task ID', 'Initiative', 'Cost Center',
       'ConsultingFeesCosted', 'Type', 'Project Hours Percentage',
       'Admin Hours Percentage', 'Lobbying', 'Lobbying Location',
       'Project or Admin', 'Project', 'Worker Cost Center',
       'PTO more than Admin', 'Location', 'Total Project Hours',
       'Needed Project Hours', 'Adjusted Hours', 'Actual Utilization Rate',
       'Standard Monthly Cost', 'Amount Banked per Hour', 'Amount Banked',
       'Cost per Project Hour', 'Utilization Target', 'Talent Pools', 'File',
       'Project PTO or Admin', 'Cost per PTO Hour', 'Project & PTO Hours',
       'Bank Month'],
      dtype='object')

In [525]:
PTOreductionsumming = PTOreductionsumming.rename(columns={'Total Cost': 'PTO Adjustment'})

In [526]:
AllCostsAllocatedPTOUpdate.to_excel('./Outputs/AllCostsAllocatedPTOUpdate'+Type+timestampreport+'.xlsx')

In [527]:
AllCostsAllocatedPTOUpdate['Total Cost'].sum()

283129.9

In [528]:
salarypercentages_table

,Employee ID,SalaryPercentage,BenefitsPercentage,SharedPercentage,BonusPercentage
0,1841.0,0.575381,0.215582,0.180946,0.028091
1,2028.0,0.569011,0.219239,0.184016,0.027735
2,7.0,0.642473,0.159986,0.134282,0.063260
3,1275.0,0.617393,0.191555,0.160780,0.030272
4,1395.0,0.612716,0.194230,0.163025,0.030029
...,...,...,...,...,...
735,617.0,0.000000,0.000000,0.000000,1.000000
736,628.0,0.000000,0.000000,0.000000,1.000000
737,813.0,0.000000,0.000000,0.000000,1.000000
738,868.0,0.000000,0.000000,0.000000,1.000000


In [529]:
len(salarypercentages_table)

740

In [530]:
AllCostsAllocatedFINALPTO = pd.merge(AllCostsAllocatedPTOUpdate, salarypercentages_table, on = 'Employee ID', how = 'left')

In [531]:
AllCostsAllocatedFINALPTO['Total Cost'].sum()

283129.9

In [532]:
AllCostsAllocatedFINALPTO['SalaryCosted'] = AllCostsAllocatedFINALPTO['Total Cost']*AllCostsAllocatedFINALPTO['SalaryPercentage']
AllCostsAllocatedFINALPTO['BenefitsCosted'] = AllCostsAllocatedFINALPTO['Total Cost']*AllCostsAllocatedFINALPTO['BenefitsPercentage']
AllCostsAllocatedFINALPTO['SharedCosted'] = AllCostsAllocatedFINALPTO['Total Cost']*AllCostsAllocatedFINALPTO['SharedPercentage']
AllCostsAllocatedFINALPTO['BonusCosted'] = AllCostsAllocatedFINALPTO['Total Cost']*AllCostsAllocatedFINALPTO['BonusPercentage']

In [533]:
AllCostsAllocatedFINALPTO['Total Cost'].sum()

283129.9

In [534]:
AllCostsAllocatedPTOUpdate

,Employee ID,Total Cost,Worker,Project Plan Task,Project Plan Task ID,Initiative,Cost Center,ConsultingFeesCosted,Type,Project Hours Percentage,...,Amount Banked per Hour,Amount Banked,Cost per Project Hour,Utilization Target,Talent Pools,File,Project PTO or Admin,Cost per PTO Hour,Project & PTO Hours,Bank Month
0,6.0,0.000000e+00,Shutong Lu | Lucy Lu,China > PTO,PROJECT_PLAN_TASK-3-5724,CH - Admin,China,0.0,PTO Time for UnderUtilized,0.952381,...,0.0,0.0,140.620574,0.65,Manager,FINAL,PTO,15.233750,48.0,Sep-2024
1,127.0,5.557860e+03,Jeannene Cruise Woolley | Frances Woolley,Accounting > PTO,PROJECT_PLAN_TASK-3-5772,ACT - Accounting,Org Finance,0.0,PTO Time for UnderUtilized,0,...,0.0,0.0,132.329995,0.65,Manager,FINAL,PTO,132.330000,42.0,Sep-2024
2,322.0,1.856600e+02,James Sherwood,Carbon-Free Electricity > PTO,PROJECT_PLAN_TASK-3-5698,POW - Admin,Carbon-Free Electricity,0.0,PTO Time for UnderUtilized,0.992063,...,0.0,0.0,232.081585,0.60,Principal,FINAL,PTO,23.207500,8.0,Sep-2024
3,378.0,5.684342e-14,Kevin Brehm,Carbon-Free Electricity > PTO,PROJECT_PLAN_TASK-3-5698,POW - Admin,Carbon-Free Electricity,0.0,PTO Time for UnderUtilized,0.979853,...,0.0,0.0,161.075956,0.65,Manager,FINAL,PTO,7.382708,48.0,Sep-2024
4,394.0,1.764040e+03,Philip Keuhn | Phil Keuhn,Carbon-Free Buildings > PTO,PROJECT_PLAN_TASK-3-5718,BLD - Admin,Carbon-Free Buildings,0.0,PTO Time for UnderUtilized,0.833333,...,0.0,0.0,220.505744,0.60,Principal,FINAL,PTO,220.505000,8.0,Sep-2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,2045.0,7.228400e+02,Nour Elgoweili,US Program > PTO,PROJECT_PLAN_TASK-3-5735,US - Admin,US,0.0,PTO Time for UnderUtilized,0.603741,...,0.0,0.0,90.355687,0.70,Associate,FINAL,PTO,90.355000,8.0,Sep-2024
156,2046.0,2.784640e+03,Lillian Mercho,Influence Team > PTO,PROJECT_PLAN_TASK-3-5737,COM - Admin,Influence Team,0.0,PTO Time for UnderUtilized,0.748299,...,0.0,0.0,94.075869,0.70,Sr. Associate,FINAL,PTO,38.675556,72.0,Sep-2024
157,2047.0,4.305390e+03,John Thompson | Jake Thompson,Influence Team > PTO,PROJECT_PLAN_TASK-3-5737,COM - Admin,Influence Team,0.0,PTO Time for UnderUtilized,0.353692,...,0.0,0.0,119.594049,0.69,Program Marketer,FINAL,PTO,119.594167,36.0,Sep-2024
158,2048.0,2.962820e+03,Evan Brooks,US Program > PTO,PROJECT_PLAN_TASK-3-5735,US - Admin,US,0.0,PTO Time for UnderUtilized,0.580357,...,0.0,0.0,92.587915,0.70,Associate,FINAL,PTO,92.588125,32.0,Sep-2024


In [535]:
#AllCostsAllocatedPTOUpdate = pd.merge(AllCostsAllocatedPTOUpdate, PTOreductionsumming, on = 'Employee ID', how = 'left')

In [536]:
PTOreductionsumming.head()

,Employee ID,PTO Adjustment
10,6.0,-731.220000
146,378.0,-31.860493
147,378.0,-322.509507
261,424.0,-518.535209
262,424.0,-33.454791


In [537]:
#AllCostsAllocatedFINALPTO.to_excel('/mnt/c/Users/jerem/anaconda3/envs/Code/RMICODING/Data/TimeCodingData/Outputs/ALLcostsAllocatedPTOFINAL.xlsx')

In [538]:
len(AllCostsAllocatedPTOUpdate)

160

In [539]:
AllCostsAllocatedPTOUpdate['Total Cost'].sum()

283129.9

Now Push PTO Accruals to Tasks

In [540]:
TaskstoFund = AllCostsAllocated[AllCostsAllocated['Type']!='PTO Time for UnderUtilized']

In [541]:
PTOreduction['TaskDrawdowns'].sum()

124785.64000000001

In [542]:
PTOreduction = pd.merge(PTOreduction, salarypercentages_table, on = 'Employee ID', how = 'left')

In [543]:
PTOreduction.to_excel('./Outputs/SalaryPercentagesTable.xlsx')

In [544]:
PTOreduction['SalaryCosted'] = PTOreduction['TaskDrawdowns']*PTOreduction['SalaryPercentage']
PTOreduction['BenefitsCosted'] = PTOreduction['TaskDrawdowns']*PTOreduction['BenefitsPercentage']
PTOreduction['SharedCosted'] = PTOreduction['TaskDrawdowns']*PTOreduction['SharedPercentage']
PTOreduction['BonusCosted'] = PTOreduction['TaskDrawdowns']*PTOreduction['BonusPercentage']

In [545]:
PTOreduction.columns

Index(['Employee ID', 'Project Plan Task', 'Project Plan Task ID',
       'Initiative', 'Cost Center', 'Project', 'Location', 'Lobbying',
       'Lobbying Location', 'Amount Banked', 'Total Banked',
       'Percent of Total Bank', 'PTO Funded', 'TaskDrawdowns',
       'RemainingPTOBankBalance', 'Employee Name', 'PTO Adjustment',
       'SalaryPercentage', 'BenefitsPercentage', 'SharedPercentage',
       'BonusPercentage', 'SalaryCosted', 'BenefitsCosted', 'SharedCosted',
       'BonusCosted'],
      dtype='object')

In [546]:

PTOreduction = pd.merge(PTOreduction, HomeCostCenterlist, on = 'Employee ID', how = 'left')
PTOreduction = pd.merge(PTOreduction, EmployeeIDsandNames, on = 'Employee ID', how = 'left')


In [547]:
TaskstoFund.columns

Index(['Worker', 'Employee ID', 'Project Plan Task', 'Project Plan Task ID',
       'SalaryCosted', 'BenefitsCosted', 'SharedCosted', 'BonusCosted',
       'Initiative', 'Cost Center', 'ConsultingFeesCosted', 'Type',
       'Project Hours Percentage', 'Admin Hours Percentage', 'Lobbying',
       'Lobbying Location', 'Project or Admin', 'Project',
       'Worker Cost Center', 'PTO more than Admin', 'Location',
       'Total Project Hours', 'Needed Project Hours', 'Adjusted Hours',
       'Actual Utilization Rate', 'Cost per Project Hour',
       'Utilization Target', 'Talent Pools', 'Total Cost', 'File',
       'Project PTO or Admin', 'Standard Monthly Cost',
       'Amount Banked per Hour', 'Amount Banked', 'Cost per PTO Hour',
       'Project & PTO Hours', 'Bank Month'],
      dtype='object')

In [548]:
PTOreduction.columns

Index(['Employee ID', 'Project Plan Task', 'Project Plan Task ID',
       'Initiative', 'Cost Center', 'Project', 'Location', 'Lobbying',
       'Lobbying Location', 'Amount Banked', 'Total Banked',
       'Percent of Total Bank', 'PTO Funded', 'TaskDrawdowns',
       'RemainingPTOBankBalance', 'Employee Name', 'PTO Adjustment',
       'SalaryPercentage', 'BenefitsPercentage', 'SharedPercentage',
       'BonusPercentage', 'SalaryCosted', 'BenefitsCosted', 'SharedCosted',
       'BonusCosted', 'Home Cost Center', 'Worker'],
      dtype='object')

In [549]:
PTOreduction = PTOreduction[['Employee ID', 'Project Plan Task', 'Project Plan Task ID',
       'Initiative', 'Cost Center', 'Project', 'Location', 'Lobbying',
       'Lobbying Location', 'TaskDrawdowns',
       'SalaryPercentage',
       'BenefitsPercentage', 'SharedPercentage', 'BonusPercentage',
       'SalaryCosted', 'BenefitsCosted', 'SharedCosted', 'BonusCosted',
       'Home Cost Center', 'Worker']]



In [550]:
PTOreduction['Total Cost'] = PTOreduction['SalaryCosted'] + PTOreduction['BenefitsCosted'] + PTOreduction['SharedCosted'] + PTOreduction['BonusCosted']
PTOreduction['Type'] = "PTO Task Accrual"
PTOreduction['Project Hours Percentage'] = 1
PTOreduction['Admin Hours Percentage'] = 0
PTOreduction['Project or Admin'] = 'Project'
PTOreduction['PTO more than Admint'] = 'NA'


PTOreduction.rename(columns={'Home Cost Center': 'Worker Cost Center'}, inplace=True)

In [551]:
#PTOreduction.to_excel('/mnt/c/Users/jerem/anaconda3/envs/Code/RMICODING/Data/TimeCodingData/Outputs/PTOReduction.xlsx')

In [552]:
PTOreduction.to_excel('./Outputs/PTO7_PTOReduction'+Type+timestampreport+'.xlsx')

In [553]:
PTOreduction['Total Cost'].sum()

124785.63999999997

In [554]:
AllCostsAllocatedFINALPTO.to_excel('./Outputs/AllCostsAllocatedFINALPTO'+Type+timestampreport+'.xlsx')

In [555]:
AllCostsAllocatedFINALPTO['Total Cost'].sum()

283129.9

In [556]:
TaskstoFund.to_excel('./Outputs/PTO5_ TaskstoFund'+Type+timestampreport+'.xlsx')

In [557]:
TaskstoFund['Total Cost'].sum()

10375060.92

In [558]:
AllCostsAllocatedPTOAccruals = pd.concat([AllCostsAllocatedFINALPTO, PTOreduction, TaskstoFund], ignore_index=True)

In [559]:
AllCostsAllocatedPTOAccruals['Employee ID'] = AllCostsAllocatedPTOAccruals['Employee ID'].astype(int)
AllCostsAllocatedPTOAccruals['Employee ID'] = AllCostsAllocatedPTOAccruals['Employee ID'].apply(str)

In [560]:
AllCostsAllocatedPTOAccruals['Employee ID'] = AllCostsAllocatedPTOAccruals['Employee ID'].apply(add_zeros)

In [745]:
AllCostsAllocatedPTOAccruals.loc[AllCostsAllocatedPTOAccruals['Initiative'] == 'MTX - Admin', 'Initiative'] = 'CFT - Admin' 

In [746]:
AllCostsAllocatedPTOAccruals.to_excel('./Outputs/ALLcostsAllocatedwACCRUALS_FINAL'+Type+timestampreport+'.xlsx')

Create Final Balances File After Withdrawals

# Funding Source Reporting

In [562]:
#FundingSourceReport = pd.merge(FundingSourceReport, FundingSourceRulesLookup, on = 'Project Plan Task ID', how = 'left')

In [563]:
'''FundingSourceReport['Amount Banked 1'] = FundingSourceReport['Amount Banked'] * (FundingSourceReport['% - Detail 1']/100)
FundingSourceReport['Amount Banked 2'] = FundingSourceReport['Amount Banked'] * (FundingSourceReport['% - Detail 2']/100)
FundingSourceReport['Amount Banked 3'] = FundingSourceReport['Amount Banked'] * (FundingSourceReport['% - Detail 3']/100)'''

"FundingSourceReport['Amount Banked 1'] = FundingSourceReport['Amount Banked'] * (FundingSourceReport['% - Detail 1']/100)\nFundingSourceReport['Amount Banked 2'] = FundingSourceReport['Amount Banked'] * (FundingSourceReport['% - Detail 2']/100)\nFundingSourceReport['Amount Banked 3'] = FundingSourceReport['Amount Banked'] * (FundingSourceReport['% - Detail 3']/100)"

In [564]:
'''FundingSourceReport1 = FundingSourceReport[['Worker', 'Employee ID', 'Project Plan Task', 'Project Plan Task ID',
       'SalaryCosted', 'BenefitsCosted', 'SharedCosted', 'BonusCosted',
       'Initiative', 'Cost Center', 'Type', 'Project Hours Percentage',
       'Admin Hours Percentage', 'Lobbying', 'Lobbying Location',
       'Project or Admin', 'Project', 'Worker Cost Center', 'Location',
       'Total Project Hours', 'Needed Project Hours', 'Adjusted Hours',
       'Actual Utilization Rate', 'Cost per Project Hour',
       'Utilization Target', 'Talent Pools', 'Total Cost', 'File',
       'Project PTO or Admin', 'Standard Monthly Cost',
       'Amount Banked per Hour', 'Hours', 'Bank Month',
       'Funding Source Count', 'Funding Source Greater than 1', 'Detail 1', '% - Detail 1','Funding Source 1', 'Amount Banked 1']]

FundingSourceReport1 = FundingSourceReport1.rename(columns={'Detail 1': 'Detail', '% - Detail 1': '% - Detail', 'Funding Source 1': 'Funding Source', 'Amount Banked 1': 'Amount Banked'})'''

"FundingSourceReport1 = FundingSourceReport[['Worker', 'Employee ID', 'Project Plan Task', 'Project Plan Task ID',\n       'SalaryCosted', 'BenefitsCosted', 'SharedCosted', 'BonusCosted',\n       'Initiative', 'Cost Center', 'Type', 'Project Hours Percentage',\n       'Admin Hours Percentage', 'Lobbying', 'Lobbying Location',\n       'Project or Admin', 'Project', 'Worker Cost Center', 'Location',\n       'Total Project Hours', 'Needed Project Hours', 'Adjusted Hours',\n       'Actual Utilization Rate', 'Cost per Project Hour',\n       'Utilization Target', 'Talent Pools', 'Total Cost', 'File',\n       'Project PTO or Admin', 'Standard Monthly Cost',\n       'Amount Banked per Hour', 'Hours', 'Bank Month',\n       'Funding Source Count', 'Funding Source Greater than 1', 'Detail 1', '% - Detail 1','Funding Source 1', 'Amount Banked 1']]\n\nFundingSourceReport1 = FundingSourceReport1.rename(columns={'Detail 1': 'Detail', '% - Detail 1': '% - Detail', 'Funding Source 1': 'Funding Source',

In [565]:
'''FundingSourceReport2 = FundingSourceReport[['Worker', 'Employee ID', 'Project Plan Task', 'Project Plan Task ID',
       'SalaryCosted', 'BenefitsCosted', 'SharedCosted', 'BonusCosted',
       'Initiative', 'Cost Center', 'Type', 'Project Hours Percentage',
       'Admin Hours Percentage', 'Lobbying', 'Lobbying Location',
       'Project or Admin', 'Project', 'Worker Cost Center', 'Location',
       'Total Project Hours', 'Needed Project Hours', 'Adjusted Hours',
       'Actual Utilization Rate', 'Cost per Project Hour',
       'Utilization Target', 'Talent Pools', 'Total Cost', 'File',
       'Project PTO or Admin', 'Standard Monthly Cost',
       'Amount Banked per Hour', 'Hours', 'Bank Month',
       'Funding Source Count', 'Funding Source Greater than 1', 'Detail 2', '% - Detail 2','Funding Source 2', 'Amount Banked 2']]

FundingSourceReport2 = FundingSourceReport2.rename(columns={'Detail 2': 'Detail', '% - Detail 2': '% - Detail', 'Funding Source 2': 'Funding Source', 'Amount Banked 2': 'Amount Banked'})
'''

"FundingSourceReport2 = FundingSourceReport[['Worker', 'Employee ID', 'Project Plan Task', 'Project Plan Task ID',\n       'SalaryCosted', 'BenefitsCosted', 'SharedCosted', 'BonusCosted',\n       'Initiative', 'Cost Center', 'Type', 'Project Hours Percentage',\n       'Admin Hours Percentage', 'Lobbying', 'Lobbying Location',\n       'Project or Admin', 'Project', 'Worker Cost Center', 'Location',\n       'Total Project Hours', 'Needed Project Hours', 'Adjusted Hours',\n       'Actual Utilization Rate', 'Cost per Project Hour',\n       'Utilization Target', 'Talent Pools', 'Total Cost', 'File',\n       'Project PTO or Admin', 'Standard Monthly Cost',\n       'Amount Banked per Hour', 'Hours', 'Bank Month',\n       'Funding Source Count', 'Funding Source Greater than 1', 'Detail 2', '% - Detail 2','Funding Source 2', 'Amount Banked 2']]\n\nFundingSourceReport2 = FundingSourceReport2.rename(columns={'Detail 2': 'Detail', '% - Detail 2': '% - Detail', 'Funding Source 2': 'Funding Source',

In [566]:
'''FundingSourceReport3 = FundingSourceReport[['Worker', 'Employee ID', 'Project Plan Task', 'Project Plan Task ID',
       'SalaryCosted', 'BenefitsCosted', 'SharedCosted', 'BonusCosted',
       'Initiative', 'Cost Center', 'Type', 'Project Hours Percentage',
       'Admin Hours Percentage', 'Lobbying', 'Lobbying Location',
       'Project or Admin', 'Project', 'Worker Cost Center', 'Location',
       'Total Project Hours', 'Needed Project Hours', 'Adjusted Hours',
       'Actual Utilization Rate', 'Cost per Project Hour',
       'Utilization Target', 'Talent Pools', 'Total Cost', 'File',
       'Project PTO or Admin', 'Standard Monthly Cost',
       'Amount Banked per Hour', 'Hours', 'Bank Month',
       'Funding Source Count', 'Funding Source Greater than 1', 'Detail 3', '% - Detail 3','Funding Source 3', 'Amount Banked 3']]

FundingSourceReport3 = FundingSourceReport3.rename(columns={'Detail 3': 'Detail', '% - Detail 3': '% - Detail', 'Funding Source 3': 'Funding Source', 'Amount Banked 3': 'Amount Banked'})
'''

"FundingSourceReport3 = FundingSourceReport[['Worker', 'Employee ID', 'Project Plan Task', 'Project Plan Task ID',\n       'SalaryCosted', 'BenefitsCosted', 'SharedCosted', 'BonusCosted',\n       'Initiative', 'Cost Center', 'Type', 'Project Hours Percentage',\n       'Admin Hours Percentage', 'Lobbying', 'Lobbying Location',\n       'Project or Admin', 'Project', 'Worker Cost Center', 'Location',\n       'Total Project Hours', 'Needed Project Hours', 'Adjusted Hours',\n       'Actual Utilization Rate', 'Cost per Project Hour',\n       'Utilization Target', 'Talent Pools', 'Total Cost', 'File',\n       'Project PTO or Admin', 'Standard Monthly Cost',\n       'Amount Banked per Hour', 'Hours', 'Bank Month',\n       'Funding Source Count', 'Funding Source Greater than 1', 'Detail 3', '% - Detail 3','Funding Source 3', 'Amount Banked 3']]\n\nFundingSourceReport3 = FundingSourceReport3.rename(columns={'Detail 3': 'Detail', '% - Detail 3': '% - Detail', 'Funding Source 3': 'Funding Source',

In [567]:
#FundingSourceReport1['File'] = '1'
#FundingSourceReport2['File'] = '2'
#FundingSourceReport3['File'] = '3'

In [568]:
#FundingSourceReport1['Total Cost'].sum()

In [569]:
#FundingSourceReport1a = FundingSourceReport1[FundingSourceReport1['Funding Source Count']!=2]
#FundingSourceReport1b = FundingSourceReport1a[FundingSourceReport1a['Funding Source Count']!=3]

In [570]:
#FundingSourceReport2a = FundingSourceReport2[FundingSourceReport2['Funding Source Count']==2]
#FundingSourceReport3a = FundingSourceReport3[FundingSourceReport3['Funding Source Count']==3]

In [571]:
#FundingSourceReportFINAL = pd.concat([FundingSourceReport1b, FundingSourceReport2a, FundingSourceReport3a], ignore_index=True)

In [572]:
#FundingSourceReportFINAL.to_excel('/mnt/c/Users/jerem/anaconda3/envs/Code/RMICODING/Data/TimeCodingData/Outputs/FundingSourceData'+Type+timestampreport+'.xlsx')

In [573]:
#FundingSourceReportFINAL['Total Cost'].sum()

In [574]:
#FundingSourceReportFINAL.columns

In [575]:
#FundingSourceReportFINAL['Funding Source'].fillna('Not Gift Funded', inplace = True)

In [576]:
#FundingSourceReportFINAL['Total Project Hours'].fillna('Not Gift Funded', inplace = True)

# Make Hours Table by Month

In [577]:
import os

Timepath = os.path.join('.','Outputs','ProgramReports')


TimeTemplatepath = './Source_Files/Time_Cover_Page.xlsx'

In [578]:
wb = openpyxl.load_workbook(filename=TimeTemplatepath)
ws = wb['Cover']

In [579]:
TimeDataDFALL['Reported Quantity'].fillna(0, inplace = True)

In [580]:
adminonlytable = AllCostsAllocated[AllCostsAllocated['Type']=='Admin Time for UnderUtilized']
PTOonlytable = AllCostsAllocated[AllCostsAllocated['Type']=='PTO Time for UnderUtilized']
AdminCostMaps = adminonlytable[['Employee ID', 'Total Cost']].drop_duplicates()
PTOCostMaps = PTOonlytable[['Employee ID', 'Total Cost']].drop_duplicates()


AdminCostMaps = AdminCostMaps.rename(columns={'Total Cost': 'Total Admin Cost'})
PTOCostMaps = PTOCostMaps.rename(columns={'Total Cost': 'Total PTO Cost'})

In [581]:

AdminCostMaps['Employee ID'] = AdminCostMaps['Employee ID'].astype(int)
PTOCostMaps['Employee ID'] = PTOCostMaps['Employee ID'].astype(int)
#TimeDataDFALL = pd.merge(TimeDataDFALL, AdminCostMaps, on = 'Employee ID', how = 'left')

In [582]:
MonthAdminTable = pd.merge(MonthAdminTable, AdminCostMaps, on = 'Employee ID', how = 'left')
MonthAdminTable = pd.merge(MonthAdminTable, PTOCostMaps, on = 'Employee ID', how = 'left')


In [583]:
MonthAdminTable = pd.merge(MonthAdminTable, HomeCostCenterlist, on = 'Employee ID', how = 'left')

In [584]:
MonthAdminTable.to_excel('./Outputs/monthadmintable.xlsx')

In [585]:
from openpyxl.styles import Border, Side, PatternFill, Font, GradientFill, Alignment
from openpyxl.utils import get_column_letter
import os

redFill = PatternFill(start_color='FFFF0000',
                   end_color='FFFF0000',
                   fill_type='solid')


lightblueFill = PatternFill(start_color='00CCFFFF',
                   end_color='00CCFFFF',
                   fill_type='solid')

darkblueFill = PatternFill(start_color='0099CCFF',
                   end_color='0099CCFF',
                   fill_type='solid')

blueFill = PatternFill(start_color='0033CCCC',
                   end_color='0033CCCC',
                   fill_type='solid')

greyFill = PatternFill(start_color='00C0C0C0',
                   end_color='00C0C0C0',
                   fill_type='solid')


In [586]:
Logopath = './Source_Files/RMILogo.png'

In [587]:
Programs = list(TimeDataDFALL['Worker Cost Center'].unique())

CoverNames = []

for i in Programs:
    CoverNames.append(i + " - " + timestampreport)
    


In [588]:
def CreateMissingTimesheets(program):
    firstdf = AllCostsAllocated[AllCostsAllocated['Cost Center']==program]
    seconddf = firstdf[firstdf['Admin Hours Percentage']==1]  
    thirdf = seconddf[['Worker', 'Type']]
    return thirdf

In [589]:
for i in CoverNames:    
    wb = load_workbook(TimeTemplatepath)
    ws = wb['Cover']   
    img = openpyxl.drawing.image.Image(Logopath)
    img.anchor = 'A1'
    ws.add_image(img)
    ws.cell(row = 14, column = 1).value = i
    ws.row_dimensions[14].height = 50  # Adjust the height as needed
    wb.create_sheet('Guide')
    sheet = wb["Guide"]
    sheet.cell(row=1, column=1).value = "Guidance For Use of Program Time Report" 
    sheet.cell(row=2, column=1).value = "Note: The Program Time Report is a report designed for POLs and MDs to show which workers are missing time for the month and to provide transparency into your Program's timekeeping.  Please note that if workers do not submit their full month's time before the Salary Costing Methodolgy is run, then we will not be able to use that data to allocate the costs from those hours to Projects.  This will mean that any time not reported on Projects will be costed to Admin if we don't receive it before Accounting runs the Salary Costing Methodology.  If there is time data that you would like to include in the Program Time Report in the future, please reach out to Jeremy Wendt or Tobias Lutz."
    sheet.cell(row=4, column=1).value = "NO TIME or ALL ADMIN: This tab shows all workers who haven't submitted any time for the current month.  This is indicated by the text 'TimeSheetsNotSubmitted'.  Also this tab shows workers that have only costed time to Admin projects, which means that all of their costs will hit Admin.  They are labeled as 'Admin Time for Underutilized'."
    sheet.cell(row=6, column=1).value = "Time Total by Week:  This tab shows time submitted by week in the month for all workers who have submitted some time.  The rows that are in red indicate employees that have at least one week with no time."
    sheet.cell(row=8, column=1).value = "Utilization by Worker:  This tab shows the utilization calculations completed for every worker in the Program that have submitted some time.  Utilization Status of 'Underutilized' indicates employees that had an Actual Utilization less than their Utilization Target for the month, and 'Fully Utilized' indicates that the worker had an Actual Utilization Rate equal to or greater than their Utilization Target for the month.  'Total Project Hours' indicates how many Project Hours were coded in the month.  'Actual Utilization Rate' indicates Total Project Hours divided by Total Hours available in the month.  'Utilization Target indicates the threshold of utilization needed for an employee to hit to become Fully Utilized.  ''Needed Project Hours' indicates how many Project Hours were needed in order for the employee to be fully utilized and not have any admin charge."
    sheet.cell(row=10, column=1).value = "Expenses by Program:  This tab shows the sum of actual monthly salary, benefits, shared costs, and bonuses that will be expensed to the Admin and Project projects in your Program.  It also shows the actual cost per project hour for all employees based on their utilization rate and whether or not they were overutilized (overutilization causes their cost per project hour to decrease)."
    sheet.cell(row=12, column=1).value = "Admin by Program:  This tab shows the sum total of all of the Admin coded in the month for employees that have a projected Admin charge.  Only employees that are underutilized will show up here, and their current Admin cost to hit will show in column B."
    sheet.cell(row=14, column=1).value = "PTO Bank Usage by Program:  This tab shows the usage from the PTO Bank by employee for the month."
    sheet.cell(row=16, column=1).value = "Employee Cost Breakdown:  This tab shows the total cost of all employees hitting your Program by Project and task."
    sheet.merge_cells('A2:O2')
    sheet.merge_cells('A4:O4')
    sheet.merge_cells('A6:O6')
    sheet.merge_cells('A8:O8')
    sheet.merge_cells('A10:O10')
    sheet.merge_cells('A12:O12')
    sheet.merge_cells('A14:O14')
    sheet.merge_cells('A16:O16')
    sheet['A2'].alignment = Alignment(wrap_text=True)
    sheet['A4'].alignment = Alignment(wrap_text=True)
    sheet['A6'].alignment = Alignment(wrap_text=True)
    sheet['A8'].alignment = Alignment(wrap_text=True)
    sheet['A10'].alignment = Alignment(wrap_text=True)
    sheet['A12'].alignment = Alignment(wrap_text=True)
    sheet['A14'].alignment = Alignment(wrap_text=True)
    sheet['A16'].alignment = Alignment(wrap_text=True)
    sheet.cell(row=1, column=1).font  = Font(size = "26")
    sheet.row_dimensions[2].height = 58
    sheet.row_dimensions[4].height = 54
    sheet.row_dimensions[6].height = 54
    sheet.row_dimensions[8].height = 100
    sheet.row_dimensions[10].height = 48
    sheet.row_dimensions[12].height = 40
    sheet.row_dimensions[14].height = 40
    sheet.row_dimensions[16].height = 40

    wb.save(Timepath+'/' + i  + filename)
    wb.close()
  


## Missing Timesheets Report

In [590]:
def CreateMissingTimesheets(program):
    firstdf = AllCostsAllocated[AllCostsAllocated['Cost Center']==program]
    seconddf = firstdf[firstdf['Admin Hours Percentage']==1]  
    thirdf = seconddf[['Worker', 'Type']]
    return thirdf

In [591]:
def to_excel_MissingTimeSheets(df, file):
    with pd.ExcelWriter(file, engine='openpyxl', mode='a') as writer:
        # Load the existing workbook
        writer.book = openpyxl.load_workbook(file)
        
        # Write the DataFrame to the specified sheet
        df.to_excel(writer, sheet_name="NO TIME or ALL ADMIN", startrow=1, index=False)
        
        # Save the workbook
        writer.save()

In [592]:
'''def to_excel_MissingTimeSheets(df, file):
    # Convert DataFrame to list of dictionaries
    data = df.to_dict(orient='records')
    
    # Create a new workbook or load an existing one
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
    else:
        book = openpyxl.Workbook()
        book.remove(book.active)  # Remove the default sheet created with a new workbook
    
    # Add a new sheet or get the existing one
    if "NO TIME or ALL ADMIN" in book.sheetnames:
        sheet = book["NO TIME or ALL ADMIN"]
    else:
        sheet = book.create_sheet(title="NO TIME or ALL ADMIN")
    
    # Write data to the sheet
    for i, row in enumerate(data, start=2):  # Start from row 2 to leave the first row for headers
        for j, (key, value) in enumerate(row.items(), start=1):
            if i == 2:  # Write headers in the first row
                sheet.cell(row=1, column=j, value=key)
            sheet.cell(row=i, column=j, value=value)
    
    # Save the workbook
    book.save(file)
'''

'def to_excel_MissingTimeSheets(df, file):\n    # Convert DataFrame to list of dictionaries\n    data = df.to_dict(orient=\'records\')\n    \n    # Create a new workbook or load an existing one\n    if os.path.exists(file):\n        book = openpyxl.load_workbook(file)\n    else:\n        book = openpyxl.Workbook()\n        book.remove(book.active)  # Remove the default sheet created with a new workbook\n    \n    # Add a new sheet or get the existing one\n    if "NO TIME or ALL ADMIN" in book.sheetnames:\n        sheet = book["NO TIME or ALL ADMIN"]\n    else:\n        sheet = book.create_sheet(title="NO TIME or ALL ADMIN")\n    \n    # Write data to the sheet\n    for i, row in enumerate(data, start=2):  # Start from row 2 to leave the first row for headers\n        for j, (key, value) in enumerate(row.items(), start=1):\n            if i == 2:  # Write headers in the first row\n                sheet.cell(row=1, column=j, value=key)\n            sheet.cell(row=i, column=j, value=value

In [593]:
#dftest = CreateMissingTimesheets('Carbon-Free Buildings')

In [594]:
def Format_NoTimePivot(file):
    book = load_workbook(file)
    sheet = book["NO TIME or ALL ADMIN"] #active means last opened sheet
    rd = sheet.row_dimensions[1]
    rd.height = 25
    sheet['A2'].font = Font(bold=True)
    sheet['B2'].font = Font(bold=True)
    sheet.column_dimensions['A'].width = 44
    sheet.column_dimensions['B'].width = 44
#    sheet.column_dimensions['C'].width = 44
#    fontStyle = Font(size = "20")
    sheet.merge_cells('A1:B1')
    top_left_cell = sheet['A1']
    top_right_cell = sheet['B1']
    top_left_cell.value = timestampreport + " MISSING TIMESHEETS OR ALL ADMIN"
    thin = Side(border_style="thin", color="000000")
    double = Side(border_style="double", color="000000")
    top_left_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    top_left_cell.fill = PatternFill("solid", fgColor="003366")
#    top_left_cell.fill = fill = GradientFill(stop=("000080", "FFFFFF"))
    top_left_cell.font  = Font(color="FFFFFF", size = "26")
#    top_left_cell.font = fontStyle
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    top_right_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    sheet.sheet_properties.tabColor = 'FF0000'
    sheet.sheet_view.zoomScale = 120
#    sheet.column_dimensions['A'].hidden= True
    book.save(file)

In [595]:
def MakeMISSINGTIME(Group, link):
#    step1 = CreateMissingTimesheets(Group).reset_index(drop=True)
    step1 = CreateMissingTimesheets(Group)
    to_excel_MissingTimeSheets(step1, link)
    Format_NoTimePivot(link)

In [596]:
#MakeMISSINGTIME('Carbon-Free Buildings', './Outputs/ProgramReports/Carbon-Free Buildings' + ' - ' +timestampreport + filename)

# Create Hours Reports for Employees with Some Time

In [597]:
TimeDataDFALL[TimeDataDFALL['MonthYear']==currentmonthandyear]['Week'].value_counts()

09/09/2024 - 09/15/2024    8222
09/16/2024 - 09/22/2024    8106
09/23/2024 - 09/29/2024    7917
09/02/2024 - 09/08/2024    7332
09/30/2024 - 10/06/2024    1509
08/26/2024 - 09/01/2024      26
Name: Week, dtype: int64

In [598]:
TimeDataThisMonth1 = TimeDataDFALL[TimeDataDFALL['MonthYear']==currentmonthandyear]

TimeDataThisMonth = TimeDataThisMonth1[TimeDataThisMonth1['Week']!= '08/26/2024 - 09/01/2024']

In [599]:
def CreateTimeDataSet(program):
    firstdf = TimeDataThisMonth[TimeDataThisMonth['Worker Cost Center']==program]
    return firstdf

In [600]:
def CreateTimeSetPivot(Group):
#    Month_order = ['Jul 2022', 'Aug 2022', 'Sep 2022', 'Oct 2022', 'Nov 2022', 'Dec 2022', 'Jan 2023', 'Feb 2023', 'Mar 2023', 'Apr 2023', 'May 2023', 'Jun 2023', 'Jul 2023', 'Aug 2023', 'Sep 2023', 'Oct 2023', 'Nov 2023', 'Dec 2023', 'Jan 2024', 'Feb 2024', 'Mar 2024', 'Apr 2024', 'May 2024', 'Jun 2024']
    table = pd.pivot_table(Group, index = ['Worker', 'Utilization Target'], values = 'Reported Quantity', columns = 'Week', aggfunc="sum")
#    table2 = table.reindex(Month_order, axis=1)
    table['Total'] = table.iloc[:,-5:].sum(axis=1)
    return table

In [601]:
def to_excel_TimePivot(df, file):
    writer = pd.ExcelWriter(file, engine='openpyxl')
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book

    df.to_excel(writer, sheet_name="Time Total by Week", startrow = 1)
    writer.save()
    writer.close()

In [602]:
def Format_TimePivot(file):
    book = load_workbook(file)
    sheet = book["Time Total by Week"] #active means last opened sheet
    rd = sheet.row_dimensions[1]
    rd.height = 25
    sheet['A2']= "Worker"
    sheet['B2']= 'Utilization Rate'
    sheet['A2'].font = Font(bold=True)
    sheet['B2'].font = Font(bold=True)
#    fontStyle = Font(size = "20")
    sheet.merge_cells('A1:H1')
    top_left_cell = sheet['A1']
    top_right_cell = sheet['H1']
    top_left_cell.value = timestampreport + " Hour Summary*"
    thin = Side(border_style="thin", color="000000")
    double = Side(border_style="double", color="000000")
    top_left_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    top_left_cell.fill = PatternFill("solid", fgColor="003366")
#    top_left_cell.fill = fill = GradientFill(stop=("000080", "FFFFFF"))
    top_left_cell.font  = Font(color="FFFFFF", size = "26")
#    top_left_cell.font = fontStyle
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    top_right_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    sheet.column_dimensions['A'].width = 24
    sheet.column_dimensions['B'].width = 24
#    sheet.column_dimensions['E'].width = 15
#    sheet.column_dimensions['F'].width = 15
#    sheet.column_dimensions['G'].width = 15
#    sheet.column_dimensions['H'].width = 15
#    sheet.column_dimensions['I'].width = 15
#    sheet.column_dimensions['J'].width = 15
#    sheet.column_dimensions['K'].width = 15
#    sheet.column_dimensions['L'].width = 15
#    sheet.column_dimensions['M'].width = 15
#    sheet.column_dimensions['N'].width = 15
#    sheet.column_dimensions['O'].width = 15
#    sheet.column_dimensions['P'].width = 15
#    sheet.column_dimensions['Q'].width = 15
#    sheet.column_dimensions['D'].hidden = True
    num_rows = sheet.max_row
    curr_row = 2
    last_col = 9
    bluesectionrightborder = 3
    for col in range(curr_row, curr_row+1):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '0.00%'
    for col in range(last_col, last_col+1):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).border = Border(left=thin)   
    for col in range(1, bluesectionrightborder):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = blueFill
    redrows = set()           
    for row in range(3, num_rows+1):
        for i in range(3,8):
            if sheet.cell(row=row, column=i).value is None:
                redrows.add(row)
    for row in range(3, num_rows):
        if row in redrows:
            for col in range(1, last_col):
                sheet.cell(column=col, row=row).fill = redFill 

                    
    print(redrows)               
#    for col in range(17, 19):
#        for row in range(2, num_rows+1):
#            sheet.cell(column=col, row=row).border = Border(left=thin)          
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == 'Worker':
            for j in range(1,last_col):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(top = thin, bottom = thin)   
                sheet.cell(column=j, row=i).fill = greyFill
    for col in range(last_col-1, last_col):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).border = Border(left=thin)

    for col in range(1, 3):
        for row in range(3, num_rows+1):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal='left', vertical='center')
            sheet.cell(column=col, row=row).font = Font(bold=True, size = "12")
#    sheet.auto_filter.ref = sheet.dimensions
    sheet.sheet_view.zoomScale = 100
    sheet.freeze_panes = "C3"
#    sheet.sheet_state = 'hidden'
    book.save(file)

In [603]:
def MakeTIMEREPORT(Group, link):
    step1 = CreateTimeDataSet(Group)
    step2 = CreateTimeSetPivot(step1)
    to_excel_TimePivot(step2, link)
    Format_TimePivot(link)

# Make CFO Table

In [604]:
ALLTABLE = ['Rocky Mountain Institute']

for i in ALLTABLE:   
    wb = load_workbook(TimeTemplatepath)
    ws = wb['Cover']   
    img = openpyxl.drawing.image.Image(Logopath)
    img.anchor = 'A1'
    ws.add_image(img)
    ws.cell(row = 14, column = 1).value = i

    '''    wb.create_sheet('Guide')
    sheet = wb["Guide"]
    sheet.cell(row=1, column=1).value = "Guidance For Use of Program Financial Package" 
    sheet.cell(row=2, column=1).value = "Note: The Program Financial Package is a continually evolving report designed by Program Operations Leads in order to give better insight into data from Workday and Salesforce.  If there are data manipulations from those databases that you would like to include in the Program Financial Package in the future, please reach out to Jeremy Wendt or Tobias Lutz."
    sheet.merge_cells('A2:O2')
    sheet.merge_cells('A4:O4')
    sheet['A2'].alignment = Alignment(wrap_text=True)
    sheet['A4'].alignment = Alignment(wrap_text=True)
    sheet.cell(row=1, column=1).font  = Font(size = "26")
    sheet.row_dimensions[2].height = 44
    sheet.row_dimensions[4].height = 58'''

    wb.save(str(Timepath+'/' + i +  ' - ' + timestampreport + filename))
    wb.close()

In [605]:

RockyMtnDF = TimeDataDFALL[TimeDataDFALL['MonthYear']==currentmonthandyear]    


In [606]:
def CreateTimeSetPivotMASTER(Group):
#    Month_order = ['Jul 2022', 'Aug 2022', 'Sep 2022', 'Oct 2022', 'Nov 2022', 'Dec 2022', 'Jan 2023', 'Feb 2023', 'Mar 2023', 'Apr 2023', 'May 2023', 'Jun 2023', 'Jul 2023', 'Aug 2023', 'Sep 2023', 'Oct 2023', 'Nov 2023', 'Dec 2023', 'Jan 2024', 'Feb 2024', 'Mar 2024', 'Apr 2024', 'May 2024', 'Jun 2024']
    table = pd.pivot_table(Group, index = ['Worker Cost Center', 'Worker', 'Utilization Target'], values = 'Reported Quantity', columns = 'Week', aggfunc="sum")
#    table2 = table.reindex(Month_order, axis=1)
    table['Total'] = table.iloc[:,-4:].sum(axis=1)
    return table

In [607]:
def to_excel_TimePivotMASTER(df, file):
    writer = pd.ExcelWriter(file, engine='openpyxl')
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book

    df.to_excel(writer, sheet_name="Time Total by Week", startrow = 1)
    writer.save()
    writer.close()

In [608]:
def Format_TimePivotMASTER(file):
    book = load_workbook(file)
    sheet = book["Time Total by Week"] #active means last opened sheet
    rd = sheet.row_dimensions[1]
    rd.height = 25
    sheet['A2']= "Worker Cost Center"
    sheet['B2']= "Worker"
    sheet['C2']= 'Utilization Rate'
    sheet['A2'].font = Font(bold=True)
    sheet['B2'].font = Font(bold=True)
#    fontStyle = Font(size = "20")
    sheet.merge_cells('A1:I1')
    top_left_cell = sheet['A1']
    top_right_cell = sheet['I1']
    top_left_cell.value = timestampreport + " Hour Summary*"
    thin = Side(border_style="thin", color="000000")
    double = Side(border_style="double", color="000000")
    top_left_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    top_left_cell.fill = PatternFill("solid", fgColor="003366")
#    top_left_cell.fill = fill = GradientFill(stop=("000080", "FFFFFF"))
    top_left_cell.font  = Font(color="FFFFFF", size = "26")
#    top_left_cell.font = fontStyle
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    top_right_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    sheet.column_dimensions['A'].width = 24
    sheet.column_dimensions['B'].width = 24
#    sheet.column_dimensions['E'].width = 15
#    sheet.column_dimensions['F'].width = 15
#    sheet.column_dimensions['G'].width = 15
#    sheet.column_dimensions['H'].width = 15
#    sheet.column_dimensions['I'].width = 15
#    sheet.column_dimensions['J'].width = 15
#    sheet.column_dimensions['K'].width = 15
#    sheet.column_dimensions['L'].width = 15
#    sheet.column_dimensions['M'].width = 15
#    sheet.column_dimensions['N'].width = 15
#    sheet.column_dimensions['O'].width = 15
#    sheet.column_dimensions['P'].width = 15
#    sheet.column_dimensions['Q'].width = 15
#    sheet.column_dimensions['D'].hidden = True
    num_rows = sheet.max_row
    curr_row = 2
    last_col = 10
    bluesectionrightborder = 4
    for col in range(curr_row+1, curr_row+2):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '0.00%'
    for col in range(last_col, last_col+1):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).border = Border(left=thin)   
    for col in range(1, bluesectionrightborder):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = blueFill
    redrows = set()           
    for row in range(3, num_rows+1):
        for i in range(4,9):
            if sheet.cell(row=row, column=i).value is None:
                redrows.add(row)
    for row in range(3, num_rows):
        if row in redrows:
            for col in range(2, last_col):
                sheet.cell(column=col, row=row).fill = redFill 

                    
    print(redrows)               
#    for col in range(17, 19):
#        for row in range(2, num_rows+1):
#            sheet.cell(column=col, row=row).border = Border(left=thin)          
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == 'Worker Cost Center':
            for j in range(1,last_col):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(top = thin, bottom = thin)   
                sheet.cell(column=j, row=i).fill = greyFill
    for col in range(last_col-1, last_col):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).border = Border(left=thin)

    for col in range(1, bluesectionrightborder):
        for row in range(3, num_rows+1):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal='left', vertical='center')
            sheet.cell(column=col, row=row).font = Font(bold=True, size = "12")
#    sheet.auto_filter.ref = sheet.dimensions
    sheet.sheet_view.zoomScale = 100
    sheet.freeze_panes = "D3"
#    sheet.sheet_state = 'hidden'
    book.save(file)

In [609]:
def MakeTIMEREPORTMASTER(Group, link):
    step2 = CreateTimeSetPivotMASTER(Group)
    to_excel_TimePivotMASTER(step2, link)
    Format_TimePivotMASTER(link)

# Funding Source Table

In [610]:
def CreateFundingSourceRules(program):
    firstdf = FundingSourceReportFINAL[FundingSourceReportFINAL['Cost Center']==program]  
    return firstdf

In [611]:
def CreateFundingSourceRulesPivot(Group):
#    Month_order = ['Jul 2022', 'Aug 2022', 'Sep 2022', 'Oct 2022', 'Nov 2022', 'Dec 2022', 'Jan 2023', 'Feb 2023', 'Mar 2023', 'Apr 2023', 'May 2023', 'Jun 2023', 'Jul 2023', 'Aug 2023', 'Sep 2023', 'Oct 2023', 'Nov 2023', 'Dec 2023', 'Jan 2024', 'Feb 2024', 'Mar 2024', 'Apr 2024', 'May 2024', 'Jun 2024']
    table = pd.pivot_table(Group, index = ['Funding Source', 'Project', 'Project Plan Task', 'Worker', 'Hours'], values = 'Total Cost', aggfunc="sum")
#    table2 = table.reindex(Month_order, axis=1)
    return table

In [612]:
def add_subtotal_FundingSourceRules(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename(('','','','','Total Drawdown'))) for k, d in PivotedGroup.groupby('Funding Source')])
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out


In [613]:
def to_excel_FundingSourceRules(df, file):
    writer = pd.ExcelWriter(file, engine='openpyxl')
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book

    df.to_excel(writer, sheet_name="Funding Source Rules Summary", startrow = 1)
    writer.save()
    writer.close()

In [614]:
def Format_FundingSourceRules(file):
    book = load_workbook(file)
    sheet = book["Funding Source Rules Summary"] #active means last opened sheet
    rd = sheet.row_dimensions[1]
    rd.height = 25
    sheet['A2']= "Gift"
    sheet['B2']= 'Project'
    sheet['C2']= 'Task'
    sheet['D2']= 'Employee'
    sheet['E2']= 'Hours'
    sheet['A2'].font = Font(bold=True)
    sheet['B2'].font = Font(bold=True)
#    fontStyle = Font(size = "20")
    sheet.merge_cells('A1:F1')
    top_left_cell = sheet['A1']
    top_right_cell = sheet['F1']
    top_left_cell.value = timestampreport + " Gift Drawdown Summary"
    thin = Side(border_style="thin", color="000000")
    double = Side(border_style="double", color="000000")
    top_left_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    top_left_cell.fill = PatternFill("solid", fgColor="003366")
#    top_left_cell.fill = fill = GradientFill(stop=("000080", "FFFFFF"))
    top_left_cell.font  = Font(color="FFFFFF", size = "26")
#    top_left_cell.font = fontStyle
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    top_right_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    sheet.column_dimensions['A'].width = 24
    sheet.column_dimensions['B'].width = 34
    sheet.column_dimensions['C'].width = 34
    sheet.column_dimensions['D'].width = 24
    sheet.column_dimensions['E'].width = 16
    num_rows = sheet.max_row
    curr_row = 2
    last_col = 8
    bluesectionrightborder = 5
    for col in range(1, bluesectionrightborder):
        for row in range(3, num_rows+1):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = blueFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Gift":
            for j in range(1,last_col-1):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(top = thin, bottom = thin)  
                sheet.cell(column=j, row=i).fill = greyFill
 
    for col in range(1, bluesectionrightborder):
        for row in range(3, num_rows+1):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal='left', vertical='center')
            sheet.cell(column=col, row=row).font = Font(bold=True, size = "12")   
    for col in range(1, 2):
        greystartrows = []
        greyendrows = []
        for i in range(3,num_rows+1):
            if sheet.cell(row=i, column=col).value:
                greystartrows.append(i)
        del  greystartrows[1::2]
        for i in range(3,num_rows+1):
            if sheet.cell(row=i, column=col).value:
                greyendrows.append(i-1)
        del  greyendrows[0::2]

        
    if len(greystartrows) != len(greyendrows):
        print('mismatch')
        print(len(greystartrows))
        print(len(greyendrows))
        greystartrows.pop()
    for col in range(1, bluesectionrightborder):
        for row in range(3, num_rows):
            for i in range(len(greystartrows)):
                if row == greystartrows[i]:
                    for row in range(greystartrows[i],greyendrows[i]):
                        sheet.cell(column=col, row=row).fill = lightblueFill 
                try:
                    if sheet.cell(row=greyendrows[i], column=bluesectionrightborder).value == "Total Drawdown":
                        sheet.cell(column=bluesectionrightborder-1, row=greyendrows[i]).fill = lightblueFill
                except:
                    pass

    for col in range(6,  last_col):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '"$"#,##0;[Red]("$"#,##0)'

        
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=bluesectionrightborder).value == "Total Drawdown":
            for j in range(bluesectionrightborder,last_col-1):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style="double")) 
    sheet.sheet_view.zoomScale = 100
    sheet.freeze_panes = "A3"
    sheet.sheet_state = 'hidden'

    book.save(file)


In [615]:
def MakeFundingSourceRules(Group, link):
    step1 = CreateFundingSourceRules(Group)
    step2 = CreateFundingSourceRulesPivot(step1)
    step3 = add_subtotal_FundingSourceRules(step2)
    to_excel_FundingSourceRules(step3, link)
    Format_FundingSourceRules(link)


In [616]:

#MakeFundingSourceRules("Carbon-Free Buildings", str(Timepath+'/' + "Carbon-Free Buildings"  + ' - ' +timestampreport + filename))


# Create Project Time Table

In [617]:
EmployeeTimeProjectLookupTable.columns

Index(['Employee ID', 'Talent Pool', 'Worker', 'Worker Cost Center', 'Project',
       'Hire Date', 'Termination Date', 'Needed Project Hours',
       'Total Employee Project Hours', 'Project Group',
       'Actual Utilization Rate', 'Utilization Target', 'Total Project Hours',
       'Fully Utilized'],
      dtype='object')

In [618]:
EmployeeTimeProjectLookupTable['Project Group'].value_counts()

Grant Funded              1816
Holiday                    638
Customer Funded            473
Admin                      462
PTO                        418
Unrestricted               187
Multilateral/Bilateral      87
ACF                         85
US Government               45
Name: Project Group, dtype: int64

In [619]:
len(EmployeeTimeProjectLookupTable)

4211

In [620]:
EmployeeTimeProjectLookupTablePROJECTS1 = EmployeeTimeProjectLookupTable[EmployeeTimeProjectLookupTable['Project Group']!='Admin']
EmployeeTimeProjectLookupTablePROJECTS2 = EmployeeTimeProjectLookupTablePROJECTS1[EmployeeTimeProjectLookupTablePROJECTS1['Project Group']!='Holiday']
EmployeeTimeProjectLookupTablePROJECTS = EmployeeTimeProjectLookupTablePROJECTS2[EmployeeTimeProjectLookupTablePROJECTS2['Project Group']!='PTO']

In [621]:
EmployeeTimeProjectLookupTablePROJECTS['Project Group'].value_counts()

Grant Funded              1816
Customer Funded            473
Unrestricted               187
Multilateral/Bilateral      87
ACF                         85
US Government               45
Name: Project Group, dtype: int64

In [622]:
def CreateProjectTimeSet(program):
    firstdf = EmployeeTimeProjectLookupTablePROJECTS[EmployeeTimeProjectLookupTablePROJECTS['Worker Cost Center']==program]  
    return firstdf

In [623]:
def CreateProjectTimePivot(Group):
#    Month_order = ['Jul 2022', 'Aug 2022', 'Sep 2022', 'Oct 2022', 'Nov 2022', 'Dec 2022', 'Jan 2023', 'Feb 2023', 'Mar 2023', 'Apr 2023', 'May 2023', 'Jun 2023', 'Jul 2023', 'Aug 2023', 'Sep 2023', 'Oct 2023', 'Nov 2023', 'Dec 2023', 'Jan 2024', 'Feb 2024', 'Mar 2024', 'Apr 2024', 'May 2024', 'Jun 2024']
    table = pd.pivot_table(Group, index = ['Worker', 'Fully Utilized', 'Actual Utilization Rate', 'Utilization Target', 'Total Project Hours', 'Needed Project Hours', 'Talent Pool', 'Project'], values = 'Total Employee Project Hours', aggfunc="sum")
#    table2 = table.reindex(Month_order, axis=1)
    return table

In [624]:
def add_subtotal_ProjectPivot(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename(('','','','','','','','Total Hours'))) for k, d in PivotedGroup.groupby('Worker')])
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [625]:
def to_excel_ProjectTime(df, file):
    writer = pd.ExcelWriter(file, engine='openpyxl')
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book

    df.to_excel(writer, sheet_name="Utilization by Worker", startrow = 1)
    writer.save()
    writer.close()

In [626]:
def Format_ProjectTime(file):
    book = load_workbook(file)
    sheet = book["Utilization by Worker"] #active means last opened sheet
    rd = sheet.row_dimensions[1]
    rd.height = 25
    sheet['A2']= "Employee"
    sheet['B2']= 'Utilization Status'
    sheet['C2']= 'Actual Utilization'
    sheet['D2']= 'Utilization Target'
    sheet['E2']= 'Total Project Hours'
    sheet['F2']= 'Total Needed Hours'
    sheet['G2']= 'Talent Pool'
    sheet['H2']= 'Project'
    sheet['A2'].font = Font(bold=True)
    sheet['B2'].font = Font(bold=True)
#    fontStyle = Font(size = "20")
    sheet.merge_cells('A1:I1')
    top_left_cell = sheet['A1']
    top_right_cell = sheet['I1']
    top_left_cell.value = timestampreport + " Utilization by Worker"
    thin = Side(border_style="thin", color="000000")
    double = Side(border_style="double", color="000000")
    top_left_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    top_left_cell.fill = PatternFill("solid", fgColor="003366")
#    top_left_cell.fill = fill = GradientFill(stop=("000080", "FFFFFF"))
    top_left_cell.font  = Font(color="FFFFFF", size = "26")
#    top_left_cell.font = fontStyle
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    top_right_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    sheet.column_dimensions['A'].width = 20
    sheet.column_dimensions['B'].width = 20
    sheet.column_dimensions['C'].width = 16
    sheet.column_dimensions['D'].width = 16
    sheet.column_dimensions['G'].width = 26
    sheet.column_dimensions['H'].width = 50
    sheet.column_dimensions['I'].width = 16
    num_rows = sheet.max_row
    curr_row = 2
    last_col = 11
    bluesectionrightborder = 8
    for col in range(3, 5):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '0.00%'

    for col in range(1, bluesectionrightborder):
        for row in range(3, num_rows+1):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = blueFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Employee":
            for j in range(1,last_col-1):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(top = thin, bottom = thin)  
                sheet.cell(column=j, row=i).fill = greyFill
 
    for col in range(1, bluesectionrightborder):
        for row in range(3, num_rows+1):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal='left', vertical='center')
            sheet.cell(column=col, row=row).font = Font(bold=True, size = "12")   
    for col in range(1, 2):
        greystartrows = []
        greyendrows = []
        for i in range(3,num_rows+1):
            if sheet.cell(row=i, column=col).value:
                greystartrows.append(i)
        del  greystartrows[1::2]
        for i in range(3,num_rows+1):
            if sheet.cell(row=i, column=col).value:
                greyendrows.append(i-1)
        del  greyendrows[0::2]

        
    if len(greystartrows) != len(greyendrows):
        print('mismatch')
        print(len(greystartrows))
        print(len(greyendrows))
        greystartrows.pop()
    for col in range(1, bluesectionrightborder):
        for row in range(3, num_rows):
            for i in range(len(greystartrows)):
                if row == greystartrows[i]:
                    for row in range(greystartrows[i],greyendrows[i]):
                        sheet.cell(column=col, row=row).fill = lightblueFill 
                try:
                    if sheet.cell(row=greyendrows[i], column=bluesectionrightborder).value == "Total Hours":
                        sheet.cell(column=bluesectionrightborder-1, row=greyendrows[i]).fill = lightblueFill
                except:
                    pass


        
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=bluesectionrightborder).value == "Total Hours":
            for j in range(bluesectionrightborder,last_col-1):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style="double")) 
    sheet.sheet_view.zoomScale = 100
    sheet.freeze_panes = "A3"
#    sheet.sheet_state = 'hidden'

    book.save(file)


In [627]:
def MakeProjectTime(Group, link):
    step1 = CreateProjectTimeSet(Group)
    step2 = CreateProjectTimePivot(step1)
    step3 = add_subtotal_ProjectPivot(step2)
    to_excel_ProjectTime(step3, link)
    Format_ProjectTime(link)


# Create PTO Bank Pivot

In [628]:
PTOBankFINAL.columns

Index(['Worker', 'Employee ID', 'Project Plan Task', 'Project Plan Task ID',
       'SalaryCosted', 'BenefitsCosted', 'SharedCosted', 'BonusCosted',
       'Initiative', 'Cost Center', 'ConsultingFeesCosted', 'Type',
       'Project Hours Percentage', 'Admin Hours Percentage', 'Lobbying',
       'Lobbying Location', 'Project or Admin', 'Project',
       'Worker Cost Center', 'PTO more than Admin', 'Location',
       'Total Project Hours', 'Needed Project Hours', 'Adjusted Hours',
       'Actual Utilization Rate', 'Cost per Project Hour',
       'Utilization Target', 'Talent Pools', 'Total Cost', 'File',
       'Project PTO or Admin', 'Standard Monthly Cost',
       'Amount Banked per Hour', 'Amount Banked', 'Cost per PTO Hour',
       'Project & PTO Hours', 'Bank Month', 'Project Task', 'Project ID',
       'Funding Source Rule', 'Project/Project Hierarchy',
       'Company/Company Hierarchy', 'Evaluation Date Option',
       'Default Funding Source', 'Inactive', 'Funding Source Condit

In [629]:

def CreatePTOBankFile(program):
    firstdf = PTOBankFINAL[PTOBankFINAL['Worker Cost Center']==program]  
    return firstdf

In [630]:
def CreateCreatePTOBankFilePivot(Group):
#    Month_order = ['Jul 2022', 'Aug 2022', 'Sep 2022', 'Oct 2022', 'Nov 2022', 'Dec 2022', 'Jan 2023', 'Feb 2023', 'Mar 2023', 'Apr 2023', 'May 2023', 'Jun 2023', 'Jul 2023', 'Aug 2023', 'Sep 2023', 'Oct 2023', 'Nov 2023', 'Dec 2023', 'Jan 2024', 'Feb 2024', 'Mar 2024', 'Apr 2024', 'May 2024', 'Jun 2024']
    table = pd.pivot_table(Group, index = ['Worker', 'Funding Source', 'Project', 'Project Plan Task', 'Project Plan Task ID', 'Standard Monthly Cost', 'Cost per Project Hour', 'Amount Banked per Hour', 'Hours'], values = 'Amount Banked', aggfunc="sum")
#    table2 = table.reindex(Month_order, axis=1)
    return table

In [631]:
def add_subtotal_PTOBankFile(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename(('','','','','','','','','Total Amount'))) for k, d in PivotedGroup.groupby('Worker')])
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [632]:
def to_excel_PTOBankFile(df, file):
    writer = pd.ExcelWriter(file, engine='openpyxl')
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book

    df.to_excel(writer, sheet_name="PTOBank by Worker", startrow = 1)
    writer.save()
    writer.close()

In [633]:
def Format_PTOBankFile(file):
    book = load_workbook(file)
    sheet = book["PTOBank by Worker"] #active means last opened sheet
    rd = sheet.row_dimensions[1]
    rd.height = 25
    sheet['A2']= "Employee"
    sheet['B2']= 'Funding Source'
    sheet['C2']= 'Project'
    sheet['D2']= 'Project Plan Task'
    sheet['E2']= 'Project Plan Task ID'
    sheet['F2']= 'Standard Monthly Cost'
    sheet['G2']= 'Cost per Project Hour'
    sheet['H2']= 'Amount Banked per Hour'
    sheet['I2']= 'Hours'
    sheet['A2'].font = Font(bold=True)
    sheet['B2'].font = Font(bold=True)
#    fontStyle = Font(size = "20")
    sheet.merge_cells('A1:J1')
    top_left_cell = sheet['A1']
    top_right_cell = sheet['J1']
    top_left_cell.value = timestampreport + " PTO Bank Summary"
    thin = Side(border_style="thin", color="000000")
    double = Side(border_style="double", color="000000")
    top_left_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    top_left_cell.fill = PatternFill("solid", fgColor="003366")
#    top_left_cell.fill = fill = GradientFill(stop=("000080", "FFFFFF"))
    top_left_cell.font  = Font(color="FFFFFF", size = "26")
#    top_left_cell.font = fontStyle
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    top_right_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    sheet.column_dimensions['A'].width = 20
    sheet.column_dimensions['B'].width = 26
    sheet.column_dimensions['C'].width = 26
    sheet.column_dimensions['D'].width = 26
    sheet.column_dimensions['E'].width = 26
    sheet.column_dimensions['G'].width = 16
    sheet.column_dimensions['H'].width = 16
    sheet.column_dimensions['I'].width = 16
    sheet.column_dimensions['J'].width = 16
    num_rows = sheet.max_row
    curr_row = 2
    last_col = 12
    bluesectionrightborder = 9
    for col in range(6, 9):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '"$"#,##0;[Red]("$"#,##0)'
    for col in range(10, 11):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '"$"#,##0;[Red]("$"#,##0)'

    for col in range(1, bluesectionrightborder):
        for row in range(3, num_rows+1):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = blueFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Employee":
            for j in range(1,last_col-1):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(top = thin, bottom = thin)  
                sheet.cell(column=j, row=i).fill = greyFill
 
    for col in range(1, bluesectionrightborder):
        for row in range(3, num_rows+1):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal='left', vertical='center')
            sheet.cell(column=col, row=row).font = Font(bold=True, size = "12")   
    for col in range(1, 2):
        greystartrows = []
        greyendrows = []
        for i in range(3,num_rows+1):
            if sheet.cell(row=i, column=col).value:
                greystartrows.append(i)
        del  greystartrows[1::2]
        for i in range(3,num_rows+1):
            if sheet.cell(row=i, column=col).value:
                greyendrows.append(i-1)
        del  greyendrows[0::2]

        
    if len(greystartrows) != len(greyendrows):
        print('mismatch')
        print(len(greystartrows))
        print(len(greyendrows))
        greystartrows.pop()
    for col in range(1, bluesectionrightborder):
        for row in range(3, num_rows):
            for i in range(len(greystartrows)):
                if row == greystartrows[i]:
                    for row in range(greystartrows[i],greyendrows[i]):
                        sheet.cell(column=col, row=row).fill = lightblueFill 
                try:
                    if sheet.cell(row=greyendrows[i], column=bluesectionrightborder).value == "Total Amount":
                        sheet.cell(column=bluesectionrightborder-1, row=greyendrows[i]).fill = lightblueFill
                except:
                    pass


        
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=bluesectionrightborder).value == "Total Amount":
            for j in range(bluesectionrightborder,last_col-1):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style="double")) 
    sheet.sheet_view.zoomScale = 100
    sheet.freeze_panes = "A3"
#    sheet.sheet_state = 'hidden'

    book.save(file)

In [634]:
def MakePTOBankFile(Group, link):
    step1 = CreatePTOBankFile(Group)
    step2 = CreateCreatePTOBankFilePivot(step1)
    step3 = add_subtotal_PTOBankFile(step2)
    to_excel_PTOBankFile(step3, link)
    Format_PTOBankFile(link)


In [635]:
#MakePTOBankFile('Carbon-Free Buildings', str(Timepath+'/' + 'Carbon-Free Buildings' + ' - ' +timestampreport + filename))

# Create Whitney's Final Output Files

## Create Salary Output File

In [636]:
SalaryLoadTemplate = './Source_Files/Workday_Accounting_Journal_Salary Allocation_Template.xlsx'


Create Holding Calculations

In [637]:
SalaryHoldingDF = TransDetailBeijingDF[TransDetailBeijingDF['Spend Category']=="Salary Holding"]

HoldingGenInst = SalaryHoldingDF[SalaryHoldingDF['Initiative']=="GEN - General Admin"]
HoldingNotGenInst = SalaryHoldingDF[SalaryHoldingDF['Initiative']!="GEN - General Admin"]

HoldingGentotal = HoldingGenInst['Net Amount'].sum()

HoldingGentotal

709255.9900000001

In [638]:
HoldingNotGenInsttotal = HoldingNotGenInst['Net Amount'].sum()
HoldingNotGenInsttotal

5723404.88

Create China Calculations

In [639]:
ChinaHoldingDF = TransDetailBeijingDF[TransDetailBeijingDF['Spend Category']=="Salaries & Wages"]
ChinaHoldingDF1 = ChinaHoldingDF[ChinaHoldingDF['Initiative']=="CH - Admin"]
ChinaHoldingDF2 = ChinaHoldingDF1[ChinaHoldingDF1['Cost Center']=="China"]

In [640]:
ChinaHoldingDF2.columns

Index(['Journal', 'Journal Number', 'CF LRV Ad Hoc Bank Number',
       'Operational Transaction', 'Company', 'Status', 'Accounting Date',
       'Source', 'Ledger', 'Currency', 'Ledger Account', 'Ledger Debit Amount',
       'Ledger Credit Amount', 'Net Amount', 'Line Memo',
       'Intercompany Affiliate', 'Bank Account', 'Revenue Category',
       'Spend Category', 'Cost Center', 'Initiative', 'Project', 'Project ID',
       'Project Plan Task', 'Project Plan Task ID', 'Gift', 'Fund',
       'Funding Source', 'Customer', 'Supplier', 'Employee', 'Employee ID',
       'Location', 'Worktags'],
      dtype='object')

In [641]:
ChinaHoldingDatatoCopy = ChinaHoldingDF2[['Net Amount', 'Cost Center', 'Initiative','Project Plan Task ID', 'Employee', 'Employee ID']]

In [642]:
len(ChinaHoldingDatatoCopy)

37

In [643]:
ChinaHoldingDatatoFinal = ChinaHoldingDatatoCopy.groupby('Employee ID')['Net Amount'].sum().reset_index(name='Net Amount')

In [644]:
ChinaHoldingDatatoFinal['Employee ID'] = ChinaHoldingDatatoFinal['Employee ID'].astype(int)
ChinaHoldingDatatoFinal['Employee ID'] = ChinaHoldingDatatoFinal['Employee ID'].apply(str)
ChinaHoldingDatatoFinal['Employee ID'] = ChinaHoldingDatatoFinal['Employee ID'].apply(add_zeros)

In [645]:
ChinaHoldingDatatoFinal['Net Amount'].sum()

290929.37000000005

In [646]:
len(ChinaHoldingDatatoFinal)

33

In [647]:
ChinaHoldingDatatoFinal.head()

,Employee ID,Net Amount
0,000001,25996.58
1,000004,9426.47
2,000005,12632.50
3,000006,10129.61
4,000007,9229.44


In [648]:
len(AllCostsAllocatedPTOAccruals)

5061

In [649]:
AllCostsAllocatedPTOAccruals['SalaryCosted'].sum()

6723590.177706925

In [650]:
SalaryDatatoCopy = AllCostsAllocatedPTOAccruals.loc[(AllCostsAllocatedPTOAccruals[["SalaryCosted"]]!=0).any(axis=1)]

In [651]:
SalaryDatatoCopy = SalaryDatatoCopy.groupby(['Worker', 'Worker Cost Center','Employee ID', 'Project', 'Project Plan Task', 'Project Plan Task ID', 'Cost Center', 'Initiative', 'Lobbying', 'Lobbying Location', 'Location'], as_index=False).sum()

In [652]:
SalaryDatatoCopy.columns

Index(['Worker', 'Worker Cost Center', 'Employee ID', 'Project',
       'Project Plan Task', 'Project Plan Task ID', 'Cost Center',
       'Initiative', 'Lobbying', 'Lobbying Location', 'Location', 'Total Cost',
       'ConsultingFeesCosted', 'Admin Hours Percentage', 'Total Project Hours',
       'Needed Project Hours', 'Adjusted Hours', 'Actual Utilization Rate',
       'Standard Monthly Cost', 'Amount Banked per Hour', 'Amount Banked',
       'Cost per Project Hour', 'Utilization Target', 'Cost per PTO Hour',
       'Project & PTO Hours', 'SalaryPercentage', 'BenefitsPercentage',
       'SharedPercentage', 'BonusPercentage', 'SalaryCosted', 'BenefitsCosted',
       'SharedCosted', 'BonusCosted', 'TaskDrawdowns'],
      dtype='object')

In [653]:
len(SalaryDatatoCopy)


3551

In [654]:
SalaryDatatoCopy['SalaryCosted'].sum()

6723590.177706924

In [655]:
SalaryDatatoCopy.columns

Index(['Worker', 'Worker Cost Center', 'Employee ID', 'Project',
       'Project Plan Task', 'Project Plan Task ID', 'Cost Center',
       'Initiative', 'Lobbying', 'Lobbying Location', 'Location', 'Total Cost',
       'ConsultingFeesCosted', 'Admin Hours Percentage', 'Total Project Hours',
       'Needed Project Hours', 'Adjusted Hours', 'Actual Utilization Rate',
       'Standard Monthly Cost', 'Amount Banked per Hour', 'Amount Banked',
       'Cost per Project Hour', 'Utilization Target', 'Cost per PTO Hour',
       'Project & PTO Hours', 'SalaryPercentage', 'BenefitsPercentage',
       'SharedPercentage', 'BonusPercentage', 'SalaryCosted', 'BenefitsCosted',
       'SharedCosted', 'BonusCosted', 'TaskDrawdowns'],
      dtype='object')

In [656]:
def createImportAccountingJournalSalary(file):
    book = load_workbook(file)
    sheet = book["Import Accounting Journal"] #active means last opened sheet
    sheet['N6'] = TemplateDate1
    #ws.cell(row=2, column=2).value = 2
    sheet['T6'] = MonthTemplateName + " " + str(yeartemplate) + " Salary Allocation"
    sheet2 = book["Journal Entry Line Replacement"]
#    for i in range(1,len(AllCostsAllocated[["SalaryCosted"]])):
    rowstart = 6

    for i in range(0,len(SalaryDatatoCopy)+len(ChinaHoldingDatatoFinal)+2):
        for j in range(2,3):
            sheet2.cell(row=i+rowstart, column=j).value =  1
    for i in range(0,len(SalaryDatatoCopy)+len(ChinaHoldingDatatoFinal)+2):
        for j in range(3,4):
            sheet2.cell(row=i+rowstart, column=j).value =  i+1
    for i in range(0,len(SalaryDatatoCopy)+len(ChinaHoldingDatatoFinal)+2):
        for j in range(6,7):
            sheet2.cell(row=i+rowstart, column=j).value =  5000
    for i in range(0,len(SalaryDatatoCopy)+len(ChinaHoldingDatatoFinal)+2):
        for j in range(7,8):
            sheet2.cell(row=i+rowstart, column=j).value =  "AS_Standard_Child"
    for i in range(0,len(SalaryDatatoCopy)):
        for j in range(8,9):
            sheet2.cell(row=i+rowstart, column=j).value =  SalaryDatatoCopy.iloc[i]["SalaryCosted"]
    for i in range(0,len(SalaryDatatoCopy)+len(ChinaHoldingDatatoFinal)+2):
        for j in range(18,19):
            sheet2.cell(row=i+rowstart, column=j).value =  MonthTemplateName + " " + str(yeartemplate) + " Salary Allocation"
    for i in range(0,len(SalaryDatatoCopy)):
        for j in range(21,22):
            sheet2.cell(row=i+rowstart, column=j).value =  "Salaries & Wages"
    for i in range(0,len(SalaryDatatoCopy)):
        for j in range(25,26):
            sheet2.cell(row=i+rowstart, column=j).value =  SalaryDatatoCopy.iloc[i]["Project Plan Task ID"]
    for i in range(0,len(SalaryDatatoCopy)):
        for j in range(29,30):
            sheet2.cell(row=i+rowstart, column=j).value =  SalaryDatatoCopy.iloc[i]["Initiative"]
    for i in range(0,len(SalaryDatatoCopy)):
        for j in range(31,32):
            sheet2.cell(row=i+rowstart, column=j).value =  SalaryDatatoCopy.iloc[i]["Cost Center"]
    for i in range(0,len(SalaryDatatoCopy)+len(ChinaHoldingDatatoFinal)+2):
        for j in range(33,34):
            sheet2.cell(row=i+rowstart, column=j).value =  "Unrestricted"
    for i in range(0,len(SalaryDatatoCopy)):
        for j in range(41,42):
            if SalaryDatatoCopy.iloc[i]["Location"] != "Beijing":
                pass
            else:
                sheet2.cell(row=i+rowstart, column=j).value =  SalaryDatatoCopy.iloc[i]["Location"]
    for i in range(0,len(SalaryDatatoCopy)):
        for j in range(43,44):
            sheet2.cell(row=i+rowstart, column=j).value =  SalaryDatatoCopy.iloc[i]["Employee ID"]
    for i in range(0,len(SalaryDatatoCopy)):
        for j in range(47,48):
            if SalaryDatatoCopy.iloc[i]["Lobbying"] == "None":
                pass
            else:
                sheet2.cell(row=i+rowstart, column=j).value =  SalaryDatatoCopy.iloc[i]["Lobbying"]            

    for i in range(0,len(SalaryDatatoCopy)):
        for j in range(49,50):
            if SalaryDatatoCopy.iloc[i]["Lobbying Location"] == "None":
                pass
            else:
                sheet2.cell(row=i+rowstart, column=j).value =  SalaryDatatoCopy.iloc[i]["Lobbying Location"]
# Copy Salary Holding Accounts below here:
    sheet2.cell(row=len(SalaryDatatoCopy)+rowstart, column=9).value = HoldingGentotal
    sheet2.cell(row=len(SalaryDatatoCopy)+rowstart+1, column=9).value = HoldingNotGenInsttotal                     
    sheet2.cell(row=len(SalaryDatatoCopy)+rowstart, column=21).value = "Salary Holding"
    sheet2.cell(row=len(SalaryDatatoCopy)+rowstart+1, column=21).value = "Salary Holding"
    sheet2.cell(row=len(SalaryDatatoCopy)+rowstart, column=25).value = "PROJECT_PLAN_TASK-3-5853"
    sheet2.cell(row=len(SalaryDatatoCopy)+rowstart+1, column=25).value = ""
    sheet2.cell(row=len(SalaryDatatoCopy)+rowstart, column=29).value = "GEN - General Admin"
    sheet2.cell(row=len(SalaryDatatoCopy)+rowstart+1, column=29).value = ""
    sheet2.cell(row=len(SalaryDatatoCopy)+rowstart, column=31).value = "Institute General"
    sheet2.cell(row=len(SalaryDatatoCopy)+rowstart+1, column=31).value = ""
    
# Copy China Holding Accounts below here:
    for i in range(0,len(ChinaHoldingDatatoFinal)):
        for j in range(9,10):
            sheet2.cell(row=len(SalaryDatatoCopy)+rowstart+2+i, column=j).value =  ChinaHoldingDatatoFinal.iloc[i]["Net Amount"]
    for i in range(0,len(ChinaHoldingDatatoFinal)):
        for j in range(43,44):
            sheet2.cell(row=len(SalaryDatatoCopy)+rowstart+2+i, column=j).value =  ChinaHoldingDatatoFinal.iloc[i]["Employee ID"]           
    for i in range(0,len(ChinaHoldingDatatoFinal)):
        for j in range(21,22):
            sheet2.cell(row=len(SalaryDatatoCopy)+rowstart+2+i, column=j).value =  "Salaries & Wages"          
    for i in range(0,len(ChinaHoldingDatatoFinal)):
        for j in range(25,26):
            sheet2.cell(row=len(SalaryDatatoCopy)+rowstart+2+i, column=j).value =  "PROJECT_PLAN_TASK-3-4027"   
    for i in range(0,len(ChinaHoldingDatatoFinal)):
        for j in range(29,30):
            sheet2.cell(row=len(SalaryDatatoCopy)+rowstart+2+i, column=j).value =  "CH - Admin"  
    for i in range(0,len(ChinaHoldingDatatoFinal)):
        for j in range(31,32):
            sheet2.cell(row=len(SalaryDatatoCopy)+rowstart+2+i, column=j).value =  "China"  
    for i in range(0,len(ChinaHoldingDatatoFinal)):
        for j in range(41,42):
            sheet2.cell(row=len(SalaryDatatoCopy)+rowstart+2+i, column=j).value =  "Beijing" 
    book.save('./Outputs/Workday_Accounting_Journal_Salary Allocation'+Type+timestampreport+'.xlsx')



In [657]:
createImportAccountingJournalSalary(SalaryLoadTemplate)




In [658]:
AllCostsAllocatedPTOAccruals.head()

,Employee ID,Total Cost,Worker,Project Plan Task,Project Plan Task ID,Initiative,Cost Center,ConsultingFeesCosted,Type,Project Hours Percentage,...,SalaryPercentage,BenefitsPercentage,SharedPercentage,BonusPercentage,SalaryCosted,BenefitsCosted,SharedCosted,BonusCosted,TaskDrawdowns,PTO more than Admint
0,000006,0.000000e+00,Shutong Lu | Lucy Lu,China > PTO,PROJECT_PLAN_TASK-3-5724,CH - Admin,China,0.0,PTO Time for UnderUtilized,0.952381,...,0.659662,0.149668,0.125623,0.065048,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN
1,000127,5.557860e+03,Jeannene Cruise Woolley | Frances Woolley,Accounting > PTO,PROJECT_PLAN_TASK-3-5772,ACT - Accounting,Org Finance,0.0,PTO Time for UnderUtilized,0,...,0.642454,0.224216,0.069085,0.064245,3.570669e+03,1.246162e+03,3.839622e+02,3.570667e+02,NaN,NaN
2,000322,1.856600e+02,James Sherwood,Carbon-Free Electricity > PTO,PROJECT_PLAN_TASK-3-5698,POW - Admin,Carbon-Free Electricity,0.0,PTO Time for UnderUtilized,0.992063,...,0.682356,0.138499,0.042674,0.136471,1.266862e+02,2.571372e+01,7.922804e+00,2.533723e+01,NaN,NaN
3,000378,5.684342e-14,Kevin Brehm,Carbon-Free Electricity > PTO,PROJECT_PLAN_TASK-3-5698,POW - Admin,Carbon-Free Electricity,0.0,PTO Time for UnderUtilized,0.979853,...,0.690039,0.184202,0.056756,0.069004,3.922415e-14,1.047067e-14,3.226180e-15,3.922413e-15,NaN,NaN
4,000394,1.764040e+03,Philip Keuhn | Phil Keuhn,Carbon-Free Buildings > PTO,PROJECT_PLAN_TASK-3-5718,BLD - Admin,Carbon-Free Buildings,0.0,PTO Time for UnderUtilized,0.833333,...,0.674430,0.145770,0.044914,0.134886,1.189722e+03,2.571436e+02,7.923003e+01,2.379442e+02,NaN,NaN


## Create Benefits Output File

In [659]:
BenefitsLoadTemplate = './Source_Files/Workday_Accounting_Journal_Benefits Allocation_Template.xlsx'

In [660]:
BenefitsHoldingDF = TransDetailBeijingDF[TransDetailBeijingDF['Ledger Account']=="5100:Employee Benefits"]
BenefitsHoldingDF1 = BenefitsHoldingDF[(BenefitsHoldingDF['Cost Center']=='China') | (BenefitsHoldingDF['Cost Center']=='Institute General') | (BenefitsHoldingDF['Cost Center']=='People Team')]
BenefitsHoldingDF1['Location'].fillna('NONE', inplace=True)

C:\Users\Jwendt\AppData\Local\anaconda3\envs\new_env_name\lib\site-packages\pandas\core\series.py:4530: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [661]:
BenefitsHoldingDFFINAL = BenefitsHoldingDF1.groupby(['Cost Center', 'Initiative', 'Location', 'Project Plan Task ID'])['Net Amount'].sum().reset_index(name='Net Amount')

In [662]:
BenefitsHoldingDFFINAL

,Cost Center,Initiative,Location,Project Plan Task ID,Net Amount
0,China,CH - Admin,Beijing,PROJECT_PLAN_TASK-3-2325,7160.48
1,China,CH - Admin,Beijing,PROJECT_PLAN_TASK-3-2332,3108.02
2,China,CH - Admin,Beijing,PROJECT_PLAN_TASK-3-4027,64511.54
3,China,CH - Admin,Beijing,PROJECT_PLAN_TASK-3-7347,0.00
4,Institute General,GEN - General Admin,NONE,PROJECT_PLAN_TASK-3-5189,19735.88
5,Institute General,GEN - General Admin,NONE,PROJECT_PLAN_TASK-3-5845,3717.07
6,Institute General,GEN - General Admin,NONE,PROJECT_PLAN_TASK-3-5847,3419.42
7,Institute General,GEN - General Admin,NONE,PROJECT_PLAN_TASK-3-5848,2732.40
8,Institute General,GEN - General Admin,NONE,PROJECT_PLAN_TASK-3-5849,1119.08
9,Institute General,GEN - General Admin,NONE,PROJECT_PLAN_TASK-3-5850,874.53


In [663]:
BenefitsHoldingDF1['Net Amount'].sum()

1443358.94

In [664]:
BenefitsCreditsDF = TransDetailBeijingDF[TransDetailBeijingDF['Ledger Account']=="5100:Employee Benefits"]
BenefitsCreditsDF1 = BenefitsCreditsDF[BenefitsCreditsDF['Cost Center']=='']


In [665]:
BenefitsHoldingDF["Cost Center Null"] = BenefitsHoldingDF['Cost Center'].isnull()
BenefitsHoldingDFblank = BenefitsHoldingDF[BenefitsHoldingDF["Cost Center Null"]==True]
BenefitsBlankTotal = BenefitsHoldingDFblank['Net Amount'].sum()

C:\Users\Jwendt\AppData\Local\Temp\ipykernel_20800\1006232518.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BenefitsHoldingDF["Cost Center Null"] = BenefitsHoldingDF['Cost Center'].isnull()


In [666]:
def createImportAccountingJournalBenefits(file):
    book = load_workbook(file)
    sheet = book["Import Accounting Journal"] #active means last opened sheet
    sheet['N6'] = TemplateDate1
    #ws.cell(row=2, column=2).value = 2
    sheet['T6'] = MonthTemplateName + " " + str(yeartemplate) + " Benefits Allocation"
    sheet2 = book["Journal Entry Line Replacement"]
#    for i in range(1,len(AllCostsAllocated[["SalaryCosted"]])):
    rowstart = 6

    for i in range(0,len(SalaryDatatoCopy)+len(BenefitsHoldingDFFINAL)+1):
        for j in range(2,3):
            sheet2.cell(row=i+rowstart, column=j).value =  1
    for i in range(0,len(SalaryDatatoCopy)+len(BenefitsHoldingDFFINAL)+1):
        for j in range(3,4):
            sheet2.cell(row=i+rowstart, column=j).value =  i+1
    for i in range(0,len(SalaryDatatoCopy)+len(BenefitsHoldingDFFINAL)+1):
        for j in range(6,7):
            sheet2.cell(row=i+rowstart, column=j).value =  5100
    for i in range(0,len(SalaryDatatoCopy)+len(BenefitsHoldingDFFINAL)+1):
        for j in range(7,8):
            sheet2.cell(row=i+rowstart, column=j).value =  "AS_Standard_Child"
    for i in range(0,len(SalaryDatatoCopy)):
        for j in range(8,9):
            sheet2.cell(row=i+rowstart, column=j).value =  SalaryDatatoCopy.iloc[i]["BenefitsCosted"]
    for i in range(0,len(SalaryDatatoCopy)+len(BenefitsHoldingDFFINAL)+1):
        for j in range(18,19):
            sheet2.cell(row=i+rowstart, column=j).value =  MonthTemplateName + " " + str(yeartemplate) + " Benefits Allocation"
    for i in range(0,len(SalaryDatatoCopy)+len(BenefitsHoldingDFFINAL)+1):
        for j in range(21,22):
            sheet2.cell(row=i+rowstart, column=j).value =  "Benefit Clearing Account"
    for i in range(0,len(SalaryDatatoCopy)):
        for j in range(25,26):
            sheet2.cell(row=i+rowstart, column=j).value =  SalaryDatatoCopy.iloc[i]["Project Plan Task ID"]
    for i in range(0,len(SalaryDatatoCopy)):
        for j in range(29,30):
            sheet2.cell(row=i+rowstart, column=j).value =  SalaryDatatoCopy.iloc[i]["Initiative"]
    for i in range(0,len(SalaryDatatoCopy)):
        for j in range(31,32):
            sheet2.cell(row=i+rowstart, column=j).value =  SalaryDatatoCopy.iloc[i]["Cost Center"]
    for i in range(0,len(SalaryDatatoCopy)+len(BenefitsHoldingDFFINAL)+1):
        for j in range(33,34):
            sheet2.cell(row=i+rowstart, column=j).value =  "Unrestricted"
    for i in range(0,len(SalaryDatatoCopy)):
        for j in range(41,42):
            if SalaryDatatoCopy.iloc[i]["Location"] != "Beijing":
                pass
            else:
                sheet2.cell(row=i+rowstart, column=j).value =  SalaryDatatoCopy.iloc[i]["Location"]
    for i in range(0,len(SalaryDatatoCopy)):
        for j in range(43,44):
            sheet2.cell(row=i+rowstart, column=j).value =  SalaryDatatoCopy.iloc[i]["Employee ID"]
    for i in range(0,len(SalaryDatatoCopy)):
        for j in range(47,48):
            if SalaryDatatoCopy.iloc[i]["Lobbying"] == "None":
                pass
            else:
                sheet2.cell(row=i+rowstart, column=j).value =  SalaryDatatoCopy.iloc[i]["Lobbying"]            

    for i in range(0,len(SalaryDatatoCopy)):
        for j in range(49,50):
            if SalaryDatatoCopy.iloc[i]["Lobbying Location"] == "None":
                pass
            else:
                sheet2.cell(row=i+rowstart, column=j).value =  SalaryDatatoCopy.iloc[i]["Lobbying Location"]
# Copy Benefits Holding Accounts below here:
    sheet2.cell(row=len(SalaryDatatoCopy)+rowstart, column=9).value = BenefitsBlankTotal                

    
# Copy Other Holding Accounts below here:
    for i in range(0,len(BenefitsHoldingDFFINAL)):
        for j in range(9,10):
            sheet2.cell(row=len(SalaryDatatoCopy)+rowstart+1+i, column=j).value =  BenefitsHoldingDFFINAL.iloc[i]["Net Amount"]             
    for i in range(0,len(BenefitsHoldingDFFINAL)):
        for j in range(25,26):
            sheet2.cell(row=len(SalaryDatatoCopy)+rowstart+1+i, column=j).value =  BenefitsHoldingDFFINAL.iloc[i]["Project Plan Task ID"]  
    for i in range(0,len(BenefitsHoldingDFFINAL)):
        for j in range(29,30):
            sheet2.cell(row=len(SalaryDatatoCopy)+rowstart+1+i, column=j).value =  BenefitsHoldingDFFINAL.iloc[i]["Initiative"]  
    for i in range(0,len(BenefitsHoldingDFFINAL)):
        for j in range(31,32):
            sheet2.cell(row=len(SalaryDatatoCopy)+rowstart+1+i, column=j).value =  BenefitsHoldingDFFINAL.iloc[i]["Cost Center"]  
    for i in range(0,len(BenefitsHoldingDFFINAL)):
        for j in range(41,42):
            if BenefitsHoldingDFFINAL.iloc[i]["Location"] == "NONE":
                pass
            else:
                sheet2.cell(row=len(SalaryDatatoCopy)+rowstart+1+i, column=j).value = BenefitsHoldingDFFINAL.iloc[i]["Location"]  
            

    book.save('./Outputs/Workday_Accounting_Journal_Benefits Allocation'+Type+timestampreport+'.xlsx')

In [667]:
createImportAccountingJournalBenefits(BenefitsLoadTemplate)

# Create Shared Output file

In [668]:
SharedLoadTemplate = './Source_Files/Workday_Accounting_Journal_Shared Costs Allocation_Template.xlsx'

In [669]:
SharedHoldingDF = TransDetailBeijingDF[(TransDetailBeijingDF['Ledger Account']=="7100:Depreciation Expenses") | (TransDetailBeijingDF['Ledger Account']=="8100:Shared Costs")]
SharedHoldingDF1 = SharedHoldingDF[(SharedHoldingDF['Initiative']=='CH - Admin') | (SharedHoldingDF['Initiative']=='GEN - General Admin')]
SharedHoldingDF1['Location'].fillna('NONE', inplace=True)

replacements = {'Boulder': 'NONE', 'Washington DC': 'NONE', 'New York': 'NONE', 'Oakland': 'NONE'}


SharedHoldingDF1['Location'] = SharedHoldingDF1['Location'].map(replacements).fillna(SharedHoldingDF1['Location'])

C:\Users\Jwendt\AppData\Local\anaconda3\envs\new_env_name\lib\site-packages\pandas\core\series.py:4530: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
C:\Users\Jwendt\AppData\Local\Temp\ipykernel_20800\582940983.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SharedHoldingDF1['Location'] = SharedHoldingDF1['Location'].map(replacements).fillna(SharedHoldingDF1['Location'])


In [670]:
SharedHoldingDFFINAL = SharedHoldingDF1.groupby(['Cost Center', 'Initiative', 'Location', 'Project Plan Task ID'])['Net Amount'].sum().reset_index(name='Net Amount')

In [671]:
SharedHoldingDFFINAL = SharedHoldingDFFINAL.drop(SharedHoldingDFFINAL[SharedHoldingDFFINAL['Net Amount'] < 0.1].index)

In [672]:
SharedHoldingDF['Net Amount'].sum()

750583.0800000001

In [673]:
def createImportAccountingJournalShared(file):
    book = load_workbook(file)
    sheet = book["Import Accounting Journal"] #active means last opened sheet
    sheet['N6'] = TemplateDate1
    #ws.cell(row=2, column=2).value = 2
    sheet['T6'] = MonthTemplateName + " " + str(yeartemplate) + " Shared Costs Allocation"
    sheet2 = book["Journal Entry Line Replacement"]
#    for i in range(1,len(AllCostsAllocated[["SalaryCosted"]])):
    rowstart = 6

    for i in range(0,len(SalaryDatatoCopy)+len(SharedHoldingDFFINAL)):
        for j in range(2,3):
            sheet2.cell(row=i+rowstart, column=j).value =  1
    for i in range(0,len(SalaryDatatoCopy)+len(SharedHoldingDFFINAL)):
        for j in range(3,4):
            sheet2.cell(row=i+rowstart, column=j).value =  i+1
    for i in range(0,len(SalaryDatatoCopy)+len(SharedHoldingDFFINAL)):
        for j in range(6,7):
            sheet2.cell(row=i+rowstart, column=j).value =  8100
    for i in range(0,len(SalaryDatatoCopy)+len(SharedHoldingDFFINAL)):
        for j in range(7,8):
            sheet2.cell(row=i+rowstart, column=j).value =  "AS_Standard_Child"
    for i in range(0,len(SalaryDatatoCopy)):
        for j in range(8,9):
            sheet2.cell(row=i+rowstart, column=j).value =  SalaryDatatoCopy.iloc[i]["SharedCosted"]
    for i in range(0,len(SalaryDatatoCopy)+len(SharedHoldingDFFINAL)):
        for j in range(18,19):
            sheet2.cell(row=i+rowstart, column=j).value =  MonthTemplateName + " " + str(yeartemplate) + " Shared Costs Allocation"
    for i in range(0,len(SalaryDatatoCopy)+len(SharedHoldingDFFINAL)):
        for j in range(21,22):
            sheet2.cell(row=i+rowstart, column=j).value =  "Shared Costs Clearing Account"
    for i in range(0,len(SalaryDatatoCopy)):
        for j in range(25,26):
            sheet2.cell(row=i+rowstart, column=j).value =  SalaryDatatoCopy.iloc[i]["Project Plan Task ID"]
    for i in range(0,len(SalaryDatatoCopy)):
        for j in range(29,30):
            sheet2.cell(row=i+rowstart, column=j).value =  SalaryDatatoCopy.iloc[i]["Initiative"]
    for i in range(0,len(SalaryDatatoCopy)):
        for j in range(31,32):
            sheet2.cell(row=i+rowstart, column=j).value =  SalaryDatatoCopy.iloc[i]["Cost Center"]
    for i in range(0,len(SalaryDatatoCopy)+len(SharedHoldingDFFINAL)):
        for j in range(33,34):
            sheet2.cell(row=i+rowstart, column=j).value =  "Unrestricted"
    for i in range(0,len(SalaryDatatoCopy)):
        for j in range(41,42):
            if SalaryDatatoCopy.iloc[i]["Location"] != "Beijing":
                pass
            else:
                sheet2.cell(row=i+rowstart, column=j).value =  SalaryDatatoCopy.iloc[i]["Location"]
    for i in range(0,len(SalaryDatatoCopy)):
        for j in range(43,44):
            sheet2.cell(row=i+rowstart, column=j).value =  SalaryDatatoCopy.iloc[i]["Employee ID"]
    for i in range(0,len(SalaryDatatoCopy)):
        for j in range(47,48):
            if SalaryDatatoCopy.iloc[i]["Lobbying"] == "None":
                pass
            else:
                sheet2.cell(row=i+rowstart, column=j).value =  SalaryDatatoCopy.iloc[i]["Lobbying"]            

    for i in range(0,len(SalaryDatatoCopy)):
        for j in range(49,50):
            if SalaryDatatoCopy.iloc[i]["Lobbying Location"] == "None":
                pass
            else:
                sheet2.cell(row=i+rowstart, column=j).value =  SalaryDatatoCopy.iloc[i]["Lobbying Location"]

# Copy Other Holding Accounts below here:
    for i in range(0,len(SharedHoldingDFFINAL)):
        for j in range(9,10):
            sheet2.cell(row=len(SalaryDatatoCopy)+rowstart+i, column=j).value =  SharedHoldingDFFINAL.iloc[i]["Net Amount"]             
    for i in range(0,len(SharedHoldingDFFINAL)):
        for j in range(25,26):
            sheet2.cell(row=len(SalaryDatatoCopy)+rowstart+i, column=j).value =  SharedHoldingDFFINAL.iloc[i]["Project Plan Task ID"]  
    for i in range(0,len(SharedHoldingDFFINAL)):
        for j in range(29,30):
            sheet2.cell(row=len(SalaryDatatoCopy)+rowstart+i, column=j).value =  SharedHoldingDFFINAL.iloc[i]["Initiative"]  
    for i in range(0,len(SharedHoldingDFFINAL)):
        for j in range(31,32):
            sheet2.cell(row=len(SalaryDatatoCopy)+rowstart+i, column=j).value =  SharedHoldingDFFINAL.iloc[i]["Cost Center"]  
    for i in range(0,len(SharedHoldingDFFINAL)):
        for j in range(41,42):
            if SharedHoldingDFFINAL.iloc[i]["Location"] == "NONE":
                pass
            if SharedHoldingDFFINAL.iloc[i]["Location"] != "Beijing":
                pass
            else:
                sheet2.cell(row=len(SalaryDatatoCopy)+rowstart+i, column=j).value = SharedHoldingDFFINAL.iloc[i]["Location"]  
            

    book.save('./Outputs/Workday_Accounting_Journal_Shared Costs Allocation'+Type+timestampreport+'.xlsx')

In [674]:
createImportAccountingJournalShared(SharedLoadTemplate)

# Create Bonus Output file

In [675]:
BonusLoadTemplate = './Source_Files/Workday_Accounting_Journal_Bonus Allocation_Template.xlsx'

In [676]:
SalaryDatatoCopy = AllCostsAllocatedPTOAccruals.loc[(AllCostsAllocatedPTOAccruals[["BonusCosted"]]!=0).any(axis=1)]

In [677]:
def createImportAccountingJournalBonus(file):
    book = load_workbook(file)
    sheet = book["Import Accounting Journal"] #active means last opened sheet
    sheet['N6'] = TemplateDate1
    #ws.cell(row=2, column=2).value = 2
    sheet['T6'] = MonthTemplateName + " " + str(yeartemplate) + " Bonus Allocation"
    sheet2 = book["Journal Entry Line Replacement"]
#    for i in range(1,len(AllCostsAllocated[["SalaryCosted"]])):
    rowstart = 6

    for i in range(0,len(SalaryDatatoCopy)+len(SharedHoldingDFFINAL)):
        for j in range(2,3):
            sheet2.cell(row=i+rowstart, column=j).value =  1
    for i in range(0,len(SalaryDatatoCopy)+len(SharedHoldingDFFINAL)):
        for j in range(3,4):
            sheet2.cell(row=i+rowstart, column=j).value =  i+1
    for i in range(0,len(SalaryDatatoCopy)+len(SharedHoldingDFFINAL)):
        for j in range(6,7):
            sheet2.cell(row=i+rowstart, column=j).value =  5000
    for i in range(0,len(SalaryDatatoCopy)+len(SharedHoldingDFFINAL)):
        for j in range(7,8):
            sheet2.cell(row=i+rowstart, column=j).value =  "AS_Standard_Child"
    for i in range(0,len(SalaryDatatoCopy)):
        for j in range(8,9):
            sheet2.cell(row=i+rowstart, column=j).value =  SalaryDatatoCopy.iloc[i]["BonusCosted"]
    for i in range(0,len(SalaryDatatoCopy)+len(SharedHoldingDFFINAL)):
        for j in range(18,19):
            sheet2.cell(row=i+rowstart, column=j).value =  MonthTemplateName + " " + str(yeartemplate) + " Bonus Allocation"
    for i in range(0,len(SalaryDatatoCopy)+len(SharedHoldingDFFINAL)):
        for j in range(21,22):
            sheet2.cell(row=i+rowstart, column=j).value =  "Bonus Accrual"
    for i in range(0,len(SalaryDatatoCopy)):
        for j in range(25,26):
            sheet2.cell(row=i+rowstart, column=j).value =  SalaryDatatoCopy.iloc[i]["Project Plan Task ID"]
    for i in range(0,len(SalaryDatatoCopy)):
        for j in range(29,30):
            sheet2.cell(row=i+rowstart, column=j).value =  SalaryDatatoCopy.iloc[i]["Initiative"]
    for i in range(0,len(SalaryDatatoCopy)):
        for j in range(31,32):
            sheet2.cell(row=i+rowstart, column=j).value =  SalaryDatatoCopy.iloc[i]["Cost Center"]
    for i in range(0,len(SalaryDatatoCopy)+len(SharedHoldingDFFINAL)):
        for j in range(33,34):
            sheet2.cell(row=i+rowstart, column=j).value =  "Unrestricted"
    for i in range(0,len(SalaryDatatoCopy)):
        for j in range(41,42):
            if SalaryDatatoCopy.iloc[i]["Location"] != "Beijing":
                pass
            else:
                sheet2.cell(row=i+rowstart, column=j).value =  SalaryDatatoCopy.iloc[i]["Location"]
    for i in range(0,len(SalaryDatatoCopy)):
        for j in range(43,44):
            sheet2.cell(row=i+rowstart, column=j).value =  SalaryDatatoCopy.iloc[i]["Employee ID"]
    for i in range(0,len(SalaryDatatoCopy)):
        for j in range(47,48):
            if SalaryDatatoCopy.iloc[i]["Lobbying"] == "None":
                pass
            else:
                sheet2.cell(row=i+rowstart, column=j).value =  SalaryDatatoCopy.iloc[i]["Lobbying"]            

    for i in range(0,len(SalaryDatatoCopy)):
        for j in range(49,50):
            if SalaryDatatoCopy.iloc[i]["Lobbying Location"] == "None":
                pass
            else:
                sheet2.cell(row=i+rowstart, column=j).value =  SalaryDatatoCopy.iloc[i]["Lobbying Location"]  
            

    book.save('./Outputs/Workday_Accounting_Journal_Bonus Allocation'+Type+timestampreport+'.xlsx')

In [678]:
createImportAccountingJournalBonus(BonusLoadTemplate)

# Create Table showing Dollars Admin and Project

In [679]:
AllCostsAllocatedPTOAccruals.columns

Index(['Employee ID', 'Total Cost', 'Worker', 'Project Plan Task',
       'Project Plan Task ID', 'Initiative', 'Cost Center',
       'ConsultingFeesCosted', 'Type', 'Project Hours Percentage',
       'Admin Hours Percentage', 'Lobbying', 'Lobbying Location',
       'Project or Admin', 'Project', 'Worker Cost Center',
       'PTO more than Admin', 'Location', 'Total Project Hours',
       'Needed Project Hours', 'Adjusted Hours', 'Actual Utilization Rate',
       'Standard Monthly Cost', 'Amount Banked per Hour', 'Amount Banked',
       'Cost per Project Hour', 'Utilization Target', 'Talent Pools', 'File',
       'Project PTO or Admin', 'Cost per PTO Hour', 'Project & PTO Hours',
       'Bank Month', 'SalaryPercentage', 'BenefitsPercentage',
       'SharedPercentage', 'BonusPercentage', 'SalaryCosted', 'BenefitsCosted',
       'SharedCosted', 'BonusCosted', 'TaskDrawdowns', 'PTO more than Admint'],
      dtype='object')

In [680]:
def CreateProjectvsAdminTable(program):
    firstdf = AllCostsAllocatedPTOAccruals[AllCostsAllocatedPTOAccruals['Cost Center']==program]  
    table = pd.pivot_table(firstdf, index = ['Cost Center','Worker', 'Talent Pools', 'Cost per Project Hour'], columns = ['Project PTO or Admin'], values = 'Total Cost', aggfunc="sum")
    return table

In [681]:
def add_subtotal_ProjectvsAdminTable(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename(('Total Cost', '', '', '', ''))) for k, d in PivotedGroup.groupby('Cost Center')])
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [682]:
def to_excel_ProjectvsAdminTable(df, file):
    writer = pd.ExcelWriter(file, engine='openpyxl')
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book

    df.to_excel(writer, sheet_name="Expenses by Program", startrow = 1)
    writer.save()
    writer.close()

In [683]:
def Format_ProjectvsAdminTable(file):
    book = load_workbook(file)
    sheet = book["Expenses by Program"] #active means last opened sheet
    rd = sheet.row_dimensions[1]
    rd.height = 25
    sheet['A2']= "Program"
    sheet['B2']= "Employee"
    sheet['C2']= "Talent Pool"
    sheet['D2']= "Cost per Project Hour"
    sheet['A2'].font = Font(bold=True)
    sheet['B2'].font = Font(bold=True)
    sheet['C2'].font = Font(bold=True)
    sheet['D2'].font = Font(bold=True)
#    fontStyle = Font(size = "20")
    sheet.merge_cells('A1:G1')
    top_left_cell = sheet['A1']
    top_right_cell = sheet['G1']
    top_left_cell.value = timestampreport + " Expense Summary"
    thin = Side(border_style="thin", color="000000")
    double = Side(border_style="double", color="000000")
    top_left_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    top_left_cell.fill = PatternFill("solid", fgColor="003366")
#    top_left_cell.fill = fill = GradientFill(stop=("000080", "FFFFFF"))
    top_left_cell.font  = Font(color="FFFFFF", size = "26")
#    top_left_cell.font = fontStyle
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    top_right_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    sheet.column_dimensions['A'].width = 24
    sheet.column_dimensions['B'].width = 24
    sheet.column_dimensions['C'].width = 24
    sheet.column_dimensions['D'].width = 13
    sheet.column_dimensions['E'].width = 16
    sheet.column_dimensions['F'].width = 16
    sheet.column_dimensions['G'].width = 16
    num_rows = sheet.max_row
    curr_row = 2
    last_col = 8
    bluesectionrightborder = 5
    for col in range(4,  last_col):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '"$"#,##0;[Red]("$"#,##0)'

    for col in range(1, bluesectionrightborder):
        for row in range(3, num_rows+1):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = blueFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Program":
            for j in range(1,last_col):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(top = thin, bottom = thin)  
                sheet.cell(column=j, row=i).fill = greyFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Total Cost":
            for j in range(1,last_col):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style="double"))  
      
    sheet.sheet_view.zoomScale = 100
    sheet.freeze_panes = "A3"
#    sheet.sheet_state = 'hidden'
    book.save(file)

In [684]:
def MakeProjectvsAdminTable(Group, link):
    step1 = CreateProjectvsAdminTable(Group)
    step2 = add_subtotal_ProjectvsAdminTable(step1)
    to_excel_ProjectvsAdminTable(step2, link)
    Format_ProjectvsAdminTable(link)

# CREATE PTO Bank Table

In [685]:
def CreatePTOTable(program):
    firstdf = AllCostsAllocatedPTOAccruals[AllCostsAllocatedPTOAccruals['Worker Cost Center']==program]  
    seconddf = firstdf[firstdf['Type']=='PTO Task Accrual']      
    table = pd.pivot_table(seconddf, index = ['Worker', 'Project', 'Project Plan Task'], values = 'Total Cost', aggfunc="sum")
    return table

In [686]:
def add_subtotal_PTOTable(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename(('PTO Bank Usage', '', '', ''))) for k, d in PivotedGroup.groupby('Worker')]).append((PivotedGroup.sum()).rename(('Grand Total','','','')))
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [687]:
def to_excel_PTOTable(df, file):
    writer = pd.ExcelWriter(file, engine='openpyxl')
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book

    df.to_excel(writer, sheet_name="PTO Bank Usage by Program", startrow = 1)
    writer.save()
    writer.close()

In [688]:
def Format_PTOTable(file):
    book = load_workbook(file)
    sheet = book["PTO Bank Usage by Program"] #active means last opened sheet
    rd = sheet.row_dimensions[1]
    rd.height = 25
    sheet['A2']= "Employee"
    sheet['B2']= "Project"
    sheet['C2']= "Task"
    sheet['A2'].font = Font(bold=True)
    sheet['B2'].font = Font(bold=True)
    sheet['C2'].font = Font(bold=True)
    sheet['D2'].font = Font(bold=True)
#    fontStyle = Font(size = "20")
    sheet.merge_cells('A1:D1')
    top_left_cell = sheet['A1']
    top_right_cell = sheet['D1']
    top_left_cell.value = timestampreport + " Expense Summary"
    thin = Side(border_style="thin", color="000000")
    double = Side(border_style="double", color="000000")
    top_left_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    top_left_cell.fill = PatternFill("solid", fgColor="003366")
#    top_left_cell.fill = fill = GradientFill(stop=("000080", "FFFFFF"))
    top_left_cell.font  = Font(color="FFFFFF", size = "26")
#    top_left_cell.font = fontStyle
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    top_right_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    sheet.column_dimensions['A'].width = 16
    sheet.column_dimensions['B'].width = 30
    sheet.column_dimensions['C'].width = 30
    sheet.column_dimensions['D'].width = 16
    num_rows = sheet.max_row
    curr_row = 2
    last_col = 5
    bluesectionrightborder = 4
    for col in range(4,  last_col):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '"$"#,##0;[Red]("$"#,##0)'

    for col in range(1, bluesectionrightborder):
        for row in range(3, num_rows+1):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = blueFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Employee":
            for j in range(1,last_col):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(top = thin, bottom = thin)  
                sheet.cell(column=j, row=i).fill = greyFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "PTO Bank Usage":
            for j in range(1,last_col):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style="double"))  
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Grand Total":
            for j in range(1,last_col):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style="double"))  
      
    sheet.sheet_view.zoomScale = 100
    sheet.freeze_panes = "A3"
#    sheet.sheet_state = 'hidden'
    book.save(file)

In [689]:
def MakePTOTable(Group, link):
    step1 = CreatePTOTable(Group)
    step2 = add_subtotal_PTOTable(step1)
    to_excel_PTOTable(step2, link)
    Format_PTOTable(link)

In [690]:
#MakePTOTable("Carbon-Free Buildings", str(Timepath+'/' + "Carbon-Free Buildings" + ' - ' +timestampreport + filename))

In [691]:
#TimeDataALL = './Source_Files/RMI_WorkerTimeBlocksforFinancialAllocationsDetailedforFinanceMASTER.xlsx'

Q1BankData = './Source_Files/PTOReclassQ1.xlsx'

Q1BankDataDF = pd.read_excel(Q1BankData, engine='openpyxl')

In [692]:
def CreatePTOQ1Table(program):
    firstdf = Q1BankDataDF[Q1BankDataDF['Cost Center']==program]      
    table = pd.pivot_table(firstdf, index = ['Employee Name', 'Project', 'Project Plan Task'], values = 'Total', aggfunc="sum")
    return table

In [693]:
def add_subtotal_PTOQ1Table(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename(('Q1 PTO Bank Usage', '', '', ''))) for k, d in PivotedGroup.groupby('Employee Name')]).append((PivotedGroup.sum()).rename(('Grand Total','','','')))
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [694]:
def to_excel_PTOQ1Table(df, file):
    writer = pd.ExcelWriter(file, engine='openpyxl')
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book

    df.to_excel(writer, sheet_name="Q1 PTO Bank Reclass by Program", startrow = 1)
    writer.save()
    writer.close()

In [695]:
def Format_PTOQ1Table(file):
    book = load_workbook(file)
    sheet = book["Q1 PTO Bank Reclass by Program"] #active means last opened sheet
    rd = sheet.row_dimensions[1]
    rd.height = 25
    sheet['A2']= "Employee"
    sheet['B2']= "Project"
    sheet['C2']= "Task"
    sheet['A2'].font = Font(bold=True)
    sheet['B2'].font = Font(bold=True)
    sheet['C2'].font = Font(bold=True)
    sheet['D2'].font = Font(bold=True)
#    fontStyle = Font(size = "20")
    sheet.merge_cells('A1:D1')
    top_left_cell = sheet['A1']
    top_right_cell = sheet['D1']
    top_left_cell.value = timestampreport + " Expense Summary"
    thin = Side(border_style="thin", color="000000")
    double = Side(border_style="double", color="000000")
    top_left_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    top_left_cell.fill = PatternFill("solid", fgColor="003366")
#    top_left_cell.fill = fill = GradientFill(stop=("000080", "FFFFFF"))
    top_left_cell.font  = Font(color="FFFFFF", size = "26")
#    top_left_cell.font = fontStyle
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    top_right_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    sheet.column_dimensions['A'].width = 16
    sheet.column_dimensions['B'].width = 30
    sheet.column_dimensions['C'].width = 30
    sheet.column_dimensions['D'].width = 16
    num_rows = sheet.max_row
    curr_row = 2
    last_col = 5
    bluesectionrightborder = 4
    for col in range(4,  last_col):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '"$"#,##0;[Red]("$"#,##0)'

    for col in range(1, bluesectionrightborder):
        for row in range(3, num_rows+1):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = blueFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Employee":
            for j in range(1,last_col):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(top = thin, bottom = thin)  
                sheet.cell(column=j, row=i).fill = greyFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Q1 PTO Bank Usage":
            for j in range(1,last_col):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style="double"))  
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Grand Total":
            for j in range(1,last_col):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style="double"))  
      
    sheet.sheet_view.zoomScale = 100
    sheet.freeze_panes = "A3"
#    sheet.sheet_state = 'hidden'
    book.save(file)

In [696]:
def MakePTOQ1Table(Group, link):
    step1 = CreatePTOQ1Table(Group)
    step2 = add_subtotal_PTOQ1Table(step1)
    to_excel_PTOQ1Table(step2, link)
    Format_PTOQ1Table(link)

# Create Total Expenses by Project and Task

In [697]:
def CreateTCTable(program):
    firstdf = AllCostsAllocatedPTOAccruals[AllCostsAllocatedPTOAccruals['Cost Center']==program]      
    table = pd.pivot_table(firstdf, index = ['Worker', 'Project', 'Project Plan Task'], values = 'Total Cost', aggfunc="sum")
    return table

In [698]:
def add_subtotal_TCTable(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename(('Total Cost', '', '', ''))) for k, d in PivotedGroup.groupby('Worker')])
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [699]:
def to_excel_TCTable(df, file):
    writer = pd.ExcelWriter(file, engine='openpyxl')
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book

    df.to_excel(writer, sheet_name="Employee Cost Breakdown", startrow = 1)
    writer.save()
    writer.close()

In [700]:
def Format_TCTable(file):
    book = load_workbook(file)
    sheet = book["Employee Cost Breakdown"] #active means last opened sheet
    rd = sheet.row_dimensions[1]
    rd.height = 25
    sheet['A2']= "Employee"
    sheet['B2']= "Project"
    sheet['C2']= "Task"
    sheet['A2'].font = Font(bold=True)
    sheet['B2'].font = Font(bold=True)
    sheet['C2'].font = Font(bold=True)
    sheet['D2'].font = Font(bold=True)
#    fontStyle = Font(size = "20")
    sheet.merge_cells('A1:D1')
    top_left_cell = sheet['A1']
    top_right_cell = sheet['D1']
    top_left_cell.value = timestampreport + " Expense Summary"
    thin = Side(border_style="thin", color="000000")
    double = Side(border_style="double", color="000000")
    top_left_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    top_left_cell.fill = PatternFill("solid", fgColor="003366")
#    top_left_cell.fill = fill = GradientFill(stop=("000080", "FFFFFF"))
    top_left_cell.font  = Font(color="FFFFFF", size = "26")
#    top_left_cell.font = fontStyle
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    top_right_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    sheet.column_dimensions['A'].width = 16
    sheet.column_dimensions['B'].width = 30
    sheet.column_dimensions['C'].width = 30
    sheet.column_dimensions['D'].width = 16
    num_rows = sheet.max_row
    curr_row = 2
    last_col = 5
    bluesectionrightborder = 4
    for col in range(4,  last_col):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '"$"#,##0;[Red]("$"#,##0)'

    for col in range(1, bluesectionrightborder):
        for row in range(3, num_rows+1):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = blueFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Employee":
            for j in range(1,last_col):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(top = thin, bottom = thin)  
                sheet.cell(column=j, row=i).fill = greyFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Total Cost":
            for j in range(1,last_col):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style="double"))  

    sheet.sheet_view.zoomScale = 100
    sheet.freeze_panes = "A3"
#    sheet.sheet_state = 'hidden'
    book.save(file)

In [701]:
def MakeTCTable(Group, link):
    step1 = CreateTCTable(Group)
    step2 = add_subtotal_TCTable(step1)
    to_excel_TCTable(step2, link)
    Format_TCTable(link)

# Create ALL ADMIN TABLE

In [702]:
MonthAdminTable['Total PTO Cost'].fillna(0, inplace=True)
MonthAdminTable['Total Admin Cost'].fillna(0, inplace=True)

In [703]:
MonthAdminTable

,Worker,Employee ID,Worker Cost Center,Talent Pool,Project Group,Utilization Target,Hire Date,Termination Date,Title,Reported Date,...,Week,Time Cost Center,Time Initiative,MonthYear,Project or Admin,Employee Hours,Percent of Total Monthly Hours,Total Admin Cost,Total PTO Cost,Home Cost Center
0,Aaron Brickman,1436,US,Talent Pool: Sr. Principal,Holiday,0.6,2022-02-28,NaT,Senior Principal,2024-09-02,...,09/02/2024 - 09/08/2024,NaN,NaN,9/2024,Admin,168.0,0.047619,0.00,0.00,US
1,Aaron Brickman,1436,US,Talent Pool: Sr. Principal,Admin,0.6,2022-02-28,NaT,Senior Principal,2024-09-03,...,09/02/2024 - 09/08/2024,US,US - Admin,9/2024,Admin,168.0,0.011905,0.00,0.00,US
2,Aaron Brickman,1436,US,Talent Pool: Sr. Principal,Admin,0.6,2022-02-28,NaT,Senior Principal,2024-09-06,...,09/02/2024 - 09/08/2024,US,US - Admin,9/2024,Admin,168.0,0.005952,0.00,0.00,US
3,Aaron Brickman,1436,US,Talent Pool: Sr. Principal,Admin,0.6,2022-02-28,NaT,Senior Principal,2024-09-04,...,09/02/2024 - 09/08/2024,US,US - Admin,9/2024,Admin,168.0,0.011905,0.00,0.00,US
4,Aaron Brickman,1436,US,Talent Pool: Sr. Principal,Admin,0.6,2022-02-28,NaT,Senior Principal,2024-09-06,...,09/02/2024 - 09/08/2024,US,US - Admin,9/2024,Admin,168.0,0.005952,0.00,0.00,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7792,Zsaria Diaz,1327,Islands,Talent Pool: Sr. Associate,Admin,0.7,2021-09-13,NaT,Senior Associate,2024-09-23,...,09/23/2024 - 09/29/2024,Islands,IS - Admin,9/2024,Admin,176.0,0.011364,2189.59,850.33,Islands
7793,Zsaria Diaz,1327,Islands,Talent Pool: Sr. Associate,Admin,0.7,2021-09-13,NaT,Senior Associate,2024-09-25,...,09/23/2024 - 09/29/2024,Islands,IS - Admin,9/2024,Admin,176.0,0.002841,2189.59,850.33,Islands
7794,Zsaria Diaz,1327,Islands,Talent Pool: Sr. Associate,Admin,0.7,2021-09-13,NaT,Senior Associate,2024-09-26,...,09/23/2024 - 09/29/2024,Islands,IS - Admin,9/2024,Admin,176.0,0.014205,2189.59,850.33,Islands
7795,Zsaria Diaz,1327,Islands,Talent Pool: Sr. Associate,Admin,0.7,2021-09-13,NaT,Senior Associate,2024-09-30,...,09/30/2024 - 10/06/2024,Islands,IS - Admin,9/2024,Admin,176.0,0.034091,2189.59,850.33,Islands


In [704]:
def CreateAdminTable(program):
    seconddf = MonthAdminTable[MonthAdminTable['Home Cost Center']==program]  
    table = pd.pivot_table(seconddf, index = ['Worker','Total Admin Cost','Total PTO Cost','Project','Project Plan Task'], values = 'Reported Quantity', aggfunc="sum")
    out = pd.concat([d.append(d.sum().rename(('', '', '', '', 'Total Admin & PTO Hours'))) for k, d in table.groupby('Worker')])
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [705]:
def to_excel_AdminTable(df, file):
    writer = pd.ExcelWriter(file, engine='openpyxl')
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book

    df.to_excel(writer, sheet_name="Admin by Program", startrow = 1)
    writer.save()
    writer.close()

In [706]:
def Format_AdminTable(file):
    book = load_workbook(file)
    sheet = book["Admin by Program"] #active means last opened sheet
    rd = sheet.row_dimensions[1]
    rd.height = 25
    sheet['A2']= "Worker"
    sheet['B2']= "Total Admin Cost"
    sheet['C2']= "Total PTO Cost"
    sheet['D2']= "Project"
    sheet['E2']= "Project Plan Task"
    sheet['F2']= "Admin Hours"
    sheet['A2'].font = Font(bold=True)
    sheet['B2'].font = Font(bold=True)
    sheet['C2'].font = Font(bold=True)
    sheet['D2'].font = Font(bold=True)
#    fontStyle = Font(size = "20")
    sheet.merge_cells('A1:F1')
    top_left_cell = sheet['A1']
    top_right_cell = sheet['F1']
    top_left_cell.value = timestampreport + " Admin & PTO Summary"
    thin = Side(border_style="thin", color="000000")
    double = Side(border_style="double", color="000000")
    top_left_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    top_left_cell.fill = PatternFill("solid", fgColor="003366")
#    top_left_cell.fill = fill = GradientFill(stop=("000080", "FFFFFF"))
    top_left_cell.font  = Font(color="FFFFFF", size = "26")
#    top_left_cell.font = fontStyle
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    top_right_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    sheet.column_dimensions['A'].width = 24
    sheet.column_dimensions['B'].width = 14
    sheet.column_dimensions['C'].width = 24
    sheet.column_dimensions['D'].width = 62
    sheet.column_dimensions['E'].width = 32
    num_rows = sheet.max_row
    curr_row = 2
    last_col = 7
    bluesectionrightborder = 6
    for col in range(2, 4):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '"$"#,##0;[Red]("$"#,##0)'

    for col in range(1, bluesectionrightborder):
        for row in range(3, num_rows+1):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = blueFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Worker":
            for j in range(1,last_col):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(top = thin, bottom = thin)  
                sheet.cell(column=j, row=i).fill = greyFill
    for col in range(1, bluesectionrightborder):
        for row in range(3, num_rows+1):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal='left', vertical='center')
            sheet.cell(column=col, row=row).font = Font(bold=True, size = "12") 
    for col in range(1, 2):
        greystartrows = []
        greyendrows = []
        for i in range(3,num_rows+1):
            if sheet.cell(row=i, column=col).value:
                greystartrows.append(i)
        del  greystartrows[1::2]
        for i in range(3,num_rows+1):
            if sheet.cell(row=i, column=col).value:
                greyendrows.append(i-1)
        del  greyendrows[0::2]
    if len(greystartrows) != len(greyendrows):
        print('mismatch')
        print(len(greystartrows))
        print(len(greyendrows))
        greystartrows.pop()       
    for col in range(1, bluesectionrightborder):
        for row in range(3, num_rows):
            for i in range(len(greystartrows)):
                if row == greystartrows[i]:
                    for row in range(greystartrows[i],greyendrows[i]):
                        sheet.cell(column=col, row=row).fill = lightblueFill 
                try:
                    if sheet.cell(row=greyendrows[i], column=bluesectionrightborder-1).value == "Total Admin & PTO Hours":
                        sheet.cell(column=bluesectionrightborder-1, row=greyendrows[i]).fill = lightblueFill
                except:
                    pass
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=5).value == "Total Admin & PTO Hours":
            for j in range(bluesectionrightborder-1,last_col):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style="double"))  
      
    sheet.sheet_view.zoomScale = 100
    sheet.freeze_panes = "A3"
#    sheet.sheet_state = 'hidden'
    book.save(file)

In [707]:
def MakeAdminTable(Group, link):
    step1 = CreateAdminTable(Group)
    to_excel_AdminTable(step1, link)
    Format_AdminTable(link)

# Create Project Time Master

In [708]:
def CreateProjectTimePivotMASTER(Group):
#    Month_order = ['Jul 2022', 'Aug 2022', 'Sep 2022', 'Oct 2022', 'Nov 2022', 'Dec 2022', 'Jan 2023', 'Feb 2023', 'Mar 2023', 'Apr 2023', 'May 2023', 'Jun 2023', 'Jul 2023', 'Aug 2023', 'Sep 2023', 'Oct 2023', 'Nov 2023', 'Dec 2023', 'Jan 2024', 'Feb 2024', 'Mar 2024', 'Apr 2024', 'May 2024', 'Jun 2024']
    table = pd.pivot_table(Group, index = ['Worker Cost Center', 'Worker', 'Fully Utilized', 'Actual Utilization Rate', 'Utilization Target', 'Total Project Hours', 'Needed Project Hours', 'Talent Pool', 'Project'], values = 'Total Employee Project Hours', aggfunc="sum")
#    table2 = table.reindex(Month_order, axis=1)
    return table

In [709]:
def add_subtotal_ProjectPivotMASTER(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename(('','','','','','','','','Total Hours'))) for k, d in PivotedGroup.groupby(level=[0,1], sort = False)])
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [710]:
def to_excel_ProjectTimeMASTER(df, file):
    writer = pd.ExcelWriter(file, engine='openpyxl')
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book

    df.to_excel(writer, sheet_name="Utilization by Worker", startrow = 1)
    writer.save()
    writer.close()

In [711]:
def Format_ProjectTimeMASTER(file):
    book = load_workbook(file)
    sheet = book["Utilization by Worker"] #active means last opened sheet
    rd = sheet.row_dimensions[1]
    rd.height = 25
    sheet['A2']= "Program"
    sheet['B2']= "Employee"
    sheet['C2']= 'Utilization Status'
    sheet['D2']= 'Actual Utilization'
    sheet['E2']= 'Utilization Target'
    sheet['F2']= 'Total Project Hours'
    sheet['G2']= 'Total Needed Hours'
    sheet['H2']= 'Talent Pool'
    sheet['I2']= 'Project'
    sheet['A2'].font = Font(bold=True)
    sheet['B2'].font = Font(bold=True)
#    fontStyle = Font(size = "20")
    sheet.merge_cells('A1:J1')
    top_left_cell = sheet['A1']
    top_right_cell = sheet['J1']
    top_left_cell.value = timestampreport + " Utilization Summary"
    thin = Side(border_style="thin", color="000000")
    double = Side(border_style="double", color="000000")
    top_left_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    top_left_cell.fill = PatternFill("solid", fgColor="003366")
#    top_left_cell.fill = fill = GradientFill(stop=("000080", "FFFFFF"))
    top_left_cell.font  = Font(color="FFFFFF", size = "26")
#    top_left_cell.font = fontStyle
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    top_right_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    sheet.column_dimensions['A'].width = 12
    sheet.column_dimensions['B'].width = 20
    sheet.column_dimensions['C'].width = 16
    sheet.column_dimensions['D'].width = 16
    sheet.column_dimensions['E'].width = 16
    sheet.column_dimensions['F'].width = 16
    sheet.column_dimensions['G'].width = 16
    sheet.column_dimensions['I'].width = 55
    sheet.column_dimensions['H'].width = 16
    num_rows = sheet.max_row
    curr_row = 2
    last_col = 12
    bluesectionrightborder = 9
    for col in range(4, 6):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '0.00%'
    for col in range(1, bluesectionrightborder):
        for row in range(3, num_rows+1):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal='left', vertical='center')
            sheet.cell(column=col, row=row).font = Font(bold=True, size = "12")

    for col in range(1, bluesectionrightborder):
        for row in range(3, num_rows+1):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = blueFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Program":
            for j in range(1,last_col-1):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(top = thin, bottom = thin)  
                sheet.cell(column=j, row=i).fill = greyFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=9).value == "Total Hours":
            for j in range(1,last_col-1):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style="double"))  
    for col in range(1, 2):
        greystartrows = []
        greyendrows = []
        for i in range(3,num_rows+1):
            if sheet.cell(row=i, column=col).value:
                greystartrows.append(i)
        del  greystartrows[1::2]
        for i in range(3,num_rows+1):
            if sheet.cell(row=i, column=col).value:
                greyendrows.append(i-1)
        del  greyendrows[0::2]

        
    if len(greystartrows) != len(greyendrows):
        print('mismatch')
        print(len(greystartrows))
        print(len(greyendrows))
        greystartrows.pop()
    for col in range(1, bluesectionrightborder):
        for row in range(3, num_rows):
            for i in range(len(greystartrows)):
                if row == greystartrows[i]:
                    for row in range(greystartrows[i],greyendrows[i]):
                        sheet.cell(column=col, row=row).fill = lightblueFill 
                try:
                    if sheet.cell(row=greyendrows[i], column=bluesectionrightborder).value == "Total Hours":
                        sheet.cell(column=bluesectionrightborder-1, row=greyendrows[i]).fill = lightblueFill
                except:
                    pass
      
    for col in range(1, bluesectionrightborder):
        for row in range(3, num_rows+1):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal='left', vertical='center')
            sheet.cell(column=col, row=row).font = Font(bold=True, size = "12")    
    sheet.sheet_view.zoomScale = 100
    sheet.freeze_panes = "H3"
#    sheet.sheet_state = 'hidden'
    book.save(file)


In [712]:
def MakeProjectTimeMASTER(link):
    step2 = CreateProjectTimePivotMASTER(EmployeeTimeProjectLookupTable)
    step3 = add_subtotal_ProjectPivotMASTER(step2)
    to_excel_ProjectTimeMASTER(step3, link)
    Format_ProjectTimeMASTER(link)


# Add ALL COSTS ADMIN AND PROJECT FOR RMI

In [713]:


AdminCostsTable = pd.pivot_table(AllCostsAllocatedPTOAccruals, index = ['Cost Center','Worker'], columns = ['Project PTO or Admin'], values = 'Total Cost', aggfunc="sum")


In [714]:
def add_subtotal_Expenses(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename(('Total Cost', ''))) for k, d in PivotedGroup.groupby('Cost Center')])
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [715]:
def to_excel_ProjectExpenses(df, file):
    writer = pd.ExcelWriter(file, engine='openpyxl')
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book

    df.to_excel(writer, sheet_name="Expenses by Program", startrow = 1)
    writer.save()
    writer.close()

In [716]:
def Format_ProjectExpenses(file):
    book = load_workbook(file)
    sheet = book["Expenses by Program"] #active means last opened sheet
    rd = sheet.row_dimensions[1]
    rd.height = 25
    sheet['A2']= "Program"
    sheet['B2']= "Employee"
    sheet['A2'].font = Font(bold=True)
    sheet['B2'].font = Font(bold=True)
#    fontStyle = Font(size = "20")
    sheet.merge_cells('A1:E1')
    top_left_cell = sheet['A1']
    top_right_cell = sheet['E1']
    top_left_cell.value = timestampreport + " Expense Summary"
    thin = Side(border_style="thin", color="000000")
    double = Side(border_style="double", color="000000")
    top_left_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    top_left_cell.fill = PatternFill("solid", fgColor="003366")
#    top_left_cell.fill = fill = GradientFill(stop=("000080", "FFFFFF"))
    top_left_cell.font  = Font(color="FFFFFF", size = "26")
#    top_left_cell.font = fontStyle
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    top_right_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    sheet.column_dimensions['A'].width = 24
    sheet.column_dimensions['B'].width = 24
    sheet.column_dimensions['C'].width = 13
    sheet.column_dimensions['D'].width = 16
    sheet.column_dimensions['E'].width = 16
    sheet.column_dimensions['F'].width = 16
    num_rows = sheet.max_row
    curr_row = 2
    last_col = 6
    bluesectionrightborder = 3
    for col in range(2, last_col):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '"$"#,##0;[Red]("$"#,##0)'

    for col in range(1, bluesectionrightborder):
        for row in range(3, num_rows+1):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = blueFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Program":
            for j in range(1,last_col):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(top = thin, bottom = thin)  
                sheet.cell(column=j, row=i).fill = greyFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Total Cost":
            for j in range(1,last_col):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style="double"))  
      
    sheet.sheet_view.zoomScale = 100
    sheet.freeze_panes = "B3"
#    sheet.sheet_state = 'hidden'
    book.save(file)

In [717]:
def MakeProjectExpensesMASTER(link):
    step3 = add_subtotal_Expenses(AdminCostsTable)
    to_excel_ProjectExpenses(step3, link)
    Format_ProjectExpenses(link)

In [718]:
timestampreport

'Sep-2024'

# Make ALL PROGRAM REPORTS

In [719]:

for i in Programs:
    try:
        MakeMISSINGTIME(i, str(Timepath+'/' + i  + ' - ' +timestampreport + filename))
    except:
        pass

for i in Programs:
    try:
        MakeTIMEREPORT(i, str(Timepath+'/' + i + ' - ' +timestampreport + filename))
    except:
        pass

for i in Programs:
    try:
        MakeProjectTime(i, str(Timepath+'/' + i + ' - ' +timestampreport + filename))
    except:
        pass


for i in Programs:
    try:
        MakeProjectvsAdminTable(i, str(Timepath+'/' + i + ' - ' +timestampreport + filename))
    except:
        pass

for i in Programs:
    try:
        MakeAdminTable(i, str(Timepath+'/' + i + ' - ' +timestampreport + filename))
    except:
        pass

for i in Programs:
    try:
        MakeFundingSourceRules(i, str(Timepath+'/' + i + ' - ' +timestampreport + filename))
    except:
        pass
    
for i in Programs:
    try:
        MakePTOTable(i, str(Timepath+'/' + i + ' - ' +timestampreport + filename))
    except:
        pass



for i in Programs:
    try:
        MakeTCTable(i, str(Timepath+'/' + i + ' - ' +timestampreport + filename))
    except:
        pass


{30, 7}
{64, 34, 12, 44, 14, 47, 18, 21, 62, 31}
{22}
{97, 73, 52, 22, 87, 88, 59, 30}
{41, 34, 38, 47}
{27, 18, 3}
{13, 5, 14, 15}
{8, 11, 15, 19, 26, 27}
{6, 14, 16, 19, 24}
{9, 12, 36}
{32, 33, 6, 7, 10, 19, 20, 24, 26, 29, 30}
set()
{4}
{25, 10, 44}
set()
set()
{17, 19, 6}
{6}
{25, 26, 11, 6}
set()
{9, 10, 12, 19, 31}
{12, 22}
{6, 9, 10, 11, 16, 21, 23}
{6}
{3, 4}
mismatch
36
35
mismatch
48
47
mismatch
8
7
mismatch
7
6
mismatch
28
27
mismatch
5
4
mismatch
4
3
mismatch
17
16
mismatch
10
9
mismatch
19
18
mismatch
38
37
mismatch
25
24
mismatch
8
7
mismatch
17
16
mismatch
3
2
mismatch
5
4
mismatch
11
10
mismatch
2
1
mismatch
17
16
mismatch
5
4


In [720]:
#for i in Programs:
#    try:
#        MakePTOQ1Table(i, str(Timepath+'/' + i + ' - ' +timestampreport + filename))
#    except:
#        pass

# MAKE RMI REPORT

In [721]:
#MakeTIMEREPORTMASTER(RockyMtnDF,str(Timepath+'/' + 'Rocky Mountain Institute'+' - ' + timestampreport + filename))
#MakeProjectTimeMASTER(str(Timepath+'/' + 'Rocky Mountain Institute'+' - ' + timestampreport + filename))
#MakeProjectExpensesMASTER(str(Timepath+'/' + 'Rocky Mountain Institute'+' - ' + timestampreport + filename))

# TIE OUT TOTALS

In [722]:
len(UnderUtilizedProgramTime)

5012

In [723]:
len(FullyUtilizedProgramTime)

2540

Review Totals

In [724]:
ALLSALARIES = RMISalariesDF['Net Amount'].sum() + GPSalariesDF['Net Amount'].sum() + BeijingSalariesTotal
ALLSALARIES

6723590.240000001

In [725]:
RMISalariesDF['Net Amount'].sum() + GPSalariesDF['Net Amount'].sum()

6432660.870000001

In [726]:
RMISalariesDF['Net Amount'].sum()

5723404.880000001

In [727]:
GPSalariesDF['Net Amount'].sum()

709255.99

In [728]:
BeijingSalariesTotal

290929.37000000005

In [729]:
ALLSALARIES

6723590.240000001

In [730]:
ALLBENEFITS = GP_RMI_BenefitsTotal + BeijingBenefitsTotal

In [731]:
GP_RMI_BenefitsTotal

2183773.7800000003

In [732]:
BeijingBenefitsTotal

75842.97000000002

In [733]:
ALLBENEFITS

2259616.7500000005

In [734]:
ALLSHARED = GP_RMI_SharedTotal + BeijingSharedTotal

In [735]:
GP_RMI_SharedTotal

672855.34

In [736]:
BeijingSharedTotal

63658.00000000001

In [737]:
ALLSHARED

736513.34

In [738]:
AllSalariesFinal['Bonus Final'].sum()



781074.5242969724

In [739]:
Allcosts = AllSalariesFinal['Bonus Final'].sum() + ALLSHARED.sum() + ALLBENEFITS.sum() + ALLSALARIES.sum()
Allcosts

10500794.854296975

In [740]:
AllSalariesFinal['Consulting Per Employee'].sum()

282181.80999999994

Review Headcounts

In [741]:
len(RMISalariesDF)

602

In [742]:
GPHeadcount

72

In [743]:
BeijingEmployeeCount

33

In [744]:
BeijingEmployeeCount

33